In [1]:
#PigLab5NoteBook
import pandas as pd

! pip install pandas
! pip install openpyxl
! pip install rdflib
! pip install pyspark
import rdflib
import hashlib
import numpy as np
import urllib.parse
from rdflib import Literal, Namespace, RDF, URIRef, Graph, RDFS, OWL
from rdflib.namespace import FOAF, XSD
from rdflib.plugins.sparql import prepareQuery
from pyspark.sql.functions import when, col, lit

## First data upload

In [2]:
# Define the file paths
files = ['Lab5/Decide-Exopol.xlsx']

# Load the data into Pandas dataframes
dfs = []
for file in files:
    df = pd.read_excel(file, engine='openpyxl')
    dfs.append(df)

df = dfs[0]


In [3]:
df = df.drop(['Procedencia provincia'], axis=1)
df = df.drop(['Categoría animal'], axis=1)
df = df.drop(['Pool o individual'], axis=1)

df.head(20)

,ID caso,Fecha,Provincia,Edad,Tipo de muestra,PCR,Resultado
0,1,2020-01-02,Castellón,Destetados,Pulmón,Influenza A,Neg.
1,1,2020-01-02,Castellón,Destetados,Pulmón,Myc. hyopneumoniae,Neg.
2,2,2020-01-02,Castellón,Cebo,Pulmón,Influenza A,Neg.
3,2,2020-01-02,Castellón,Cebo,Pulmón,Myc. hyopneumoniae,Positivo
4,3,2020-01-02,Huesca,ND,Pulmón,Influenza A,Neg.
5,3,2020-01-02,Huesca,ND,Pulmón,Myc. hyopneumoniae,Positivo
6,4,2020-01-02,Huesca,Cebo,Pulmón,Influenza A,Neg.
7,4,2020-01-02,Huesca,Cebo,Pulmón,Myc. hyopneumoniae,Neg.
8,5,2020-01-02,Huesca,Cebo,Pulmón,Influenza A,Neg.
9,5,2020-01-02,Huesca,Cebo,Pulmón,Myc. hyopneumoniae,Neg.


In [4]:
# Errors arised with the coding of the Age 
import unicodedata

def clean_text(text):
    if pd.isna(text):
        return None
    text = str(text).strip().lower()
    text = unicodedata.normalize('NFKD', text).encode('ASCII', 'ignore').decode('utf-8')
    return text


# Create a column with the clean text
df['Edad'] = df['Edad'].apply(clean_text)
df['Provincia'] = df['Provincia'].apply(clean_text)
df['Resultado'] = df['Resultado'].apply(clean_text)

print(df['Provincia'].unique())

['castellon' 'huesca' 'zaragoza' 'barcelona' 'albacete' 'badajoz'
 'sevilla' 'caceres' 'teruel' 'murcia' 'salamanca' 'girona' 'navarra'
 'toledo' 'lleida' 'zamora' 'avila' 'segovia' 'valencia' 'cordoba' 'lugo'
 'huelva' 'malaga' 'balears, illes' 'gipuzkoa' 'granada' 'leon'
 'valladolid' 'almeria' 'jaen' 'tarragona' 'soria' 'pontevedra' 'madrid'
 'palmas, las' 'nd' 'alicante' 'cuenca' 'ciudad real'
 'santa cruz de tenerife' 'rioja, la' 'coruna, a' 'ourense' 'cadiz'
 'palencia' 'burgos' 'alava' 'a coruna' 'asturias' 'bizkaia' 'guadalajara']


In [5]:
#Rename Colums

df = df.rename(columns={
    'ID caso': 'SampleNumber',
    'Provincia': 'Province',
    'Fecha': 'Date',
    'PCR': 'Pathogen',
    'Resultado': 'Result',
    'Tipo de muestra': 'SampleType',
    'Edad': 'Age', #Age of animal
})

# Set DiagnosticTest to "PCR" Country to "Spain", Breed to "Meat", and LabReference to "1".
df['DiagnosticTest'] = 'PCR'
df['Country'] = 'Spain'
df['Breed'] = 'Meat'
df["Lab_Reference"] = "5"


In [6]:
#remove ' symbol
df['Age'] = df['Age'].str.replace("'", ' ', regex=False).str.strip()
df['Age'] = df['Age'].str.replace('"', ' ', regex=False).str.strip()
#print(df['Age'].unique())

In [7]:
print(df['Result'].unique())

['neg.' 'positivo']


In [8]:
#Annotation Mapping

# Add Result mapping for NEG = 0 and POS =1
resultMapping = {
    "positivo": "1",
    "neg.": "0",
  }
df["Result"] = df["Result"].map(resultMapping).fillna("Missing")

PathogenMapping = {

    "Lawsonia intracellularis": "Lawsonia intracellularis",
    "L. intracellularis": "Lawsonia intracellularis",
    "L.intracelullaris": "Lawsonia intracellularis",
    "M. hyopneumoniae": "Mycoplasma hyopneumoniae",
    "Myc. hyopneumoniae": "Mycoplasma hyopneumoniae",
    "Mycoplasma hyopneumoniae antistoffen" : "Mycoplasma hyopneumoniae",
    "Mycoplasma hyopneumoniae" : "Mycoplasma hyopneumoniae",
    "M. hyopneumoniae" : "Mycoplasma hyopneumoniae",
    "SIV" : "Swine influenza virus",
    "Influenza A" : "Swine influenza virus",
  
}

df["Pathogen"] = df["Pathogen"].map(PathogenMapping).fillna("Missing")

# SampleTypeMapping for the additional sample types
SampleTypeMapping = {
    'Pulmón': 'Tissue',
    'Fluido oral': 'Fluid',
    'Hisopo': 'Swab',
    'Raspado': 'Swab',
    'Lavado': 'Other',
    'Cordón umbilical': 'Tissue',
    'Digestivo': 'Tissue',
    'Suero': 'Blood',
    'Coriolis': 'Other',
    'Muestra ambiental': 'Other',
    'Heces': 'Faeces',
    'Linfonodo': 'Tissue',
    'Lengua': 'Tissue',
    'Pulmón, Corazón': 'Mixed',
    'Pulmón, Bazo': 'Mixed',
    'Órganos feto': 'Tissue',
    'Pulmón, Linfonodo': 'Mixed',
    'Hisopo, Líquido pericárdico': 'Mixed',
    'Pulmón, Líquido pericárdico': 'Mixed',
    'Hígado': 'Tissue',
    'Pulmón, Hisopo': 'Mixed',
    'Pulmón, Corazón, Líquido pericárdico': 'Mixed',
    'Calzas': 'Other',
    'Heces, Digestivo': 'Mixed',
    'Intestino Delgado': 'Tissue',
    'Válvula Ileocecal': 'Tissue',
    'Pulmón, Ganglio nervioso': 'Mixed',
    'Toallita': 'Other',
    'Articulación': 'Tissue',
    'Pulmón, Hígado': 'Mixed',
    'Pulmón, Tráquea, Tonsila': 'Mixed',
    'Pulmón, Tráquea': 'Mixed',
#N/D
    'FTA': 'Other',
    'Feto': 'Tissue',
    'Gánglio, Pulmón': 'Mixed',
    'Intestino Delgado, Intestino Grueso': 'Mixed',
    'Placenta, Pulmón, Hígado': 'Mixed',
    'Contenido Intestino': 'Faeces',
    'Hisopo, Intestino Delgado, Intestino Grueso': 'Mixed',
    'Sangre': 'Blood',
    'Órganos': 'Mixed',
    'Intestino Grueso': 'Tissue',
    'Semen': 'Fluid',
    'Placenta': 'Tissue',
    'A-Pulmón': 'Tissue',
    'B-Pulmón': 'Tissue',
    'C-Pulmón': 'Tissue',
    'D-Pulmón': 'Tissue',
    'Testículo': 'Tissue',
    'Hisopo, Heces': 'Mixed',
    'Líquido pericárdico': 'Fluid',
    'Hisopo, Intestino Delgado': 'Mixed',
    'Rabo': 'Tissue',
    'Pulmón, Tonsila, Linfonodo': 'Mixed',
    'Intestino Delgado, Ciego': 'Mixed',
    'A-ADN': 'Other',
    'B-ADN': 'Other',
    'D-ADN': 'Other',
    'Hisopo, Digestivo': 'Mixed',
    'Tonsila': 'Tissue',
    'Pulmón, Tonsila': 'Mixed',
    'Heces, Contenido Estógamo': 'Mixed',
    'Exudado': 'Fluid',
    'Ganglio nervioso': 'Tissue',
    'Líquido Articular': 'Fluid',
    'Intestino Delgado, Heces': 'Mixed',
    'Fluido testicular': 'Fluid',
    'Intestino Grueso, Intestino Delgado, Heces': 'Mixed',
    'Contenido Estómago': 'Other',
    'Placenta, Pulmón, Hígado, Contenido Estómago': 'Mixed',
    'A-Fluido oral': 'Fluid',
    'C-Fluido oral': 'Fluid',
    'B-Fluido oral': 'Fluid',
    'Pulmón, Hígado, Contenido Estómago': 'Mixed',
    'Corazón': 'Tissue',
    'ADN': 'Other',
    'Útero': 'Tissue',
    'ARN': 'Other',
    'Hisopo articular': 'Swab',
    'Digestivo, Hisopo': 'Mixed',
    'Intestino Delgado, Hisopo': 'Mixed',
    'Heces, Intestino Delgado, Intestino Grueso': 'Mixed',
    'J-Fluido oral': 'Fluid',
    'I-Fluido oral': 'Fluid',
    'H-Fluido oral': 'Fluid',
    'G-Fluido oral': 'Fluid',
    'F-Fluido oral': 'Fluid',
    'E-Fluido oral': 'Fluid',
    'D-Fluido oral': 'Fluid',
    'Tonsila, Hisopo Tonsila': 'Mixed',
    'Bazo, Pulmón': 'Mixed',
    'Intesindo Delgado, Intestino Grueso': 'Mixed',
    'Exudado de lengua': 'Fluid',
    'Intestino Delgado, Intestino Grueso, Heces': 'Mixed',
    'Hisopo, Pulmón': 'Mixed',
    'Digestivo, Heces': 'Mixed',
    'Pulmón, Riñón, Linfonodo, Tonsila': 'Mixed',
    'Pulmón, Hígado, Bazo': 'Mixed',
    'Líquido torácico': 'Fluid',
    'Pulmón, Hígado, Contenido Estómago, Placenta': 'Mixed',
    'Hisopo, Líquido Articular': 'Mixed',
    'Lengua, Rabo': 'Mixed',
    'A-Hisopo': 'Swab',
    'B-Hisopo': 'Swab',
    'C-Hisopo': 'Swab',
    'Líquido, Pulmón': 'Mixed',
    'DNA, Pulmón': 'Mixed',
    'Líquido Articular, Articulación': 'Mixed',
    'Lengua, Cordón Umbilical': 'Mixed',
    'Tonsila, Lionfondo': 'Mixed',
    'Pulmón, Placenta, Hígado, Contenido Estómago': 'Mixed',
    'Hisopo, Placenta': 'Mixed',
    'Exudado vaginal': 'Fluid',
    'Digestivo, Estómago': 'Mixed',
    'Púlmón, Hígado, Digestivo': 'Mixed',
    'Articulación, Hisopo': 'Mixed',
    'Pulmón, Hígado, Estómago': 'Mixed',
    'Placenta, Feto': 'Mixed',
    'Agua': 'Other',
    'Intestino Grueso, Hisopo': 'Mixed',
    'Pulmón, Hígado, Bazo, Riñón': 'Mixed',
    'Pulmón, Ganglio': 'Mixed',
    'Colon, Ciego': 'Mixed',
    'Pulmón, Hígado, Placenta': 'Mixed',
    'Digestivo, Calzas': 'Mixed',
    'Hisopo, Intestino Grueso': 'Mixed',
    'Contenido': 'Other',
    'Placenta, Hisopo': 'Mixed',
    'Placenta, Hígado, Pulmón': 'Mixed',
    'Cordón Ubilical, Lengua, Rabo': 'Mixed',
    'Hígado, Pulmón': 'Mixed',
    'Encéfalo, Placenta, Pulmón, Hígado, Contenido Estómago': 'Mixed',
    'Pulmón, Raspado': 'Mixed',
    'Hisopo, Raspado': 'Mixed',
    'Líquido': 'Fluid',
    'Líquido sinovial': 'Fluid',
    'Contenido Digestivo': 'Other',
    'Heces, Calzas': 'Mixed',
    'Hisopo Vaginal, Pulmón, Hígado, Contenido Estómago': 'Mixed',
    'Hisopo, Heces, Digestivo': 'Mixed',
    'Líquido testicular': 'Fluid',
    'Heces, Intestino Delgado': 'Mixed',
    'Rabo, Fluido testicular': 'Mixed',
    'Digestivo, Hígado, Bazo': 'Mixed',
    'Lengua, Rabo, Cordón Umbilical': 'Mixed',
    'Hisopo, Articulación': 'Mixed',
    'Ciego': 'Tissue',
    'Intestino Grueso, Heces': 'Mixed',
    'Placenta, Tejido Fetal': 'Mixed',
    'Placcenta, Feto': 'Mixed',
    'Placenta, Contenido Estómago': 'Mixed',
    'Intestino': 'Tissue',
    'Pulmón, Hígado, Contenido Estomágo': 'Mixed',
    'Pulmón, Hígado, Riñón, Contenido Estómago': 'Mixed',
    'Extracto de riñón': 'Tissue',
    'Rabo, Testículo': 'Mixed',
    'Hígado, Riñón, Bazo': 'Mixed',
    'Linfonodo Mesentérico, Linfonodo Inguinal': 'Mixed',
    'Hisopo, Intestino': 'Mixed',
    'Contenido Digestivo, Heces': 'Mixed',
    'Heces, Hisopo, Digestivo': 'Mixed',
    'Intestino, Hisopo': 'Mixed',
    'Intestino Delgado, Intestino Grueso, Heces, Hisopo': 'Mixed',
    'Articulación, Líquido articular': 'Mixed',
    'Hiospo, Pulmón': 'Mixed',
    'Pulmón, Corazón, Linfonodo': 'Mixed',
    'Tejido': 'Tissue',
    'Suero, Lengua': 'Mixed',
    'Pulmón, Líquido': 'Mixed',
    'Pulmón, Hígado, Encéfalo': 'Mixed',
    'Heces, Hisopo': 'Mixed',
    'Lengua, Rabo, Feto': 'Mixed',
    'Lengua, Rabo, Testículo': 'Mixed',
    'Intestino, Contenido Digestivo': 'Mixed',
    'Hisoppo, Heces': 'Mixed',
    'Intestino, Heces': 'Mixed',
    'Digestivo, Intestino Delgado': 'Mixed',
    'Heces, Digestivo, Estómago': 'Mixed',
    'Pulmón, Estómago': 'Mixed',
    'Líquido placentario': 'Fluid',
    'Pulmón, Hígado, Lengua, Placenta': 'Mixed',
    'Pulmón, Hígado, Lengua, Contenido Estómago': 'Mixed',
    'Pulmón, Hígado, Contenido Estómago, Lengua': 'Mixed',
    'Bazo, Pulmón, Hígado': 'Mixed',
    'Intestino Delgado, Linfonodo Mesentérico': 'Mixed',
    'Linfonodo Inguinal': 'Tissue',
    'Digestivo, Contenido Intestino': 'Mixed',
    'Pulmón, Pericardio, Hisopo': 'Mixed',
    'Tonsila, Linfonodo Inguinal': 'Mixed',
    'Hígado, Bazo, Pulmón, Placenta': 'Mixed',
    'Líquido, Placenta': 'Mixed',
    'Líquido Articular, Hisopo': 'Mixed',
    'Esponja': 'Other',
    'Heces, Intestino': 'Mixed',
    'Pulmón, Hígado, Bazo, Linfonodo': 'Mixed',
    'Pulmón, Hígado, Bazo, Corazón': 'Mixed',
    'Pulmón, Linfonodo, Riñón': 'Mixed',
    'Intestino, Digestivo': 'Mixed',
    'A-Exudado': 'Fluid',
    'B-Exudado': 'Fluid',
    'C-Exudado': 'Fluid',
    'Pulmón, Hisopo Tráquea': 'Mixed',
    'Hisopo Digestivo': 'Swab',
    'Bazo, Riñón, Digestivo, Hígado, Hisopo Digestivo': 'Mixed',
    'Hisopo Pulmón': 'Swab',
    'Hisopo Placenta': 'Swab',
    'Intestino Grueso, Intestino Delgado, Hisopo Digestivo': 'Mixed',
    'Placenta, Feto, Lavado': 'Mixed',
    'Lengua, Placenta, Hisopo Vaginal': 'Mixed',
    'Heces, Intestino Grueso': 'Mixed',
    'Hisopo Digestivo, Heces': 'Mixed',
    'Intestino Delgado, Intestino Grueso, Estómago': 'Mixed',
    'Intestino Delgado, Hisopo Digestivo': 'Mixed',
    'Pulmón, Líquido torácico': 'Mixed',
    'Hisopo Cutáneo': 'Swab',
    'Pulmón, Bazo, Hígado, Linfonodo': 'Mixed',
    'Hisopo Intestino': 'Swab',
    'Lengua, Placenta, Pulmón, Hígado, Contenido Estómago': 'Mixed',
    'Pulmón, Bazo, Linfonodo': 'Mixed',
    'Hisopo Intestino Delgado': 'Mixed',
    'Hisopo Tráquea, Pulmón': 'Mixed',
    'Lengua, Placenta, Pulmón': 'Mixed',
    'Hisopo, Pulmón, Linfonodo': 'Mixed',
    'Intestino Delgado, Intestino Grueso, Heces, Hisopo Digestivo': 'Mixed',
    'Intestino Delgado, Intestino Grueso, Hisopo Digestivo ': 'Mixed',
    'Hígado, Bazo, Riñón': 'Mixed',
    'Hisopo Digestivo, Contenido Digestivo': 'Mixed',
    'Descargas cecales': 'Other',
    'Lengua, Testículo': 'Mixed',
    'Hisopo Intestino, Intestino Grueso': 'Mixed',
    'Pulmón, Rabo': 'Mixed',
    'Lengua, Pulmón, Hígado': 'Mixed',
    'Placenta, Hígado, Pulmón, Digestivo': 'Mixed',
    'Placenta, Pulmón, Contenido Estómago, Hisopo': 'Mixed',
    'Hígado, Bazo': 'Mixed',
    'Hispo Digestivo, Heces': 'Mixed',
    'Intestino Delgado, Intestino Grueso, Digestivo': 'Mixed',
    'Heces, Hisopo Digestivo': 'Mixed',
    'Pulmón, Linfonodo, Tráquea': 'Mixed',
    ' ': 'Missing',

}

# Map the SampleType column using the defined mapping
df["SampleType"] = df["SampleType"].map(SampleTypeMapping).fillna("Missing")


# Province mapping
ProvinceMapping = {
    'a coruna': 'ACoruna',
    'alava': 'Alava',
    'albacete': 'Albacete',
    'alicante': 'Alicante',
    'almeria': 'Almeria',
    'asturias': 'Asturias',
    'avila': 'Avila',
    'badajoz': 'Badajoz',
    'balears, illes': 'IslasBaleares',
    'barcelona': 'Barcelona',
    'bizkaia': 'Bizkaia',
    'burgos': 'Burgos',
    'caceres': 'Caceres',
    'cadiz': 'Cadiz',
    'cantabria': 'Cantabria',
    'castellon': 'Castellon',
    'ceuta': 'Ceuta',
    'ciudad real': 'CiudadReal',
    'cordoba': 'Cordoba',
    'coruna, a': 'ACoruna',
    'cuenca': 'Cuenca',
    'gipuzkoa': 'Gipuzkoa',
    'girona': 'Girona',
    'granada': 'Granada',
    'guadalajara': 'Guadalajara',
    'huelva': 'Huelva',
    'huesca': 'Huesca',
    'illes balears': 'IslasBaleares',
    'jaen': 'Jaen',
    'la rioja': 'LaRioja',
    'las palmas': 'LasPalmas',
    'leon': 'Leon',
    'lleida': 'Lleida',
    'lugo': 'Lugo',
    'madrid': 'Madrid',
    'malaga': 'Malaga',
    'melilla': 'Melilla',
    'murcia': 'Murcia',
    'navarra': 'Navarre',
    'nd': 'Missing',
    'ourense': 'Ourense',
    'palencia': 'Palencia',
    'palmas, las': 'LasPalmas',
    'pontevedra': 'Pontevedra',
    'rioja, la': 'LaRioja',
    'salamanca': 'Salamanca',
    'santa cruz de tenerife': 'SantaCruzDeTenerife',
    'segovia': 'Segovia',
    'sevilla': 'Sevilla',
    'soria': 'Soria',
    'sta. cruz tenerife': 'SantaCruzDeTenerife',
    'tarragona': 'Tarragona',
    'teruel': 'Teruel',
    'toledo': 'Toledo',
    'valencia': 'Valencia',
    'valladolid': 'Valladolid',
    'zamora': 'Zamora',
    'zaragoza': 'Zaragoza',

}


df["Province"] = df["Province"].map(ProvinceMapping).fillna("Missing")


In [9]:
# PigAge mapping
PigStageMapping = {

    'destetados': 'Weaning',
'cebo': 'Finishing',
'nd': 'Missing',
'reposicion (6 meses)': 'Gilt',
'reposicion': 'Gilt',
'transicion 6 semanas': 'Nursery',
'6-7 semanas': 'Nursery',
'cebo 9 semanas': 'Finishing',
'adultos': 'Other',
'cebo 40 a 150 dias de vida.': 'Mixed',
'transicion (6 semanas)': 'Nursery',
'cebo (final)': 'Finishing',
'reposicion (gilt)': 'Gilt',
'reposicion 16 y 20 semanas de vida': 'Gilt',
'transicion': 'Nursery',
'lact.>15': 'Suckling',
'4 meses.': 'Finishing',
'iberico cebo (5-6 meses)': 'Finishing',
'2-3 meses iberico': 'Finishing',
'lact.>15 (4 sem)': 'Suckling',
'cebo 14 semanas': 'Finishing',
'lechon': 'Suckling',
'adulto': 'Other',
'(iberico 2 meses)': 'Nursery',
'transicion/adultos': 'Mixed',
'lact.': 'Suckling',
'cebo ( 3 meses)': 'Finishing',
'transicion (2 meses)': 'Nursery',
'transicion (7 semanas)': 'Nursery',
'6 semanas': 'Nursery',
'transicion (2-3 meses)': 'Nursery',
'lact. (2 dias)': 'Suckling',
'transicion (5 semanas)': 'Nursery',
'cebo (4 meses)': 'Finishing',
'transicion (9-10 semanas)': 'Nursery',
'transicion (6-7 semanas)': 'Nursery',
'desconocido': 'Missing',
'transicion 7 semanas': 'Nursery',
'cebo ( 3 meses)': 'Finishing',
'cebo (100kg)': 'Finishing',
'lact. (20 dias)': 'Suckling',
'cebo ( 2,5 meses)': 'Finishing',
'cebo (futura reproductora)': 'Gilt',
'transicion 25 dias': 'Weaning',
'cebo 100kg': 'Finishing',
'transicion 8 semanas': 'Nursery',
'cebo (70kg)': 'Finishing',
'lact. (<15 dias)': 'Suckling',
'cebo ( 1,5 meses)': 'Nursery',
'cebo ( 30-150kg.)': 'Finishing',
'cebo ( 13 semanas)': 'Finishing',
'entrada a cebo': 'Finishing',
'cebo ( meses)': 'Finishing',
'transicion (30kg)': 'Finishing',
'cebo ( 4 meses)': 'Finishing',
'cebo 45-50 kg.': 'Finishing',
'cuarentena': 'Other',
'transicion (8 semanas)': 'Nursery',
'cebo (30 kg)': 'Finishing',
'cebo (8 semanas)': 'Finishing',
'cebo (5 meses)': 'Finishing',
'transicion (4-5-5-7 semanas)': 'Nursery',
'transicion (3-5-6-7 semanas)': 'Nursery',
'transicion (3-5-7 semanas)': 'Nursery',
'lact. transicion': 'Mixed',
'cebo (3 meses)': 'Finishing',
'cebo (16 semanas)': 'Finishing',
'cebo (50kg)': 'Finishing',
'cebo (18 kg a entrada)': 'Finishing',
'transicion (5 semanas de vida)': 'Nursery',
'transicion (9 semanas de vida)': 'Nursery',
'transicion (12 semanas de vida)': 'Finishing',
'cebo (entrada a cebo)': 'Finishing',
'3-5-7-9 semanas': 'Nursery',
'cebo ( meses)': 'Finishing',
'transicion (25-30 dias)': 'Weaning',
'lact. transicion 2-5 semanas': 'Mixed',
'cebo ( 6 meses)': 'Finishing',
'lact. transicion (40 dias)': 'Nursery',
'primerizas': 'Sow',
'cebo (110 kg)': 'Finishing',
'recria': 'Gilt',
'transicion (6 kg)': 'Weaning',
'cebo (12/13 semanas)': 'Finishing',
'cebo (entrada)': 'Finishing',
'cebo (30 kg) iberico': 'Finishing',
'cebo (50-60kg). (16-17 sem).': 'Finishing',
'transicion (7 sem)': 'Nursery',
'transicion/ cuarentena': 'Mixed',
'cebo vida (90kg)': 'Finishing',
'cebo (13 sem)': 'Finishing',
'reposicion (14-20 semanas)': 'Gilt',
'cebo (15 sem)': 'Finishing',
'cebo (2 meses)': 'Nursery',
'transicion (5 kg)': 'Weaning',
'adultos (1 ano)': 'Other',
'cebo 20 kg.': 'Finishing',
'cebo (1 meses)': 'Nursery',
'cebo (1,5 meses) entrada a cebo 7 kg': 'Nursery',
'transicion (10-30 kg).': 'Mixed',
'cebo (50 kg)': 'Finishing',
'lact. (20 dias)': 'Suckling',
'3-6-10 semanas': 'Nursery',
'cebo (6 semanas )': 'Nursery',
'cebo vida (50 kg)': 'Finishing',
'11 semanas': 'Finishing',
'cebo (3,5 meses)': 'Finishing',
'primerizas,adultos': 'Mixed',
'lact. ( 25 dias)': 'Suckling',
'transicion (6-8 semanas)': 'Nursery',
'3-6-9 semanas': 'Nursery',
'adultos (10 meses)': 'Other',
'lact. (25 dias)': 'Suckling',
'lact. (25 dias de vida)': 'Suckling',
'5 semanas': 'Nursery',
'3 semanas': 'Nursery',
'9 semanas': 'Nursery',
'4 semanas': 'Nursery',
'cebo (entrada cebo)': 'Finishing',
'lact. (18 dias)': 'Suckling',
'lact. adulto': 'Mixed',
'transicion (destetados)': 'Weaning',
'cebo ( 3 meses)': 'Finishing',
'cebo (70 kg)': 'Finishing',
'cebo (90 kg)': 'Finishing',
'cebo (1 ano)': 'Finishing',
'cebo (1 mes)': 'Nursery',
'transicion (7-8 semanas)': 'Nursery',
'cebo (110 dias en cebo)': 'Finishing',
'transicion (1 semana)': 'Nursery',
'cebo (14 semanas)': 'Finishing',
'lact. (>15 dias)': 'Suckling',
'cebo (17 sem)': 'Finishing',
'cebo (13-14 semanas)': 'Finishing',
'cebo (17 semanas)': 'Finishing',
'transicion (5 y 7 semanas)': 'Nursery',
'cebo (100 kg)': 'Finishing',
'transicion (40 dias)': 'Nursery',
'lact. (3 semanas)': 'Suckling',
'lact. (5 kg)': 'Suckling',
'cebo (90-100kg)': 'Finishing',
'transicion 3 y 5 semanas': 'Nursery',
'cebo (40 kg)': 'Finishing',
'cebo (2 meses)': 'Nursery',
'cebo ( meses) transicion': 'Mixed',
'reposicion (2o parto)': 'Sow',
'cebo (12 semanas)': 'Finishing',
'cebo (5, 9 y 17 semanas)': 'Mixed',
'cebo (90 dias)': 'Finishing',
'cebo (4 meses)': 'Finishing',
'cebo (90-100 kg)': 'Finishing',
'transicion (5 y 8 semanas)': 'Nursery',
'cebo (18 semanas )': 'Finishing',
'transicion (5-7-9 semanas)': 'Nursery',
'reposicion (60-70 kg)': 'Gilt',
'transicion (30 dias)': 'Nursery',
'transicion (10 semanas)': 'Nursery',
'cebo (3-4 meses)': 'Finishing',
'lact. ( dias)': 'Suckling',
'transicion (9 sem)': 'Nursery',
'lact. (14 dias)': 'Suckling',
'transicion (3 semanas)': 'Weaning',
'cebo (12 meses)': 'Finishing',
'cebo (120 kg)': 'Finishing',
'transicion (60 kg)': 'Finishing',
'transicion (25kg)': 'Finishing',
'reproductoras': 'Sow',
'cebo (22 semanas, 100 kg)': 'Finishing',
'cebo (7 meses, 80-90 kg)': 'Finishing',
'cebo (60 kg)': 'Finishing',
'transicion (40-50 dias)': 'Nursery',
'lact. cebo, transicion': 'Mixed',
'adultos transicion (42 dias)': 'Mixed',
'adultos transicion': 'Mixed',
'transicion ( 7 semanas)': 'Nursery',
'transicion (3-6-9-11 semanas)': 'Nursery',
'transicion (4 meses)': 'Finishing',
'lactantes': 'Suckling',
'gestantes': 'Sow',
'transicion (9 semanas)': 'Nursery',
'transicion (42 dias)': 'Nursery',
'cebo (81 y 91 dias)': 'Finishing',
'cebo (180 kg)': 'Finishing',
'transicion (3 y 6 semanas postdestete)': 'Nursery',
'cebo (8-12 semanas)': 'Finishing',
'destetado': 'Weaning',
'cebo (2,5 meses)': 'Finishing',
'cebo (vida)': 'Finishing',
'cebo (hibridas)': 'Finishing',
'postweaning': 'Nursery',
'transicion (4 y 5 semanas de vida)': 'Nursery',
'cebo (10-30kg.)': 'Mixed',
'cebo ( 24 semanas de vida)': 'Finishing',
'transicion (3 meses)': 'Nursery',
'cebo (4 meses)': 'Finishing',
'cebo ( 4 meses)': 'Finishing',
'cebo (2 meses en cebo)': 'Finishing',
'cebo (4-5 meses)': 'Finishing',
'transicion cebo reposicion': 'Mixed',
'transicion (5 semanas postdestete)': 'Nursery',
'transicion (1 semanas postdestete)': 'Nursery',
'transicion (6, 7 y 9 semanas)': 'Nursery',
'cebo (100 dias)': 'Finishing',
'transicion (6 semanas de vida)': 'Nursery',
'transicion (4-9 semanas)': 'Nursery',
'cebo (115 dias de vida)': 'Finishing',
'cebo (15 dias en cebo)': 'Finishing',
'transicion (3-4 semanas)': 'Weaning',
'transicion (3-6-9 semanas)': 'Nursery',
'lact. ( 21 dias)': 'Suckling',
'cebo (1 mes y medio en cebo)': 'Finishing',
'destete': 'Weaning',
'cebo (20 sem)': 'Finishing',
'cebo ( 3-4 meses)': 'Finishing',
'cebo ( 10 sem.)': 'Finishing',
'lact. (<15 dias) adultos': 'Mixed',
'cebo (100 dias )': 'Finishing',
'transicion (12 kg)': 'Nursery',
'cebo vida': 'Finishing',
'transicion (2-3 sem postdestete)': 'Nursery',
'cebo (3 meses) adultos': 'Mixed',
'cebo (6 meses)': 'Finishing',
'cebo ( 3,5 meses)': 'Finishing',
'reposicion (4-5 meses) transicion (8 semanas)': 'Mixed',
'lact. (28 dias)': 'Weaning',
'transicion (15 dias postdestete)': 'Nursery',
'adultos (nuliparas)': 'Gilt',
'transicion (20 y 28 dias de vida)': 'Weaning',
'cebo (100 kg.)': 'Finishing',
'transicion (15 kg)': 'Nursery',
'lact (final lactacion)': 'Suckling',
'lact. (12 dias)': 'Suckling',
'transicion (10-11 semanas de vida)': 'Nursery',
'transicion (4, 5 y 6 semanas de vida)': 'Nursery',
'transicion (4-5-6-7 semanas de vida)': 'Nursery',
'transicion (6-7 semanas de vida)': 'Nursery',
'transicion (5 y 6 semanas de vida)': 'Nursery',
'transicion (5-6 y 7 semanas de vida)': 'Nursery',
'lact. (24 dias)': 'Weaning',
'transicion (6-9 semanas)': 'Nursery',
'cebo (8-20 semanas)': 'Finishing',
'lact. ( >21 dias)': 'Suckling',
'transicion (28 dias de vida)': 'Weaning',
'cebo (3, 3,5, 4 y 5 meses)': 'Finishing',
'lact. ( <15 dias)': 'Suckling',
'transicion cebo': 'Mixed',
'transicion (60 dias)': 'Nursery',
'lact. (0 dias) transicion': 'Mixed',
'transicion (5-6 semanas de vida)': 'Nursery',
'transicion (10 semanas de vida)': 'Nursery',
'nuliparas multiparas': 'Mixed',
'transicion reposicion': 'Mixed',
'cebo (3-6 meses)': 'Finishing',
'transicion (3,5 meses; 20kg)': 'Finishing',
'transicion (8-9 semanas)': 'Nursery',
'transicion (6-7 sem de vida)': 'Nursery',
'lechones (15 kg)': 'Nursery',
'lechones (30 kg)': 'Finishing',
'cebo (50, 70 y 100 kg.)': 'Finishing',
'transicion (1 sem. postdestete)': 'Nursery',
'cebo (22-23 semanas)': 'Finishing',
'lact. (35 dias)': 'Weaning',
'transicion (3, 6, 9 semanas de vida)': 'Nursery',
'iberico cebo (3 meses)': 'Finishing',
'cebo (5 meses, 65kg)': 'Finishing',
'cebo (20 kg)': 'Finishing',
'cebo (55 kg)': 'Finishing',
'cebo (20-25 kg)': 'Finishing',
'transicion (3 dias postdestete)': 'Nursery',
'iberico (6 meses) "cebo (55 meses)" lact. (22-24 dias)': 'Mixed',
'lact. ( 21-22 dias)': 'Suckling',
'cebo vida iberico': 'Finishing',
'lact. (25 dias) transicion': 'Mixed',
'transicion (8 semanas de vida)': 'Nursery',
'transicion 4 semanas de vida': 'Weaning',
'cebo ( 6 meses)': 'Finishing',
'cebo ( 5 meses)': 'Finishing',
'cebo (150 dias de vida)': 'Finishing',
'transicion (21-24 dias vida)': 'Weaning',
'adultos lact.': 'Mixed',
'cebo (fase inicial cebo)': 'Finishing',
'transicion (4 semanas)': 'Weaning',
'transicion (80 dias)': 'Finishing',
'transicion (3-6 semanas postdestete)': 'Nursery',
'transicion (5-7 semanas)': 'Nursery',
'cebo (1,5 meses)': 'Nursery',
'transicion (70 dias), cebo (5 meses)': 'Mixed',
'lact. (23 dias)': 'Suckling',
'cebo ( 2 meses y medio)': 'Finishing',
'cebo (21 dias)': 'Weaning',
'transicion (1 semana postdestete)': 'Nursery',
'transicion (36, 47 y 83 dias de vida)': 'Mixed',
'9 semanas de vida': 'Nursery',
'cebo (12 semanas de vida)': 'Finishing',
'cebo (4 meses de vida)': 'Finishing',
'm150718': 'Other',
'reposicion (nuliparas)': 'Gilt',
'transicion (6 y 9 semanas)': 'Nursery',
'cebo 15 kg': 'Finishing',
'cebo (11 semanas de vida)': 'Finishing',
'transicion (1, 2 y 3 semans)': 'Nursery',
'transicion (4 semanas de vida)': 'Weaning',
'lact. (21 dias)': 'Suckling',
'transicion (7 semanas de vida)': 'Nursery',
'cebo (45 kg)': 'Finishing',
'adultos (reproductoras)': 'Sow',
'cebo (15 semanas)': 'Finishing',
'cebo (3 meses en cebadero)': 'Finishing',
'cebo ( 10 semanas)': 'Nursery',
'transicion (7 dias destetados)': 'Nursery',
'lact. (4 y 7 semanas)': 'Nursery',
'transicion adultos': 'Mixed',
'cebo (3 meses)': 'Finishing',
'verracos (>12 meses)': 'Boar',
'transicion (6, 8 y 9 semanas)': 'Nursery',
'transicion (12 semanas)': 'Finishing',
'reposicion (5 meses)': 'Gilt',
'cebo 2 meses': 'Finishing',
'cebo (120 dias)': 'Finishing',
'destete transicion (4 semanas)': 'Weaning',
'transicion (2-3 semanas)': 'Nursery',
'transicion (4, 5, 6 y 8 semanas)': 'Nursery',
'adultos (multipara)': 'Sow',
'transicion (3-9 semanas)': 'Nursery',
'transicion 30 dias': 'Weaning',
'transicion (3, 6 y 7 semanas)': 'Nursery',
'lactantes ( 14-21 dias)': 'Suckling',
'transicion 5 semanas': 'Nursery',
'7 semanas': 'Nursery',
'cebo ( 2,5 meses)': 'Finishing',
'cebo (10 sem y 4 meses)': 'Finishing',
'transicion (60 dias de vida)': 'Nursery',
'transicion (70 dias)': 'Nursery',
'adultos >12 meses': 'Other',
'cebo (70 dias)': 'Nursery',
'transicion (6 y 8 semanas)': 'Nursery',
'destetado (3 semanas)': 'Weaning',
'destetado (40 dias de vida)': 'Nursery',
'cebo (3 meses vida)': 'Finishing',
'cebo (18 kg.)': 'Nursery',
'cebo (inicio)': 'Finishing',
'cebo (inicio, 15 kg)': 'Nursery',
'cebo ( 11 semanas) 25-30kg.': 'Finishing',
'destetado 8 semanas vida': 'Nursery',
'transicion (50 kg)': 'Finishing',
'cebo ( 65 dias)': 'Nursery',
'adultos verraco >12m': 'Boar',
'verraco >12 meses': 'Boar',
'transicion (3-6-8/9 semanas de vida)': 'Nursery',
'cebo (18 kg pv)': 'Nursery',
'4-5-6-7-8-9 semanas': 'Nursery',
'cebo (4,5 meses)': 'Finishing',
'cebo ( 2 semanas)': 'Finishing',
'transicion (3 semanas de vida)': 'Weaning',
'transicion (2-3, 4 y 5 semanas de vida)': 'Mixed',
'cebo (50-60 kg)': 'Finishing',
'transicion (3, 5 y 8 semanas)': 'Mixed',
'cebo (60 dias)': 'Nursery',
'4 semanas de vida': 'Weaning',
'transicion (17 kg)': 'Nursery',
'transicion 7 y 8 semanas': 'Nursery',
'transicion 8 y 9 semanas de vida': 'Nursery',
'transicion 6-9 semanas': 'Nursery',
'adultos (>12 meses)': 'Other',
'cebo ( >18 kg)': 'Finishing',
'adultos verraco >12 meses': 'Boar',
'cebo ( 20 kg)': 'Finishing',
'lactantes ( dias) transicion': 'Mixed',
'cebo (13 y 24 semanas)': 'Finishing',
'lactantes transicion': 'Mixed',
'cebo (entrada hace 10 dias)': 'Finishing',
'transicion 1 y 4 semanas de destetados': 'Nursery',
'destetado (4-5 semanas postdestete)': 'Nursery',
'cebo ( 15 kg.)': 'Nursery',
'cebo (6,5 meses)': 'Finishing',
'cebo (3 meses de vida)': 'Finishing',
'lactantes (21 dias)': 'Suckling',
'cebo (>12 meses)': 'Finishing',
'reposicion 70 kg.': 'Gilt',
'lactantes (2-3 semanas)': 'Suckling',
'transicion (5, 6, 7 y 8 semanas)': 'Nursery',
'cebo (0,5 meses)': 'Suckling',
'verracos > 12 meses': 'Boar',
'cebo (3 semanas en cebo)': 'Finishing',
'lactantes (21 dias) transicion': 'Mixed',
'lactantes (17-20 dias)': 'Suckling',
'transicion (5-6 y 7-8 semanas)': 'Nursery',
'adultos (5-6 meses)': 'Sow',
'transicion cebo ( 50 kg.)': 'Mixed',
'cebo ( 60 kg.)': 'Finishing',
'cebo (45-50 kg)': 'Finishing',
'reposicion adultos primerizas': 'Mixed',
'primerizas reposicion': 'Mixed',
'cebo (1.5 meses)': 'Nursery',
'transicion (3 y 4 semanas)': 'Weaning',
'transicion 4 sem': 'Weaning',
'cebo (>12 meses)': 'Finishing',
'transicion (40, 45 y 50 dias, 8 semanas)': 'Nursery',
'transicion (3, 4, 5, 6 semanas)': 'Mixed',
'destetado (3 semanas postdestete)': 'Nursery',
'destetado cebo (3 meses)': 'Finishing',
'cebo ( 2 meses)': 'Nursery',
'lactantes (19 dias)': 'Suckling',
'transicion 4-5 semanas': 'Nursery',
'cebo ( 4 meses)': 'Finishing',
'cebo (8 meses)': 'Finishing',
'cebo (10 meses)': 'Finishing',
'cebo ( 1 meses)': 'Nursery',
'destetado 1-2-3 semanas': 'Nursery',
'destetado (9 semanas)': 'Nursery',
'cebo (2 meses vida)': 'Nursery',
'cebo (1-2 meses)': 'Nursery',
'cebo (3.5 meses)': 'Finishing',
'lactantes ( 10-12 dias)': 'Suckling',
'lactantes (> 15dias)': 'Suckling',
'transicion (5-6 sem vida)': 'Nursery',
'transicion (2-8 sem vida)': 'Mixed',
'transicion y adultos': 'Mixed',
'cebo (1 y 3 meses)': 'Nursery',
'cebo (75-80 kg)': 'Finishing',
'transicion (6 y 7 semanas)': 'Nursery',
'transicion (3,6,9 semanas)': 'Nursery',
'cebo (50, 60, 70 y 80 dias)': 'Mixed',
'cebo (80 dias)': 'Finishing',
'destetado cebo': 'Nursery',
'transicion (21, 42 y 63 dias)': 'Mixed',
'transicion (3, 5 y 6 sem)': 'Mixed',
'3-9 semanas transicion': 'Mixed',
'cebo (7 y 8 semanas)': 'Nursery',
'lactantes (18 dias)': 'Suckling',
'cebo (24 kg)': 'Finishing',
'nuliparas (7,5 meses)': 'Gilt',
'cebo (25 y 80 kg)': 'Finishing',
'lactantes (4-7 y 18 dias)': 'Suckling',
'transicion (5-6/ 7-8 semanas vida)': 'Nursery',
'transicion primerizas': 'Mixed',
'transicion (4-6 semanas de vida)': 'Nursery',
'destetado reposicion adultos': 'Mixed',
'cebo (30, 60 y 100 kg)': 'Finishing',
'transicion (3y 4 semanas) cebo (7 y 8 semanas)': 'Mixed',
'cebo (80-100 kg)': 'Finishing',
'primerizas adultos': 'Sow',
'reposicion destetados lactantes': 'Mixed',
'cebo (15-20 kg.)': 'Nursery',
'transicion (3 meses de vida)': 'Finishing',
'transicion (7-8 semanas de vida)': 'Nursery',
'transicion (7 y 8 semanas)': 'Nursery',
'transicion (4 y 6 semanas)': 'Nursery',
'transicion (40, 50 y 75 dias)': 'Nursery',
'transicion (5-9 semanas de vida)': 'Nursery',
'cebo (1 semana)': 'Nursery',
'cebo (10-15 kg.)': 'Nursery',
'cebo ( 20-25 kg meses)': 'Finishing',
'cebo ( 20-25 kg)': 'Finishing',
'cebo (15 dias)': 'Nursery',
'lactantes (26 dias)': 'Suckling',
'cebo (25 kg)': 'Finishing',
'transicion (45-60-80 dias)': 'Nursery',
'transicion (30-45-60 dias)': 'Nursery',
'transicion (5,6 y 7 semanas)': 'Nursery',
'cebo (140 dias)': 'Finishing',
'destetado (21 dias)': 'Weaning',
'verracos': 'Boar',
'transicion (1, 2, 5 y 9 semanas)': 'Mixed',
'cebo (25-30 kg)': 'Finishing',
'lactantes (20 dias)': 'Suckling',
'lactantes (5 y 10 dias)': 'Suckling',
'cebo (25-30 kg.)': 'Finishing',
'lactantes (2 dias)': 'Suckling',
'cebo (15-16 kg.)': 'Nursery',
'adultos > 12 meses': 'Other',
'cebo (5 semanas)': 'Nursery',
'cebo (15 kg.)': 'Nursery',
'destetado (10 -17 dias postdestete)': 'Nursery',
'cebo (1 mes en cebo)': 'Nursery',
'cebo (3 meses en cebo)': 'Finishing',
'cebo (22 semanas)': 'Finishing',
'transicion (9-11 semanas)': 'Nursery',
'transicion (23 kg)': 'Finishing',
'destetado,transicion': 'Nursery',
'otra': 'Other',
'3-5-7 semanas': 'Nursery',
'transicion (3, 5 y 7 semanas)': 'Nursery',
'transicion (28-49 y 63 dias)': 'Nursery',
'transicionn (28-49 y 63 dias)': 'Nursery',
'transicion 10 kg': 'Nursery',
'transicion (6,9,12 semanas)': 'Nursery',
'destetado (20-30 dias)': 'Weaning',
'transicion (30 dias de vida)': 'Nursery',
'lactantes (10 dias)': 'Suckling',
'nuliparas': 'Gilt',
'lactantes (21-25 dias)': 'Suckling',
'lactantes (2-35 dias)': 'Mixed',
'lactantes (10 kg)': 'Nursery',
'cebo ( 3 meses) reproductoras': 'Mixed',
'cebo ( 2 meses)': 'Nursery',
'cebo (15 kg)': 'Nursery',
'lactantes destetado': 'Mixed',
'transicion (3, 5, 7 y 9 semanas de vida)': 'Nursery',
'adultos lactantes (nm)': 'Mixed',
'transicion (10 kg)': 'Nursery',
'cebo ( 80 kg.)': 'Finishing',
'transicion 8semanas': 'Nursery',
'transicion (4, 6 y 8 semanas)': 'Nursery',
'lactantes (25 dias)': 'Weaning',
'cebo (10 semanas)': 'Nursery',
'transicion (4-5 semanas)': 'Nursery',
'8-9-10 semanas': 'Nursery',
'cebo ( 15 kg.)': 'Nursery',
'transicion 3 semanas': 'Weaning',
'cebo (11 semanas)': 'Finishing',
'transicion (10-11 semanas)': 'Nursery',
'transicion (30-50 dias de vida)': 'Nursery',
'lactantes (10-20 dias)': 'Mixed',
'cebo (18-20 kg)': 'Nursery',
'transicion (4 y 8 semanas de vida)': 'Nursery',
'cebo (35 kg)': 'Finishing',
'destetado (40-60 dias vida)': 'Nursery',
'transicion (42 dias vida)': 'Nursery',
'transicion (65 dias vida)': 'Nursery',
'transicion (65 dias vida)': 'Nursery',
'transicion (6 y 9 semanas)': 'Nursery',
'transicion (3 y 5 semanas vida)': 'Mixed',
'cebo (2 5 meses)': 'Nursery',
'cebo (80 kg)': 'Finishing',
'destetado (5-10 semanas vida)': 'Nursery',
'cebo ( 2,5 meses)': 'Nursery',
'cebo (16 semanas vida)': 'Finishing',
'lactantes (24 dias)': 'Weaning',
'transicion (35 dias vida)': 'Nursery',
'lactantes (28 dias)': 'Weaning',
'transicion (5- 8 semanas vida)': 'Nursery',
'lactantes (21 dias) transicion 5 semanas': 'Mixed',
'cebo (3 y 5 meses)': 'Finishing',
'cebo (6 semanas)': 'Nursery',
'cebo (9 semanas)': 'Nursery',
'lactantes ( dias)': 'Suckling',
'cebo (2.5 meses)': 'Nursery',
'cebo 19 semanas': 'Finishing',
'transicion (45 dias de vida)': 'Nursery',
'lactante (21 dias)': 'Weaning',
'cebo ( 1,5 meses)': 'Nursery',
'cebo reposicion': 'Mixed',
'cebo ( 26 semanas)': 'Finishing',
'cebo ( 12-13 semanas)': 'Finishing',
'transicion (4, 5 y 6 semanas)': 'Nursery',
'cebo ( 60 kg)': 'Finishing',
'cebo (7 meses)': 'Finishing',
'transicion (2, 6 y 7 semanas)': 'Mixed',
'cebo ( 18 kg)': 'Nursery',
'transicion (2, 3 y 5 semanas)': 'Mixed',
'cebo ( 50 kg)': 'Finishing',
'lactantes ( dias)': 'Suckling',
'cebo ( 2 meses)': 'Nursery',
'cebo (,5 meses )': 'Finishing',
'cebo 5 meses': 'Finishing',
'cebo 4 meses': 'Finishing',
'adultos lactantes (1 dias)': 'Mixed',
'lactantes ( 7-15 dias)': 'Suckling',
'lactantes,transicion': 'Mixed',
'transicion (8 semanas vida)': 'Nursery',
'lactantes ( 12 dias)': 'Suckling',
'transicion 9 semanas': 'Nursery',
'8 semanas': 'Nursery',
'lactantes ( 15-21 dias)': 'Suckling',
'cebo ( 40 kg.)': 'Finishing',
'transicion 5sdv': 'Nursery',
'destetado (2-3 semanas vida)': 'Weaning',
'lactantes (15 dias)': 'Suckling',
'cebo (13 semanas)': 'Finishing',
'cebo ( 20kg)': 'Finishing',
'cebo ( 6-10 semanas)': 'Nursery',
'destetado (60 dias)': 'Nursery',
'lactantes (3 semanas)': 'Suckling',
'reposicion lactantes (26 dias)': 'Mixed',
'cebo (15-70kg)': 'Mixed',
'lactantes ( 14 dias)': 'Suckling',
'cebo (110 dias)': 'Finishing',
'transicion (2 semanas)': 'Nursery',
'destetado adultos': 'Mixed',
'adultos (verraco)': 'Boar',
'cebo ( 1 meses)': 'Nursery',
'transicion 1 semana posdestete': 'Nursery',
'transicion 3 y 4 sem posdestete': 'Nursery',
'destetado transicion': 'Nursery',
'cebo (16 kg)': 'Nursery',
'destetado transicion (6 y 9 semanas)': 'Nursery',
'destetado, transicion (9-10 semanas)': 'Nursery',
'7 y 8 semanas': 'Nursery',
'transicion (4, 5, 6, 7 y 9 semanas)': 'Nursery',
'lactantes ( 21 dias)': 'Weaning',
'cebo ( 1,5 meses)': 'Nursery',
'lactantes (26 dias) adultos': 'Mixed',
'transicion 24-52 dias': 'Mixed',
'entrada cebo': 'Nursery',
'transicion ( 9 semanas)': 'Nursery',
' cebo (25 meses)  lactantes ( 28 dias)': 'Mixed',
'lactantes ( 1-10-20 dias) destetado (5 semanas de vida)': 'Mixed',
'lactantes ( 28 dias)': 'Weaning',
'lactantes (17 dias)': 'Suckling',
'reposicion 9 meses': 'Gilt',
'cebo ( 50 kg. )': 'Finishing',
'adultos 90 kg': 'Other',
'transicion (8-10 semanas)': 'Nursery',
'transicion (6 semanas vida)': 'Nursery',
'transicion (1, 2 y 5 semanas)': 'Mixed',
'cebo (16-17 semanas)': 'Finishing',
'lactantes (14 dias)': 'Suckling',
'cebo (8-10 semanas)': 'Nursery',
'cebo ( 1 mes)': 'Nursery',
'lactantes ( 20 dias)': 'Suckling',
'transicion 28-63 dias': 'Nursery',
'transicion (15-18 kg) cebo (7 meses)': 'Mixed',
'cebo ( 11 meses)': 'Finishing',
'cebo (5-6 meses)': 'Finishing',
'destetado (10 semanas de vida)': 'Nursery',
'destetado (9 semanas de vida)': 'Nursery',
'adultos lactantes ( 15 dias) transicion 3-9 semanas': 'Mixed',
'cebo ( 1 meses)': 'Nursery',
'cebo (80 kg.)': 'Finishing',
'lactantes (1 semanas)': 'Suckling',
'cebo (3 semanas)': 'Nursery',
'cebo (1 meses)': 'Nursery',
'destetado (33 dias)': 'Nursery',
'transicion 7w': 'Nursery',
'transicion 6-7 semanas': 'Nursery',
'cebo ( 2,5 meses)': 'Nursery',
'transicion 63d': 'Nursery',
'cebo (59 dias)': 'Nursery',
'cebo (56 y 45 dias)': 'Nursery',
'cebo ( 9 semanas)': 'Nursery',
'lactantes ( 7 dias)': 'Suckling',
'destetado (2-3 semanas)': 'Weaning',
'transicion 4 semanas': 'Weaning',
'cebo (30 kg.)': 'Finishing',
'destetado (7 semanas de vida)': 'Nursery',
'cebo ( 4 meses)': 'Finishing',
'lactantes (10 dias)': 'Suckling',
'transicion (1, 2, 3, 4 y 5 semanas)': 'Mixed',
'destetado (30 y 70 dias)': 'Nursery',
'transicion cebo (42 dias)': 'Mixed',
'destetado (1 semana)': 'Weaning',
'cebo (18 semanas)': 'Finishing',
'destetado (28 dias vida)': 'Weaning',
'transicion (28 dias vida)': 'Weaning',
'recria transicion': 'Mixed',
'transicion (26 dias en transicion)': 'Nursery',
'destetado (28 dias)': 'Weaning',
'destetado (28 dias)': 'Weaning',
'lactantes (21 dias) destetado': 'Weaning',
'lactantes (21 dias)': 'Weaning',
'transicion 3-5 semanas': 'Mixed',
'adultos (3 anos)': 'Other',
'destetado (4 sem vida)': 'Weaning',
'transicion (7-8 sem vida)': 'Nursery',
'adultos (3-4 meses)': 'Other',
'lactantes (24 dias)': 'Weaning',
'destetado (30-60 dias)': 'Nursery',
'post destetado': 'Nursery',
'reposicion transicion': 'Mixed',
'transicion 11 semanas': 'Finishing',
'destetado (4,5,6 y 7 sem)': 'Nursery',
'destetado (40 dias vida)': 'Nursery',
'destetado (7-14 dias )': 'Nursery',
'destetado (45 dias)': 'Nursery',
'destetado (35 dias)': 'Nursery',
'destetado (30 dias)': 'Nursery',
'reproductoras (90 dias gestacion)': 'Sow',
'transicion 10 semanas': 'Nursery',
'cebo (15-16 semanas)': 'Finishing',
'cebo (6-8-10 semanas)': 'Nursery',
'cebo (5-7-9 semanas)': 'Nursery',
'cebo (4-6-8-10 semanas)': 'Nursery',
'cebo ( 60 dias de vida)': 'Nursery',
'cebo ( 3meses)': 'Finishing',
'destetado (20 dias)': 'Weaning',
'lactantes ( 7 dias)': 'Suckling',
'cebo (28 dias)': 'Weaning',
'cebo (66 dias)': 'Nursery',
'transicion 28d': 'Weaning',
'cebo ( 1 meses)': 'Nursery',
'cebo ( 9 meses)': 'Finishing',
'adultos reproductoras': 'Sow',
'cebo (6-7 semanas)': 'Nursery',
'16 semanas': 'Finishing',
'lactantes (3 dias)': 'Suckling',
'cebo (45 dias)': 'Nursery',
'cebo ( 9-15 semanas)': 'Finishing',
'cebo ( 30-40 kg)': 'Finishing',
'cebo (30-40 kg)': 'Finishing',
'cebo ( 17w)': 'Finishing',
'cebo ( 40 kg.)': 'Finishing',
'lactantes (14 dias) transicion': 'Mixed',
'cebo (20 semanas)': 'Finishing',
'cebo (25 semanas)': 'Finishing',
'cebo adultos': 'Mixed',
'transicion 0-3-6 semanas': 'Mixed',
'lactantes ( 23 dias)': 'Weaning',
'lactantes (20 kg)': 'Suckling',
'cebo ( 30 kg.)': 'Finishing',
'destetado (6 kg)': 'Weaning',
'reproductoras (30 dias gestacion)': 'Sow',
'cebo ( 80 kg )': 'Finishing',
'cebo (8 meses - 100 kg)': 'Finishing',
'cebo ( 50 kg)': 'Finishing',
'transicion 4-5-9 semanas': 'Nursery',
'lactantes (22 dias)': 'Weaning',
'adultos (1,5 anos)': 'Other',
'cebo (6 kg.)': 'Nursery',
'cebo ( 60-70 kg.)': 'Finishing',
'cebo ( 4 meses)': 'Finishing',
'destetado (60-70 dias vida)': 'Nursery',
'lactantes (1 dias) destetado': 'Weaning',
'destetado (6 y 9 semanas)': 'Nursery',
'reproductoras/es': 'Other',
'cebo/engorde': 'Finishing',
'cebo/engorde (11 sem)': 'Finishing',
'destetado/transicion': 'Mixed',
'cebo/engorde (2 meses)': 'Nursery',
'cebo/engorde ( 7 meses sem/mes/ano)': 'Finishing',
'cebo/engorde ( 2 meses)': 'Nursery',
'cebo/engorde (35 dias)': 'Finishing',
'lactantes ( 30 dias)': 'Weaning',
'cebo/engorde (1 mes)': 'Nursery',
'recria/reposicion': 'Gilt',
'cebo/engorde ( 3 meses)': 'Finishing',
'cebo/engorde ( sem/mes/ano)': 'Finishing',
'cebo/engorde (2 meses, <60kg)': 'Finishing',
'cebo/engorde (2 semanas, <30 kg)': 'Finishing',
'cebo/engorde (3 meses)': 'Finishing',
'cebo/engorde ( 3 meses': 'Finishing',
'cebo/engorde (2,5 meses)': 'Nursery',
'cebo/engorde ( 16 sem)': 'Finishing',
'destetado/transicion (3-6-9 sem)': 'Mixed',
'cebo (1,5 meses, <60 kg)': 'Finishing',
'destetado/transicion (2 y 4 semanas)': 'Mixed',
'cebo/engorde ( 3 meses)': 'Finishing',
'cebo/engorde (10 sem)': 'Nursery',
'cebo/engorde (3 mes)': 'Finishing',
'cebo/engorde ( 2 meses)': 'Nursery',
'destetado/transicion (6 y 9 semanas) primerizas': 'Mixed',
'cebo/engorde (1 sem)': 'Nursery',
'destetado/transicion (2 meses vida)': 'Nursery',
'lactantes ( 22 dias)': 'Weaning',
'cebo/engorde (2 sem/mes)': 'Nursery',
'cebo/engorde 20 kg.': 'Finishing',
'cebo/engorde (15 dias)': 'Nursery',
'cebo/engorde (6 meses)': 'Finishing',
'cebo/engorde ( 3 mes)': 'Finishing',
'destetado/transicion (9 semanas)': 'Nursery',
'destetado/transicion (7 semanas)': 'Nursery',
'destetado/transicion (6 semanas)': 'Nursery',
'destetado/transicion 7 semanas': 'Nursery',
'cebo/engorde ( 4 meses )': 'Finishing',
'destetado/transicion adultos': 'Mixed',
'cebo/engorde (4meses)': 'Finishing',
'destetado/transicion 5 semanas': 'Nursery',
'destetado/transicion 5 sem': 'Nursery',
'cebo/engorde ( 1 meses)': 'Nursery',
'cebo/engorde ( 16 semanas)': 'Finishing',
'destetado/transicion 6 semanas': 'Nursery',
'cebo/engorde ( sem/mes/ano)': 'Finishing',
'cebo/engorde ( 4 meses)': 'Finishing',
'recria/reposicion 5-6 meses': 'Gilt',
'cebo/engorde (30 kg.)': 'Finishing',
'cebo/engorde ( 6 meses)': 'Finishing',
'cebo/engorde (4 meses)': 'Finishing',
'cebo/engorde ( 3 sem)': 'Finishing',
'cebo/engorde ( 10 semanas)': 'Nursery',
'transicion 30-35 dias': 'Nursery',
'cebo/engorde (4 mes)': 'Finishing',
'destetado/transicion (3 semanas)': 'Weaning',
'adultos 18 meses': 'Other',
'semanas 44, 45, 46 y 47': 'Finishing',
'cebo/engorde ( 8 sem)': 'Nursery',
'lactantes ( 15 dias)': 'Suckling',
'primalas': 'Gilt',
'reproductoras/es, adultos': 'Other',
'cebo/engorde (1 meses)': 'Nursery',
'cebo/engorde (5 semanas)': 'Nursery',
'cebo/engorde (3 semanas)': 'Nursery',
'cebo/engorde (6 semanas)': 'Nursery',
'cebo/engorde (5 meses)': 'Finishing',
'destetado/transicion (6 y 9 semanas)': 'Nursery',
'destetado/transicion cebo 2, 5 meses': 'Nursery',
'destetado/transicion (7 y 14 semanas)': 'Nursery',
'cebo/engorde (6 sem)': 'Nursery',
'cebo/engorde ( 9 sem)': 'Nursery',
'cebo/engorde (50-60 kg)': 'Finishing',
'destetado/transicion (5 semanas)': 'Nursery',
'cebo/engorde (14 sem)': 'Finishing',
'cebo/engorde (4 dias)': 'Nursery',
'18 kgs': 'Nursery',
'cebo/engorde (41 dias)': 'Nursery',
'destetado/transicion (42 dias)': 'Nursery',
'cebo/engorde (80 dias)': 'Finishing',
'cebo/engorde (21 semanas)': 'Finishing',
'destetado/transicion (35-40 dias)': 'Nursery',
'cebo/engorde (2 semanas)': 'Nursery',
'cebo/engorde ( 1 mes)': 'Nursery',
  
}

# Map the 'Edad' column using the English translation mapping
df['PigStage'] = df['Age'].map(PigStageMapping).fillna("Finishing")
df = df.drop(['Age'], axis=1)


In [10]:
print(df['PigStage'].unique())

['Weaning' 'Finishing' 'Missing' 'Gilt' 'Nursery' 'Other' 'Mixed'
 'Suckling' 'Sow' 'Boar']


In [11]:
df.to_csv('output/PigLab5_1.csv', index=False)

## Second data upload

In [12]:
# Define the file paths
files1 = ['Lab5/2-Decide-Exopol.xlsx']

# Load the data into Pandas dataframes
dfs1 = []
for file in files1:
    df1 = pd.read_excel(file, engine='openpyxl')
    dfs1.append(df1)

df1 = dfs1[0]

In [13]:
df1 = df1.drop(['Procedencia provincia'], axis=1)
df1 = df1.drop(['Categoría animal'], axis=1)
df1 = df1.drop(['Pool o individual'], axis=1)
df1 = df1.drop(['Proceso'], axis=1)

df1.head(20)

,ID caso,Fecha,Provincia,Edad,Tipo de muestra,PCR,Resultado
0,1,2020-01-02,Castellón,Destetados,Pulmón,PRRS (cepas europeas),Positivo
1,1,2020-01-02,Castellón,Destetados,Pulmón,PRRS (cepas americanas),Neg.
2,1,2020-01-02,Castellón,Destetados,Pulmón,Actinobacillus pleuropneumoniae,Neg.
3,2,2020-01-02,Castellón,Cebo,Pulmón,PRRS (cepas europeas),Positivo
4,2,2020-01-02,Castellón,Cebo,Pulmón,PRRS (cepas americanas),Neg.
5,2,2020-01-02,Castellón,Cebo,Pulmón,Actinobacillus pleuropneumoniae,Positivo
6,3,2020-01-02,Zaragoza,Adultos,Feto,PRRS (cepas europeas),Positivo
7,3,2020-01-02,Zaragoza,Adultos,Feto,PRRS (cepas americanas),Neg.
8,4,2020-01-02,Huesca,ND,Pulmón,PRRS (cepas europeas),Positivo
9,4,2020-01-02,Huesca,ND,Pulmón,PRRS (cepas americanas),Neg.


In [14]:
# Errors arised with the coding of the Age 
import unicodedata

def clean_text(text):
    if pd.isna(text):
        return None
    text = str(text).strip().lower()
    text = unicodedata.normalize('NFKD', text).encode('ASCII', 'ignore').decode('utf-8')
    return text


# Create a column with the clean text
df1['Edad'] = df1['Edad'].apply(clean_text)
df1['Provincia'] = df1['Provincia'].apply(clean_text)
df1['Resultado'] = df1['Resultado'].apply(clean_text)

print(df1['Provincia'].unique())

['castellon' 'zaragoza' 'huesca' 'badajoz' 'toledo' 'salamanca' 'cordoba'
 'teruel' 'lleida' 'burgos' 'cadiz' 'girona' 'segovia' 'sevilla' 'alava'
 'malaga' 'albacete' 'murcia' 'caceres' 'ciudad real' 'bizkaia' 'navarra'
 'tarragona' 'huelva' 'madrid' 'almeria' 'avila' 'rioja, la' 'lugo'
 'zamora' 'barcelona' 'coruna, a' 'balears, illes'
 'santa cruz de tenerife' 'valencia' 'leon' 'valladolid' 'gipuzkoa'
 'soria' 'jaen' 'alicante' 'palmas, las' 'las palmas' 'granada' 'ourense'
 'cuenca' 'a coruna' 'pontevedra' 'melilla' 'palencia' 'guadalajara'
 'la rioja' 'asturias' 'ceuta' 'illes balears' 'sta. cruz tenerife'
 'cantabria']


In [15]:
#Rename Colums

df1 = df1.rename(columns={
    'ID caso': 'SampleNumber',
    'Provincia': 'Province',
    'Fecha': 'Date',
    'PCR': 'Pathogen',
    'Resultado': 'Result',
    'Tipo de muestra': 'SampleType',
    'Edad': 'Age', #Age of animal
})

# Set DiagnosticTest to "PCR" Country to "Spain", Breed to "Meat", and LabReference to "1".
df1['DiagnosticTest'] = 'PCR'
df1['Country'] = 'Spain'
df1['Breed'] = 'Meat'
df1["Lab_Reference"] = "5"

In [16]:
#remove ' symbol
df1['Age'] = df1['Age'].str.replace("'", ' ', regex=False).str.strip()
df1['Age'] = df1['Age'].str.replace('"', ' ', regex=False).str.strip()
#print(df1['Age'].unique())


In [17]:
print(df1['Result'].unique())

['positivo' 'neg.']


In [18]:
#Annotation Mapping

# Add Result mapping for NEG = 0 and POS =1
resultMapping = {
    "positivo": "1",
    "neg.": "0",
}
df1["Result"] = df1["Result"].map(resultMapping).fillna("Missing")

PathogenMapping = {

    "Lawsonia intracellularis": "Lawsonia intracellularis",
    "L. intracellularis": "Lawsonia intracellularis",
    "L.intracelullaris": "Lawsonia intracellularis",
    "M. hyopneumoniae": "Mycoplasma hyopneumoniae",
    "Myc. hyopneumoniae": "Mycoplasma hyopneumoniae",
    "Mycoplasma hyopneumoniae antistoffen" : "Mycoplasma hyopneumoniae",
    "Mycoplasma hyopneumoniae" : "Mycoplasma hyopneumoniae",
    "M. hyopneumoniae" : "Mycoplasma hyopneumoniae",
    "SIV" : "Swine influenza virus",
    "Influenza A" : "Swine influenza virus",
    ##
    "PRRS (cepas europeas)": "Porcine reproductive and respiratory syndrome virus",
    "PRRS (cepas americanas)": "Porcine reproductive and respiratory syndrome virus",
    "Actinobacillus pleuropneumoniae": "Actinobacillus pleuropneumoniae",
    "APP 1": "Actinobacillus pleuropneumoniae",
    "APP 2": "Actinobacillus pleuropneumoniae",
    "APP 3": "Actinobacillus pleuropneumoniae",
    "APP 4": "Actinobacillus pleuropneumoniae",
    "APP 5": "Actinobacillus pleuropneumoniae",
    "APP 6": "Actinobacillus pleuropneumoniae",
    "APP 7": "Actinobacillus pleuropneumoniae",
    "APP 8": "Actinobacillus pleuropneumoniae",
    "APP 9/11": "Actinobacillus pleuropneumoniae",
    "APP 10": "Actinobacillus pleuropneumoniae",
    "APP 12": "Actinobacillus pleuropneumoniae",
    "APP 13": "Actinobacillus pleuropneumoniae",
    "APP 14": "Actinobacillus pleuropneumoniae",
    "APP 15": "Actinobacillus pleuropneumoniae",
    "APP 16": "Actinobacillus pleuropneumoniae",
    "APP 17": "Actinobacillus pleuropneumoniae",
    "APP 18": "Actinobacillus pleuropneumoniae",
    "APP 19": "Actinobacillus pleuropneumoniae",
    "PEDV": "Porcine epidemic diarrhea virus",
    "Brachyspira hyodysenteriae": "Brachyspira hyodysenteriae"
}

df1["Pathogen"] = df1["Pathogen"].map(PathogenMapping).fillna("Missing")

# SampleTypeMapping for the additional sample types
SampleTypeMapping = {
    'Pulmón': 'Tissue',
    'Fluido oral': 'Fluid',
    'Hisopo': 'Swab',
    'Raspado': 'Swab',
    'Lavado': 'Other',
    'Cordón umbilical': 'Tissue',
    'Digestivo': 'Tissue',
    'Suero': 'Blood',
    'Coriolis': 'Other',
    'Muestra ambiental': 'Other',
    'Heces': 'Faeces',
    'Linfonodo': 'Tissue',
    'Lengua': 'Tissue',
    'Pulmón, Corazón': 'Mixed',
    'Pulmón, Bazo': 'Mixed',
    'Órganos feto': 'Tissue',
    'Pulmón, Linfonodo': 'Mixed',
    'Hisopo, Líquido pericárdico': 'Mixed',
    'Pulmón, Líquido pericárdico': 'Mixed',
    'Hígado': 'Tissue',
    'Pulmón, Hisopo': 'Mixed',
    'Pulmón, Corazón, Líquido pericárdico': 'Mixed',
    'Calzas': 'Other',
    'Heces, Digestivo': 'Mixed',
    'Intestino Delgado': 'Tissue',
    'Válvula Ileocecal': 'Tissue',
    'Pulmón, Ganglio nervioso': 'Mixed',
    'Toallita': 'Other',
    'Articulación': 'Tissue',
    'Pulmón, Hígado': 'Mixed',
    'Pulmón, Tráquea, Tonsila': 'Mixed',
    'Pulmón, Tráquea': 'Mixed',
#N/D
    'FTA': 'Other',
    'Feto': 'Tissue',
    'Gánglio, Pulmón': 'Mixed',
    'Intestino Delgado, Intestino Grueso': 'Mixed',
    'Placenta, Pulmón, Hígado': 'Mixed',
    'Contenido Intestino': 'Faeces',
    'Hisopo, Intestino Delgado, Intestino Grueso': 'Mixed',
    'Sangre': 'Blood',
    'Órganos': 'Mixed',
    'Intestino Grueso': 'Tissue',
    'Semen': 'Fluid',
    'Placenta': 'Tissue',
    'A-Pulmón': 'Tissue',
    'B-Pulmón': 'Tissue',
    'C-Pulmón': 'Tissue',
    'D-Pulmón': 'Tissue',
    'Testículo': 'Tissue',
    'Hisopo, Heces': 'Mixed',
    'Líquido pericárdico': 'Fluid',
    'Hisopo, Intestino Delgado': 'Mixed',
    'Rabo': 'Tissue',
    'Pulmón, Tonsila, Linfonodo': 'Mixed',
    'Intestino Delgado, Ciego': 'Mixed',
    'A-ADN': 'Other',
    'B-ADN': 'Other',
    'D-ADN': 'Other',
    'Hisopo, Digestivo': 'Mixed',
    'Tonsila': 'Tissue',
    'Pulmón, Tonsila': 'Mixed',
    'Heces, Contenido Estógamo': 'Mixed',
    'Exudado': 'Fluid',
    'Ganglio nervioso': 'Tissue',
    'Líquido Articular': 'Fluid',
    'Intestino Delgado, Heces': 'Mixed',
    'Fluido testicular': 'Fluid',
    'Intestino Grueso, Intestino Delgado, Heces': 'Mixed',
    'Contenido Estómago': 'Other',
    'Placenta, Pulmón, Hígado, Contenido Estómago': 'Mixed',
    'A-Fluido oral': 'Fluid',
    'C-Fluido oral': 'Fluid',
    'B-Fluido oral': 'Fluid',
    'Pulmón, Hígado, Contenido Estómago': 'Mixed',
    'Corazón': 'Tissue',
    'ADN': 'Other',
    'Útero': 'Tissue',
    'ARN': 'Other',
    'Hisopo articular': 'Swab',
    'Digestivo, Hisopo': 'Mixed',
    'Intestino Delgado, Hisopo': 'Mixed',
    'Heces, Intestino Delgado, Intestino Grueso': 'Mixed',
    'J-Fluido oral': 'Fluid',
    'I-Fluido oral': 'Fluid',
    'H-Fluido oral': 'Fluid',
    'G-Fluido oral': 'Fluid',
    'F-Fluido oral': 'Fluid',
    'E-Fluido oral': 'Fluid',
    'D-Fluido oral': 'Fluid',
    'Tonsila, Hisopo Tonsila': 'Mixed',
    'Bazo, Pulmón': 'Mixed',
    'Intesindo Delgado, Intestino Grueso': 'Mixed',
    'Exudado de lengua': 'Fluid',
    'Intestino Delgado, Intestino Grueso, Heces': 'Mixed',
    'Hisopo, Pulmón': 'Mixed',
    'Digestivo, Heces': 'Mixed',
    'Pulmón, Riñón, Linfonodo, Tonsila': 'Mixed',
    'Pulmón, Hígado, Bazo': 'Mixed',
    'Líquido torácico': 'Fluid',
    'Pulmón, Hígado, Contenido Estómago, Placenta': 'Mixed',
    'Hisopo, Líquido Articular': 'Mixed',
    'Lengua, Rabo': 'Mixed',
    'A-Hisopo': 'Swab',
    'B-Hisopo': 'Swab',
    'C-Hisopo': 'Swab',
    'Líquido, Pulmón': 'Mixed',
    'DNA, Pulmón': 'Mixed',
    'Líquido Articular, Articulación': 'Mixed',
    'Lengua, Cordón Umbilical': 'Mixed',
    'Tonsila, Lionfondo': 'Mixed',
    'Pulmón, Placenta, Hígado, Contenido Estómago': 'Mixed',
    'Hisopo, Placenta': 'Mixed',
    'Exudado vaginal': 'Fluid',
    'Digestivo, Estómago': 'Mixed',
    'Púlmón, Hígado, Digestivo': 'Mixed',
    'Articulación, Hisopo': 'Mixed',
    'Pulmón, Hígado, Estómago': 'Mixed',
    'Placenta, Feto': 'Mixed',
    'Agua': 'Other',
    'Intestino Grueso, Hisopo': 'Mixed',
    'Pulmón, Hígado, Bazo, Riñón': 'Mixed',
    'Pulmón, Ganglio': 'Mixed',
    'Colon, Ciego': 'Mixed',
    'Pulmón, Hígado, Placenta': 'Mixed',
    'Digestivo, Calzas': 'Mixed',
    'Hisopo, Intestino Grueso': 'Mixed',
    'Contenido': 'Other',
    'Placenta, Hisopo': 'Mixed',
    'Placenta, Hígado, Pulmón': 'Mixed',
    'Cordón Ubilical, Lengua, Rabo': 'Mixed',
    'Hígado, Pulmón': 'Mixed',
    'Encéfalo, Placenta, Pulmón, Hígado, Contenido Estómago': 'Mixed',
    'Pulmón, Raspado': 'Mixed',
    'Hisopo, Raspado': 'Mixed',
    'Líquido': 'Fluid',
    'Líquido sinovial': 'Fluid',
    'Contenido Digestivo': 'Other',
    'Heces, Calzas': 'Mixed',
    'Hisopo Vaginal, Pulmón, Hígado, Contenido Estómago': 'Mixed',
    'Hisopo, Heces, Digestivo': 'Mixed',
    'Líquido testicular': 'Fluid',
    'Heces, Intestino Delgado': 'Mixed',
    'Rabo, Fluido testicular': 'Mixed',
    'Digestivo, Hígado, Bazo': 'Mixed',
    'Lengua, Rabo, Cordón Umbilical': 'Mixed',
    'Hisopo, Articulación': 'Mixed',
    'Ciego': 'Tissue',
    'Intestino Grueso, Heces': 'Mixed',
    'Placenta, Tejido Fetal': 'Mixed',
    'Placcenta, Feto': 'Mixed',
    'Placenta, Contenido Estómago': 'Mixed',
    'Intestino': 'Tissue',
    'Pulmón, Hígado, Contenido Estomágo': 'Mixed',
    'Pulmón, Hígado, Riñón, Contenido Estómago': 'Mixed',
    'Extracto de riñón': 'Tissue',
    'Rabo, Testículo': 'Mixed',
    'Hígado, Riñón, Bazo': 'Mixed',
    'Linfonodo Mesentérico, Linfonodo Inguinal': 'Mixed',
    'Hisopo, Intestino': 'Mixed',
    'Contenido Digestivo, Heces': 'Mixed',
    'Heces, Hisopo, Digestivo': 'Mixed',
    'Intestino, Hisopo': 'Mixed',
    'Intestino Delgado, Intestino Grueso, Heces, Hisopo': 'Mixed',
    'Articulación, Líquido articular': 'Mixed',
    'Hiospo, Pulmón': 'Mixed',
    'Pulmón, Corazón, Linfonodo': 'Mixed',
    'Tejido': 'Tissue',
    'Suero, Lengua': 'Mixed',
    'Pulmón, Líquido': 'Mixed',
    'Pulmón, Hígado, Encéfalo': 'Mixed',
    'Heces, Hisopo': 'Mixed',
    'Lengua, Rabo, Feto': 'Mixed',
    'Lengua, Rabo, Testículo': 'Mixed',
    'Intestino, Contenido Digestivo': 'Mixed',
    'Hisoppo, Heces': 'Mixed',
    'Intestino, Heces': 'Mixed',
    'Digestivo, Intestino Delgado': 'Mixed',
    'Heces, Digestivo, Estómago': 'Mixed',
    'Pulmón, Estómago': 'Mixed',
    'Líquido placentario': 'Fluid',
    'Pulmón, Hígado, Lengua, Placenta': 'Mixed',
    'Pulmón, Hígado, Lengua, Contenido Estómago': 'Mixed',
    'Pulmón, Hígado, Contenido Estómago, Lengua': 'Mixed',
    'Bazo, Pulmón, Hígado': 'Mixed',
    'Intestino Delgado, Linfonodo Mesentérico': 'Mixed',
    'Linfonodo Inguinal': 'Tissue',
    'Digestivo, Contenido Intestino': 'Mixed',
    'Pulmón, Pericardio, Hisopo': 'Mixed',
    'Tonsila, Linfonodo Inguinal': 'Mixed',
    'Hígado, Bazo, Pulmón, Placenta': 'Mixed',
    'Líquido, Placenta': 'Mixed',
    'Líquido Articular, Hisopo': 'Mixed',
    'Esponja': 'Other',
    'Heces, Intestino': 'Mixed',
    'Pulmón, Hígado, Bazo, Linfonodo': 'Mixed',
    'Pulmón, Hígado, Bazo, Corazón': 'Mixed',
    'Pulmón, Linfonodo, Riñón': 'Mixed',
    'Intestino, Digestivo': 'Mixed',
    'A-Exudado': 'Fluid',
    'B-Exudado': 'Fluid',
    'C-Exudado': 'Fluid',
    'Pulmón, Hisopo Tráquea': 'Mixed',
    'Hisopo Digestivo': 'Swab',
    'Bazo, Riñón, Digestivo, Hígado, Hisopo Digestivo': 'Mixed',
    'Hisopo Pulmón': 'Swab',
    'Hisopo Placenta': 'Swab',
    'Intestino Grueso, Intestino Delgado, Hisopo Digestivo': 'Mixed',
    'Placenta, Feto, Lavado': 'Mixed',
    'Lengua, Placenta, Hisopo Vaginal': 'Mixed',
    'Heces, Intestino Grueso': 'Mixed',
    'Hisopo Digestivo, Heces': 'Mixed',
    'Intestino Delgado, Intestino Grueso, Estómago': 'Mixed',
    'Intestino Delgado, Hisopo Digestivo': 'Mixed',
    'Pulmón, Líquido torácico': 'Mixed',
    'Hisopo Cutáneo': 'Swab',
    'Pulmón, Bazo, Hígado, Linfonodo': 'Mixed',
    'Hisopo Intestino': 'Swab',
    'Lengua, Placenta, Pulmón, Hígado, Contenido Estómago': 'Mixed',
    'Pulmón, Bazo, Linfonodo': 'Mixed',
    'Hisopo Intestino Delgado': 'Mixed',
    'Hisopo Tráquea, Pulmón': 'Mixed',
    'Lengua, Placenta, Pulmón': 'Mixed',
    'Hisopo, Pulmón, Linfonodo': 'Mixed',
    'Intestino Delgado, Intestino Grueso, Heces, Hisopo Digestivo': 'Mixed',
    'Intestino Delgado, Intestino Grueso, Hisopo Digestivo ': 'Mixed',
    'Hígado, Bazo, Riñón': 'Mixed',
    'Hisopo Digestivo, Contenido Digestivo': 'Mixed',
    'Descargas cecales': 'Other',
    'Lengua, Testículo': 'Mixed',
    'Hisopo Intestino, Intestino Grueso': 'Mixed',
    'Pulmón, Rabo': 'Mixed',
    'Lengua, Pulmón, Hígado': 'Mixed',
    'Placenta, Hígado, Pulmón, Digestivo': 'Mixed',
    'Placenta, Pulmón, Contenido Estómago, Hisopo': 'Mixed',
    'Hígado, Bazo': 'Mixed',
    'Hispo Digestivo, Heces': 'Mixed',
    'Intestino Delgado, Intestino Grueso, Digestivo': 'Mixed',
    'Heces, Hisopo Digestivo': 'Mixed',
    'Pulmón, Linfonodo, Tráquea': 'Mixed',
    ' ': 'Missing',
}

# Map the SampleType column using the defined mapping
df1["SampleType"] = df1["SampleType"].map(SampleTypeMapping).fillna("Missing")


# Province mapping
ProvinceMapping = {
    'a coruna': 'ACoruna',
    'alava': 'Alava',
    'albacete': 'Albacete',
    'alicante': 'Alicante',
    'almeria': 'Almeria',
    'asturias': 'Asturias',
    'avila': 'Avila',
    'badajoz': 'Badajoz',
    'balears, illes': 'IslasBaleares',
    'barcelona': 'Barcelona',
    'bizkaia': 'Bizkaia',
    'burgos': 'Burgos',
    'caceres': 'Caceres',
    'cadiz': 'Cadiz',
    'cantabria': 'Cantabria',
    'castellon': 'Castellon',
    'ceuta': 'Ceuta',
    'ciudad real': 'CiudadReal',
    'cordoba': 'Cordoba',
    'coruna, a': 'ACoruna',
    'cuenca': 'Cuenca',
    'gipuzkoa': 'Gipuzkoa',
    'girona': 'Girona',
    'granada': 'Granada',
    'guadalajara': 'Guadalajara',
    'huelva': 'Huelva',
    'huesca': 'Huesca',
    'illes balears': 'IslasBaleares',
    'jaen': 'Jaen',
    'la rioja': 'LaRioja',
    'las palmas': 'LasPalmas',
    'leon': 'Leon',
    'lleida': 'Lleida',
    'lugo': 'Lugo',
    'madrid': 'Madrid',
    'malaga': 'Malaga',
    'melilla': 'Melilla',
    'murcia': 'Murcia',
    'navarra': 'Navarre',
    'nd': 'Missing',
    'ourense': 'Ourense',
    'palencia': 'Palencia',
    'palmas, las': 'LasPalmas',
    'pontevedra': 'Pontevedra',
    'rioja, la': 'LaRioja',
    'salamanca': 'Salamanca',
    'santa cruz de tenerife': 'SantaCruzDeTenerife',
    'segovia': 'Segovia',
    'sevilla': 'Sevilla',
    'soria': 'Soria',
    'sta. cruz tenerife': 'SantaCruzDeTenerife',
    'tarragona': 'Tarragona',
    'teruel': 'Teruel',
    'toledo': 'Toledo',
    'valencia': 'Valencia',
    'valladolid': 'Valladolid',
    'zamora': 'Zamora',
    'zaragoza': 'Zaragoza',

}


df1["Province"] = df1["Province"].map(ProvinceMapping).fillna("Missing")

In [19]:
print(df1['Pathogen'].unique())

['Porcine reproductive and respiratory syndrome virus'
 'Actinobacillus pleuropneumoniae' 'Porcine epidemic diarrhea virus'
 'Brachyspira hyodysenteriae']


In [20]:
print(df1['SampleType'].unique())

['Tissue' 'Swab' 'Mixed' 'Faeces' 'Fluid' 'Blood' 'Other' 'Missing']


In [21]:
# PigAge mapping
PigStageMapping = {

'destetados': 'Weaning',
'cebo': 'Finishing',
'nd': 'Missing',
'reposicion (6 meses)': 'Gilt',
'reposicion': 'Gilt',
'transicion 6 semanas': 'Nursery',
'6-7 semanas': 'Nursery',
'cebo 9 semanas': 'Nursery',
'adultos': 'Other',
'cebo 40 a 150 dias de vida.': 'Mixed',
'transicion (6 semanas)': 'Nursery',
'cebo (final)': 'Finishing',
'reposicion (gilt)': 'Gilt',
'reposicion 16 y 20 semanas de vida': 'Gilt',
'transicion': 'Nursery',
'lact.>15': 'Suckling',
'4 meses.': 'Finishing',
'iberico cebo (5-6 meses)': 'Finishing',
'2-3 meses iberico': 'Finishing',
'lact.>15 (4 sem)': 'Suckling',
'cebo 14 semanas': 'Finishing',
'lechon': 'Suckling',
'adulto': 'Other',
'(iberico 2 meses)': 'Nursery',
'transicion/adultos': 'Mixed',
'lact.': 'Suckling',
'cebo ( 3 meses)': 'Finishing',
'transicion (2 meses)': 'Nursery',
'transicion (7 semanas)': 'Nursery',
'6 semanas': 'Nursery',
'transicion (2-3 meses)': 'Nursery',
'lact. (2 dias)': 'Suckling',
'transicion (5 semanas)': 'Nursery',
'cebo (4 meses)': 'Finishing',
'transicion (9-10 semanas)': 'Nursery',
'transicion (6-7 semanas)': 'Nursery',
'desconocido': 'Missing',
'transicion 7 semanas': 'Nursery',
'cebo (100kg)': 'Finishing',
'lact. (20 dias)': 'Suckling',
'cebo ( 2,5 meses)': 'Finishing',
'cebo (futura reproductora)': 'Gilt',
'transicion 25 dias': 'Weaning',
'cebo 100kg': 'Finishing',
'transicion 8 semanas': 'Nursery',
'cebo (70kg)': 'Finishing',
'lact. (<15 dias)': 'Suckling',
'cebo ( 1,5 meses)': 'Nursery',
'cebo ( 30-150kg.)': 'Finishing',
'cebo ( 13 semanas)': 'Finishing',
'entrada a cebo': 'Finishing',
'cebo ( meses)': 'Finishing',
'transicion (30kg)': 'Finishing',
'cebo ( 4 meses)': 'Finishing',
'cebo 45-50 kg.': 'Finishing',
'cuarentena': 'Other',
'transicion (8 semanas)': 'Nursery',
'cebo (30 kg)': 'Finishing',
'cebo (8 semanas)': 'Finishing',
'cebo (5 meses)': 'Finishing',
'transicion (4-5-5-7 semanas)': 'Nursery',
'transicion (3-5-6-7 semanas)': 'Nursery',
'transicion (3-5-7 semanas)': 'Nursery',
'lact. transicion': 'Mixed',
'cebo (3 meses)': 'Finishing',
'cebo (16 semanas)': 'Finishing',
'cebo (50kg)': 'Finishing',
'cebo (18 kg a entrada)': 'Finishing',
'transicion (5 semanas de vida)': 'Nursery',
'transicion (9 semanas de vida)': 'Nursery',
'transicion (12 semanas de vida)': 'Finishing',
'cebo (entrada a cebo)': 'Finishing',
'3-5-7-9 semanas': 'Nursery',
'transicion (25-30 dias)': 'Weaning',
'lact. transicion 2-5 semanas': 'Mixed',
'cebo ( 6 meses)': 'Finishing',
'lact. transicion (40 dias)': 'Nursery',
'primerizas': 'Sow',
'cebo (110 kg)': 'Finishing',
'recria': 'Gilt',
'transicion (6 kg)': 'Weaning',
'cebo (12/13 semanas)': 'Finishing',
'cebo (entrada)': 'Finishing',
'cebo (30 kg) iberico': 'Finishing',
'cebo (50-60kg). (16-17 sem).': 'Finishing',
'transicion (7 sem)': 'Nursery',
'transicion/ cuarentena': 'Mixed',
'cebo vida (90kg)': 'Finishing',
'cebo (13 sem)': 'Finishing',
'reposicion (14-20 semanas)': 'Gilt',
'cebo (15 sem)': 'Finishing',
'cebo (2 meses)': 'Nursery',
'transicion (5 kg)': 'Weaning',
'adultos (1 ano)': 'Other',
'cebo 20 kg.': 'Finishing',
'cebo (1 meses)': 'Nursery',
'cebo (1,5 meses) entrada a cebo 7 kg': 'Nursery',
'transicion (10-30 kg).': 'Mixed',
'cebo (50 kg)': 'Finishing',
'3-6-10 semanas': 'Nursery',
'cebo (6 semanas )': 'Nursery',
'cebo vida (50 kg)': 'Finishing',
'11 semanas': 'Finishing',
'cebo (3,5 meses)': 'Finishing',
'primerizas,adultos': 'Mixed',
'lact. ( 25 dias)': 'Suckling',
'transicion (6-8 semanas)': 'Nursery',
'3-6-9 semanas': 'Nursery',
'adultos (10 meses)': 'Other',
'lact. (25 dias)': 'Suckling',
'lact. (25 dias de vida)': 'Suckling',
'5 semanas': 'Nursery',
'3 semanas': 'Nursery',
'9 semanas': 'Nursery',
'4 semanas': 'Nursery',
'cebo (entrada cebo)': 'Finishing',
'lact. (18 dias)': 'Suckling',
'lact. adulto': 'Mixed',
'transicion (destetados)': 'Weaning',
'cebo (70 kg)': 'Finishing',
'cebo (90 kg)': 'Finishing',
'cebo (1 ano)': 'Finishing',
'cebo (1 mes)': 'Nursery',
'transicion (7-8 semanas)': 'Nursery',
'cebo (110 dias en cebo)': 'Finishing',
'transicion (1 semana)': 'Nursery',
'cebo (14 semanas)': 'Finishing',
'lact. (>15 dias)': 'Suckling',
'cebo (17 sem)': 'Finishing',
'cebo (13-14 semanas)': 'Finishing',
'cebo (17 semanas)': 'Finishing',
'transicion (5 y 7 semanas)': 'Nursery',
'cebo (100 kg)': 'Finishing',
'transicion (40 dias)': 'Nursery',
'lact. (3 semanas)': 'Suckling',
'lact. (5 kg)': 'Suckling',
'cebo (90-100kg)': 'Finishing',
'transicion 3 y 5 semanas': 'Nursery',
'cebo (40 kg)': 'Finishing',
'cebo ( meses) transicion': 'Mixed',
'reposicion (2o parto)': 'Sow',
'cebo (12 semanas)': 'Finishing',
'cebo (5, 9 y 17 semanas)': 'Mixed',
'cebo (90 dias)': 'Finishing',
'cebo (90-100 kg)': 'Finishing',
'transicion (5 y 8 semanas)': 'Nursery',
'cebo (18 semanas )': 'Finishing',
'transicion (5-7-9 semanas)': 'Nursery',
'reposicion (60-70 kg)': 'Gilt',
'transicion (30 dias)': 'Nursery',
'transicion (10 semanas)': 'Nursery',
'cebo (3-4 meses)': 'Finishing',
'lact. ( dias)': 'Suckling',
'transicion (9 sem)': 'Nursery',
'lact. (14 dias)': 'Suckling',
'transicion (3 semanas)': 'Weaning',
'cebo (12 meses)': 'Finishing',
'cebo (120 kg)': 'Finishing',
'transicion (60 kg)': 'Finishing',
'transicion (25kg)': 'Finishing',
'reproductoras': 'Sow',
'cebo (22 semanas, 100 kg)': 'Finishing',
'cebo (7 meses, 80-90 kg)': 'Finishing',
'cebo (60 kg)': 'Finishing',
'transicion (40-50 dias)': 'Nursery',
'lact. cebo, transicion': 'Mixed',
'adultos transicion (42 dias)': 'Mixed',
'adultos transicion': 'Mixed',
'transicion ( 7 semanas)': 'Nursery',
'transicion (3-6-9-11 semanas)': 'Nursery',
'transicion (4 meses)': 'Finishing',
'lactantes': 'Suckling',
'gestantes': 'Sow',
'transicion (9 semanas)': 'Nursery',
'transicion (42 dias)': 'Nursery',
'cebo (81 y 91 dias)': 'Finishing',
'cebo (180 kg)': 'Finishing',
'transicion (3 y 6 semanas postdestete)': 'Nursery',
'cebo (8-12 semanas)': 'Finishing',
'destetado': 'Weaning',
'cebo (2,5 meses)': 'Finishing',
'cebo (vida)': 'Finishing',
'cebo (hibridas)': 'Finishing',
'postweaning': 'Nursery',
'transicion (4 y 5 semanas de vida)': 'Nursery',
'cebo (10-30kg.)': 'Mixed',
'cebo ( 24 semanas de vida)': 'Finishing',
'transicion (3 meses)': 'Nursery',
'cebo (2 meses en cebo)': 'Finishing',
'cebo (4-5 meses)': 'Finishing',
'transicion cebo reposicion': 'Mixed',
'transicion (5 semanas postdestete)': 'Nursery',
'transicion (1 semanas postdestete)': 'Nursery',
'transicion (6, 7 y 9 semanas)': 'Nursery',
'cebo (100 dias)': 'Finishing',
'transicion (6 semanas de vida)': 'Nursery',
'transicion (4-9 semanas)': 'Nursery',
'cebo (115 dias de vida)': 'Finishing',
'cebo (15 dias en cebo)': 'Finishing',
'transicion (3-4 semanas)': 'Weaning',
'transicion (3-6-9 semanas)': 'Nursery',
'lact. ( 21 dias)': 'Suckling',
'cebo (1 mes y medio en cebo)': 'Finishing',
'destete': 'Weaning',
'cebo (20 sem)': 'Finishing',
'cebo ( 3-4 meses)': 'Finishing',
'cebo ( 10 sem.)': 'Finishing',
'lact. (<15 dias) adultos': 'Mixed',
'cebo (100 dias )': 'Finishing',
'transicion (12 kg)': 'Nursery',
'cebo vida': 'Finishing',
'transicion (2-3 sem postdestete)': 'Nursery',
'cebo (3 meses) adultos': 'Mixed',
'cebo (6 meses)': 'Finishing',
'cebo ( 3,5 meses)': 'Finishing',
'reposicion (4-5 meses) transicion (8 semanas)': 'Mixed',
'lact. (28 dias)': 'Weaning',
'transicion (15 dias postdestete)': 'Nursery',
'adultos (nuliparas)': 'Gilt',
'transicion (20 y 28 dias de vida)': 'Weaning',
'cebo (100 kg.)': 'Finishing',
'transicion (15 kg)': 'Nursery',
'lact (final lactacion)': 'Suckling',
'lact. (12 dias)': 'Suckling',
'transicion (10-11 semanas de vida)': 'Nursery',
'transicion (4, 5 y 6 semanas de vida)': 'Nursery',
'transicion (4-5-6-7 semanas de vida)': 'Nursery',
'transicion (6-7 semanas de vida)': 'Nursery',
'transicion (5 y 6 semanas de vida)': 'Nursery',
'transicion (5-6 y 7 semanas de vida)': 'Nursery',
'lact. (24 dias)': 'Weaning',
'transicion (6-9 semanas)': 'Nursery',
'cebo (8-20 semanas)': 'Finishing',
'lact. ( >21 dias)': 'Suckling',
'transicion (28 dias de vida)': 'Weaning',
'cebo (3, 3,5, 4 y 5 meses)': 'Finishing',
'lact. ( <15 dias)': 'Suckling',
'transicion cebo': 'Mixed',
'transicion (60 dias)': 'Nursery',
'lact. (0 dias) transicion': 'Mixed',
'transicion (5-6 semanas de vida)': 'Nursery',
'transicion (10 semanas de vida)': 'Nursery',
'nuliparas multiparas': 'Mixed',
'transicion reposicion': 'Mixed',
'cebo (3-6 meses)': 'Finishing',
'transicion (3,5 meses; 20kg)': 'Finishing',
'transicion (8-9 semanas)': 'Nursery',
'transicion (6-7 sem de vida)': 'Nursery',
'lechones (15 kg)': 'Nursery',
'lechones (30 kg)': 'Finishing',
'cebo (50, 70 y 100 kg.)': 'Finishing',
'transicion (1 sem. postdestete)': 'Nursery',
'cebo (22-23 semanas)': 'Finishing',
'lact. (35 dias)': 'Weaning',
'transicion (3, 6, 9 semanas de vida)': 'Nursery',
'iberico cebo (3 meses)': 'Finishing',
'cebo (5 meses, 65kg)': 'Finishing',
'cebo (20 kg)': 'Finishing',
'cebo (55 kg)': 'Finishing',
'cebo (20-25 kg)': 'Finishing',
'transicion (3 dias postdestete)': 'Nursery',
'iberico (6 meses) cebo (55 meses) lact. (22-24 dias)': 'Mixed',
'lact. ( 21-22 dias)': 'Suckling',
'cebo vida iberico': 'Finishing',
'lact. (25 dias) transicion': 'Mixed',
'transicion (8 semanas de vida)': 'Nursery',
'transicion 4 semanas de vida': 'Weaning',
'cebo ( 5 meses)': 'Finishing',
'cebo (150 dias de vida)': 'Finishing',
'transicion (21-24 dias vida)': 'Weaning',
'adultos lact.': 'Mixed',
'cebo (fase inicial cebo)': 'Finishing',
'transicion (4 semanas)': 'Weaning',
'transicion (80 dias)': 'Finishing',
'transicion (3-6 semanas postdestete)': 'Nursery',
'transicion (5-7 semanas)': 'Nursery',
'cebo (1,5 meses)': 'Nursery',
'transicion (70 dias), cebo (5 meses)': 'Mixed',
'lact. (23 dias)': 'Suckling',
'cebo ( 2 meses y medio)': 'Finishing',
'cebo (21 dias)': 'Weaning',
'transicion (1 semana postdestete)': 'Nursery',
'transicion (36, 47 y 83 dias de vida)': 'Mixed',
'9 semanas de vida': 'Nursery',
'cebo (12 semanas de vida)': 'Finishing',
'cebo (4 meses de vida)': 'Finishing',
'm150718': 'Other',
'reposicion (nuliparas)': 'Gilt',
'transicion (6 y 9 semanas)': 'Nursery',
'cebo 15 kg': 'Finishing',
'cebo (11 semanas de vida)': 'Finishing',
'transicion (1, 2 y 3 semans)': 'Nursery',
'transicion (4 semanas de vida)': 'Weaning',
'lact. (21 dias)': 'Suckling',
'transicion (7 semanas de vida)': 'Nursery',
'cebo (45 kg)': 'Finishing',
'adultos (reproductoras)': 'Sow',
'cebo (15 semanas)': 'Finishing',
'cebo (3 meses en cebadero)': 'Finishing',
'cebo ( 10 semanas)': 'Nursery',
'transicion (7 dias destetados)': 'Nursery',
'lact. (4 y 7 semanas)': 'Nursery',
'transicion adultos': 'Mixed',
'verracos (>12 meses)': 'Boar',
'transicion (6, 8 y 9 semanas)': 'Nursery',
'transicion (12 semanas)': 'Finishing',
'reposicion (5 meses)': 'Gilt',
'cebo 2 meses': 'Finishing',
'cebo (120 dias)': 'Finishing',
'destete transicion (4 semanas)': 'Weaning',
'transicion (2-3 semanas)': 'Nursery',
'transicion (4, 5, 6 y 8 semanas)': 'Nursery',
'adultos (multipara)': 'Sow',
'transicion (3-9 semanas)': 'Nursery',
'transicion 30 dias': 'Weaning',
'transicion (3, 6 y 7 semanas)': 'Nursery',
'lactantes ( 14-21 dias)': 'Suckling',
'transicion 5 semanas': 'Nursery',
'7 semanas': 'Nursery',
'cebo (10 sem y 4 meses)': 'Finishing',
'transicion (60 dias de vida)': 'Nursery',
'transicion (70 dias)': 'Nursery',
'adultos >12 meses': 'Other',
'cebo (70 dias)': 'Nursery',
'transicion (6 y 8 semanas)': 'Nursery',
'destetado (3 semanas)': 'Weaning',
'destetado (40 dias de vida)': 'Nursery',
'cebo (3 meses vida)': 'Finishing',
'cebo (18 kg.)': 'Nursery',
'cebo (inicio)': 'Finishing',
'cebo (inicio, 15 kg)': 'Nursery',
'cebo ( 11 semanas) 25-30kg.': 'Finishing',
'destetado 8 semanas vida': 'Nursery',
'transicion (50 kg)': 'Finishing',
'cebo ( 65 dias)': 'Nursery',
'adultos verraco >12m': 'Boar',
'verraco >12 meses': 'Boar',
'transicion (3-6-8/9 semanas de vida)': 'Nursery',
'cebo (18 kg pv)': 'Nursery',
'4-5-6-7-8-9 semanas': 'Nursery',
'cebo (4,5 meses)': 'Finishing',
'cebo ( 2 semanas)': 'Finishing',
'transicion (3 semanas de vida)': 'Weaning',
'transicion (2-3, 4 y 5 semanas de vida)': 'Mixed',
'cebo (50-60 kg)': 'Finishing',
'transicion (3, 5 y 8 semanas)': 'Mixed',
'cebo (60 dias)': 'Nursery',
'4 semanas de vida': 'Weaning',
'transicion (17 kg)': 'Nursery',
'transicion 7 y 8 semanas': 'Nursery',
'transicion 8 y 9 semanas de vida': 'Nursery',
'transicion 6-9 semanas': 'Nursery',
'adultos (>12 meses)': 'Other',
'cebo ( >18 kg)': 'Finishing',
'adultos verraco >12 meses': 'Boar',
'cebo ( 20 kg)': 'Finishing',
'lactantes ( dias) transicion': 'Mixed',
'cebo (13 y 24 semanas)': 'Finishing',
'lactantes transicion': 'Mixed',
'cebo (entrada hace 10 dias)': 'Finishing',
'transicion 1 y 4 semanas de destetados': 'Nursery',
'destetado (4-5 semanas postdestete)': 'Nursery',
'cebo ( 15 kg.)': 'Nursery',
'cebo (6,5 meses)': 'Finishing',
'cebo (3 meses de vida)': 'Finishing',
'lactantes (21 dias)': 'Suckling',
'cebo (>12 meses)': 'Finishing',
'reposicion 70 kg.': 'Gilt',
'lactantes (2-3 semanas)': 'Suckling',
'transicion (5, 6, 7 y 8 semanas)': 'Nursery',
'cebo (0,5 meses)': 'Suckling',
'verracos > 12 meses': 'Boar',
'cebo (3 semanas en cebo)': 'Finishing',
'lactantes (21 dias) transicion': 'Mixed',
'lactantes (17-20 dias)': 'Suckling',
'transicion (5-6 y 7-8 semanas)': 'Nursery',
'adultos (5-6 meses)': 'Sow',
'transicion cebo ( 50 kg.)': 'Mixed',
'cebo ( 60 kg.)': 'Finishing',
'cebo (45-50 kg)': 'Finishing',
'reposicion adultos primerizas': 'Mixed',
'primerizas reposicion': 'Mixed',
'cebo (1.5 meses)': 'Nursery',
'transicion (3 y 4 semanas)': 'Weaning',
'transicion 4 sem': 'Weaning',
'transicion (40, 45 y 50 dias, 8 semanas)': 'Nursery',
'transicion (3, 4, 5, 6 semanas)': 'Mixed',
'destetado (3 semanas postdestete)': 'Nursery',
'destetado cebo (3 meses)': 'Finishing',
'cebo ( 2 meses)': 'Nursery',
'lactantes (19 dias)': 'Suckling',
'transicion 4-5 semanas': 'Nursery',
'cebo (8 meses)': 'Finishing',
'cebo (10 meses)': 'Finishing',
'cebo ( 1 meses)': 'Nursery',
'destetado 1-2-3 semanas': 'Nursery',
'destetado (9 semanas)': 'Nursery',
'cebo (2 meses vida)': 'Nursery',
'cebo (1-2 meses)': 'Nursery',
'cebo (3.5 meses)': 'Finishing',
'lactantes ( 10-12 dias)': 'Suckling',
'lactantes (> 15dias)': 'Suckling',
'transicion (5-6 sem vida)': 'Nursery',
'transicion (2-8 sem vida)': 'Mixed',
'transicion y adultos': 'Mixed',
'cebo (1 y 3 meses)': 'Nursery',
'cebo (75-80 kg)': 'Finishing',
'transicion (6 y 7 semanas)': 'Nursery',
'transicion (3,6,9 semanas)': 'Nursery',
'cebo (50, 60, 70 y 80 dias)': 'Mixed',
'cebo (80 dias)': 'Finishing',
'destetado cebo': 'Nursery',
'transicion (21, 42 y 63 dias)': 'Mixed',
'transicion (3, 5 y 6 sem)': 'Mixed',
'3-9 semanas transicion': 'Mixed',
'cebo (7 y 8 semanas)': 'Nursery',
'lactantes (18 dias)': 'Suckling',
'cebo (24 kg)': 'Finishing',
'nuliparas (7,5 meses)': 'Gilt',
'cebo (25 y 80 kg)': 'Finishing',
'lactantes (4-7 y 18 dias)': 'Suckling',
'transicion (5-6/ 7-8 semanas vida)': 'Nursery',
'transicion primerizas': 'Mixed',
'transicion (4-6 semanas de vida)': 'Nursery',
'destetado reposicion adultos': 'Mixed',
'cebo (30, 60 y 100 kg)': 'Finishing',
'transicion (3y 4 semanas) cebo (7 y 8 semanas)': 'Mixed',
'cebo (80-100 kg)': 'Finishing',
'primerizas adultos': 'Sow',
'reposicion destetados lactantes': 'Mixed',
'cebo (15-20 kg.)': 'Nursery',
'transicion (3 meses de vida)': 'Finishing',
'transicion (7-8 semanas de vida)': 'Nursery',
'transicion (7 y 8 semanas)': 'Nursery',
'transicion (4 y 6 semanas)': 'Nursery',
'transicion (40, 50 y 75 dias)': 'Nursery',
'transicion (5-9 semanas de vida)': 'Nursery',
'cebo (1 semana)': 'Nursery',
'cebo (10-15 kg.)': 'Nursery',
'cebo ( 20-25 kg meses)': 'Finishing',
'cebo ( 20-25 kg)': 'Finishing',
'cebo (15 dias)': 'Nursery',
'lactantes (26 dias)': 'Suckling',
'cebo (25 kg)': 'Finishing',
'transicion (45-60-80 dias)': 'Nursery',
'transicion (30-45-60 dias)': 'Nursery',
'transicion (5,6 y 7 semanas)': 'Nursery',
'cebo (140 dias)': 'Finishing',
'destetado (21 dias)': 'Weaning',
'verracos': 'Boar',
'transicion (1, 2, 5 y 9 semanas)': 'Mixed',
'cebo (25-30 kg)': 'Finishing',
'lactantes (20 dias)': 'Suckling',
'lactantes (5 y 10 dias)': 'Suckling',
'cebo (25-30 kg.)': 'Finishing',
'lactantes (2 dias)': 'Suckling',
'cebo (15-16 kg.)': 'Nursery',
'adultos > 12 meses': 'Other',
'cebo (5 semanas)': 'Nursery',
'cebo (15 kg.)': 'Nursery',
'destetado (10 -17 dias postdestete)': 'Nursery',
'cebo (1 mes en cebo)': 'Nursery',
'cebo (3 meses en cebo)': 'Finishing',
'cebo (22 semanas)': 'Finishing',
'transicion (9-11 semanas)': 'Nursery',
'transicion (23 kg)': 'Finishing',
'destetado,transicion': 'Nursery',
'otra': 'Other',
'3-5-7 semanas': 'Nursery',
'transicion (3, 5 y 7 semanas)': 'Nursery',
'transicion (28-49 y 63 dias)': 'Nursery',
'transicionn (28-49 y 63 dias)': 'Nursery',
'transicion 10 kg': 'Nursery',
'transicion (6,9,12 semanas)': 'Nursery',
'destetado (20-30 dias)': 'Weaning',
'transicion (30 dias de vida)': 'Nursery',
'lactantes (10 dias)': 'Suckling',
'nuliparas': 'Gilt',
'lactantes (21-25 dias)': 'Suckling',
'lactantes (2-35 dias)': 'Mixed',
'lactantes (10 kg)': 'Nursery',
'cebo ( 3 meses) reproductoras': 'Mixed',
'cebo (15 kg)': 'Nursery',
'lactantes destetado': 'Mixed',
'transicion (3, 5, 7 y 9 semanas de vida)': 'Nursery',
'adultos lactantes (nm)': 'Mixed',
'transicion (10 kg)': 'Nursery',
'cebo ( 80 kg.)': 'Finishing',
'transicion 8semanas': 'Nursery',
'transicion (4, 6 y 8 semanas)': 'Nursery',
'lactantes (25 dias)': 'Weaning',
'cebo (10 semanas)': 'Nursery',
'transicion (4-5 semanas)': 'Nursery',
'8-9-10 semanas': 'Nursery',
'transicion 3 semanas': 'Weaning',
'cebo (11 semanas)': 'Finishing',
'transicion (10-11 semanas)': 'Nursery',
'transicion (30-50 dias de vida)': 'Nursery',
'lactantes (10-20 dias)': 'Mixed',
'cebo (18-20 kg)': 'Nursery',
'transicion (4 y 8 semanas de vida)': 'Nursery',
'cebo (35 kg)': 'Finishing',
'destetado (40-60 dias vida)': 'Nursery',
'transicion (42 dias vida)': 'Nursery',
'transicion (65 dias vida)': 'Nursery',
'transicion (3 y 5 semanas vida)': 'Mixed',
'cebo (2 5 meses)': 'Nursery',
'cebo (80 kg)': 'Finishing',
'destetado (5-10 semanas vida)': 'Nursery',
'cebo (16 semanas vida)': 'Finishing',
'lactantes (24 dias)': 'Weaning',
'transicion (35 dias vida)': 'Nursery',
'lactantes (28 dias)': 'Weaning',
'transicion (5- 8 semanas vida)': 'Nursery',
'lactantes (21 dias) transicion 5 semanas': 'Mixed',
'cebo (3 y 5 meses)': 'Finishing',
'cebo (6 semanas)': 'Nursery',
'cebo (9 semanas)': 'Nursery',
'lactantes ( dias)': 'Suckling',
'cebo (2.5 meses)': 'Nursery',
'cebo 19 semanas': 'Finishing',
'transicion (45 dias de vida)': 'Nursery',
'lactante (21 dias)': 'Weaning',
'cebo reposicion': 'Mixed',
'cebo ( 26 semanas)': 'Finishing',
'cebo ( 12-13 semanas)': 'Finishing',
'transicion (4, 5 y 6 semanas)': 'Nursery',
'cebo ( 60 kg)': 'Finishing',
'cebo (7 meses)': 'Finishing',
'transicion (2, 6 y 7 semanas)': 'Mixed',
'cebo ( 18 kg)': 'Nursery',
'transicion (2, 3 y 5 semanas)': 'Mixed',
'cebo ( 50 kg)': 'Finishing',
'cebo (,5 meses )': 'Finishing',
'cebo 5 meses': 'Finishing',
'cebo 4 meses': 'Finishing',
'adultos lactantes (1 dias)': 'Mixed',
'lactantes ( 7-15 dias)': 'Suckling',
'lactantes,transicion': 'Mixed',
'transicion (8 semanas vida)': 'Nursery',
'lactantes ( 12 dias)': 'Suckling',
'transicion 9 semanas': 'Nursery',
'8 semanas': 'Nursery',
'lactantes ( 15-21 dias)': 'Suckling',
'cebo ( 40 kg.)': 'Finishing',
'transicion 5sdv': 'Nursery',
'destetado (2-3 semanas vida)': 'Weaning',
'lactantes (15 dias)': 'Suckling',
'cebo (13 semanas)': 'Finishing',
'cebo ( 20kg)': 'Finishing',
'cebo ( 6-10 semanas)': 'Nursery',
'destetado (60 dias)': 'Nursery',
'lactantes (3 semanas)': 'Suckling',
'reposicion lactantes (26 dias)': 'Mixed',
'cebo (15-70kg)': 'Mixed',
'lactantes ( 14 dias)': 'Suckling',
'cebo (110 dias)': 'Finishing',
'transicion (2 semanas)': 'Nursery',
'destetado adultos': 'Mixed',
'adultos (verraco)': 'Boar',
'transicion 1 semana posdestete': 'Nursery',
'transicion 3 y 4 sem posdestete': 'Nursery',
'destetado transicion': 'Nursery',
'cebo (16 kg)': 'Nursery',
'destetado transicion (6 y 9 semanas)': 'Nursery',
'destetado, transicion (9-10 semanas)': 'Nursery',
'7 y 8 semanas': 'Nursery',
'transicion (4, 5, 6, 7 y 9 semanas)': 'Nursery',
'lactantes ( 21 dias)': 'Weaning',
'lactantes (26 dias) adultos': 'Mixed',
'transicion 24-52 dias': 'Mixed',
'entrada cebo': 'Nursery',
'transicion ( 9 semanas)': 'Nursery',
'cebo (25 meses) lactantes ( 28 dias)': 'Mixed',
'lactantes ( 1-10-20 dias) destetado (5 semanas de vida)': 'Mixed',
'lactantes ( 28 dias)': 'Weaning',
'lactantes (17 dias)': 'Suckling',
'reposicion 9 meses': 'Gilt',
'cebo ( 50 kg. )': 'Finishing',
'adultos 90 kg': 'Other',
'transicion (8-10 semanas)': 'Nursery',
'transicion (6 semanas vida)': 'Nursery',
'transicion (1, 2 y 5 semanas)': 'Mixed',
'cebo (16-17 semanas)': 'Finishing',
'lactantes (14 dias)': 'Suckling',
'cebo (8-10 semanas)': 'Nursery',
'cebo ( 1 mes)': 'Nursery',
'lactantes ( 20 dias)': 'Suckling',
'transicion 28-63 dias': 'Nursery',
'transicion (15-18 kg) cebo (7 meses)': 'Mixed',
'cebo ( 11 meses)': 'Finishing',
'cebo (5-6 meses)': 'Finishing',
'destetado (10 semanas de vida)': 'Nursery',
'destetado (9 semanas de vida)': 'Nursery',
'adultos lactantes ( 15 dias) transicion 3-9 semanas': 'Mixed',
'cebo (80 kg.)': 'Finishing',
'lactantes (1 semanas)': 'Suckling',
'cebo (3 semanas)': 'Nursery',
'destetado (33 dias)': 'Nursery',
'transicion 7w': 'Nursery',
'transicion 6-7 semanas': 'Nursery',
'transicion 63d': 'Nursery',
'cebo (59 dias)': 'Nursery',
'cebo (56 y 45 dias)': 'Nursery',
'cebo ( 9 semanas)': 'Nursery',
'lactantes ( 7 dias)': 'Suckling',
'destetado (2-3 semanas)': 'Weaning',
'transicion 4 semanas': 'Weaning',
'cebo (30 kg.)': 'Finishing',
'destetado (7 semanas de vida)': 'Nursery',
'transicion (1, 2, 3, 4 y 5 semanas)': 'Mixed',
'destetado (30 y 70 dias)': 'Nursery',
'transicion cebo (42 dias)': 'Mixed',
'destetado (1 semana)': 'Weaning',
'cebo (18 semanas)': 'Finishing',
'destetado (28 dias vida)': 'Weaning',
'transicion (28 dias vida)': 'Weaning',
'recria transicion': 'Mixed',
'transicion (26 dias en transicion)': 'Nursery',
'destetado (28 dias)': 'Weaning',
'lactantes (21 dias) destetado': 'Weaning',
'transicion 3-5 semanas': 'Mixed',
'adultos (3 anos)': 'Other',
'destetado (4 sem vida)': 'Weaning',
'transicion (7-8 sem vida)': 'Nursery',
'adultos (3-4 meses)': 'Other',
'destetado (30-60 dias)': 'Nursery',
'post destetado': 'Nursery',
'reposicion transicion': 'Mixed',
'transicion 11 semanas': 'Finishing',
'destetado (4,5,6 y 7 sem)': 'Nursery',
'destetado (40 dias vida)': 'Nursery',
'destetado (7-14 dias )': 'Nursery',
'destetado (45 dias)': 'Nursery',
'destetado (35 dias)': 'Nursery',
'destetado (30 dias)': 'Nursery',
'reproductoras (90 dias gestacion)': 'Sow',
'transicion 10 semanas': 'Nursery',
'cebo (15-16 semanas)': 'Finishing',
'cebo (6-8-10 semanas)': 'Nursery',
'cebo (5-7-9 semanas)': 'Nursery',
'cebo (4-6-8-10 semanas)': 'Nursery',
'cebo ( 60 dias de vida)': 'Nursery',
'cebo ( 3meses)': 'Finishing',
'destetado (20 dias)': 'Weaning',
'cebo (28 dias)': 'Weaning',
'cebo (66 dias)': 'Nursery',
'transicion 28d': 'Weaning',
'cebo ( 9 meses)': 'Finishing',
'adultos reproductoras': 'Sow',
'cebo (6-7 semanas)': 'Nursery',
'16 semanas': 'Finishing',
'lactantes (3 dias)': 'Suckling',
'cebo (45 dias)': 'Nursery',
'cebo ( 9-15 semanas)': 'Finishing',
'cebo ( 30-40 kg)': 'Finishing',
'cebo (30-40 kg)': 'Finishing',
'cebo ( 17w)': 'Finishing',
'lactantes (14 dias) transicion': 'Mixed',
'cebo (20 semanas)': 'Finishing',
'cebo (25 semanas)': 'Finishing',
'cebo adultos': 'Mixed',
'transicion 0-3-6 semanas': 'Mixed',
'lactantes ( 23 dias)': 'Weaning',
'lactantes (20 kg)': 'Suckling',
'cebo ( 30 kg.)': 'Finishing',
'destetado (6 kg)': 'Weaning',
'reproductoras (30 dias gestacion)': 'Sow',
'cebo ( 80 kg )': 'Finishing',
'cebo (8 meses - 100 kg)': 'Finishing',
'transicion 4-5-9 semanas': 'Nursery',
'lactantes (22 dias)': 'Weaning',
'adultos (1,5 anos)': 'Other',
'cebo (6 kg.)': 'Nursery',
'cebo ( 60-70 kg.)': 'Finishing',
'destetado (60-70 dias vida)': 'Nursery',
'lactantes (1 dias) destetado': 'Weaning',
'destetado (6 y 9 semanas)': 'Nursery',
'reproductoras/es': 'Other',
'cebo/engorde': 'Finishing',
'cebo/engorde (11 sem)': 'Finishing',
'destetado/transicion': 'Mixed',
'cebo/engorde (2 meses)': 'Nursery',
'cebo/engorde ( 7 meses sem/mes/ano)': 'Finishing',
'cebo/engorde ( 2 meses)': 'Nursery',
'cebo/engorde (35 dias)': 'Finishing',
'lactantes ( 30 dias)': 'Weaning',
'cebo/engorde (1 mes)': 'Nursery',
'recria/reposicion': 'Gilt',
'cebo/engorde ( 3 meses)': 'Finishing',
'cebo/engorde ( sem/mes/ano)': 'Finishing',
'cebo/engorde (2 meses, <60kg)': 'Finishing',
'cebo/engorde (2 semanas, <30 kg)': 'Finishing',
'cebo/engorde (3 meses)': 'Finishing',
'cebo/engorde ( 3 meses': 'Finishing',
'cebo/engorde (2,5 meses)': 'Nursery',
'cebo/engorde ( 16 sem)': 'Finishing',
'destetado/transicion (3-6-9 sem)': 'Mixed',
'cebo (1,5 meses, <60 kg)': 'Finishing',
'destetado/transicion (2 y 4 semanas)': 'Mixed',
'cebo/engorde (10 sem)': 'Nursery',
'cebo/engorde (3 mes)': 'Finishing',
'destetado/transicion (6 y 9 semanas) primerizas': 'Mixed',
'cebo/engorde (1 sem)': 'Nursery',
'destetado/transicion (2 meses vida)': 'Nursery',
'lactantes ( 22 dias)': 'Weaning',
'cebo/engorde (2 sem/mes)': 'Nursery',
'cebo/engorde 20 kg.': 'Finishing',
'cebo/engorde (15 dias)': 'Nursery',
'cebo/engorde (6 meses)': 'Finishing',
'cebo/engorde ( 3 mes)': 'Finishing',
'destetado/transicion (9 semanas)': 'Nursery',
'destetado/transicion (7 semanas)': 'Nursery',
'destetado/transicion (6 semanas)': 'Nursery',
'destetado/transicion 7 semanas': 'Nursery',
'cebo/engorde ( 4 meses )': 'Finishing',
'destetado/transicion adultos': 'Mixed',
'cebo/engorde (4meses)': 'Finishing',
'destetado/transicion 5 semanas': 'Nursery',
'destetado/transicion 5 sem': 'Nursery',
'cebo/engorde ( 1 meses)': 'Nursery',
'cebo/engorde ( 16 semanas)': 'Finishing',
'destetado/transicion 6 semanas': 'Nursery',
'cebo/engorde ( 4 meses)': 'Finishing',
'recria/reposicion 5-6 meses': 'Gilt',
'cebo/engorde (30 kg.)': 'Finishing',
'cebo/engorde ( 6 meses)': 'Finishing',
'cebo/engorde (4 meses)': 'Finishing',
'cebo/engorde ( 3 sem)': 'Finishing',
'cebo/engorde ( 10 semanas)': 'Nursery',
'transicion 30-35 dias': 'Nursery',
'cebo/engorde (4 mes)': 'Finishing',
'destetado/transicion (3 semanas)': 'Weaning',
'adultos 18 meses': 'Other',
'semanas 44, 45, 46 y 47': 'Finishing',
'cebo/engorde ( 8 sem)': 'Nursery',
'lactantes ( 15 dias)': 'Suckling',
'primalas': 'Gilt',
'reproductoras/es, adultos': 'Other',
'cebo/engorde (1 meses)': 'Nursery',
'cebo/engorde (5 semanas)': 'Nursery',
'cebo/engorde (3 semanas)': 'Nursery',
'cebo/engorde (6 semanas)': 'Nursery',
'cebo/engorde (5 meses)': 'Finishing',
'destetado/transicion (6 y 9 semanas)': 'Nursery',
'destetado/transicion cebo 2, 5 meses': 'Nursery',
'destetado/transicion (7 y 14 semanas)': 'Nursery',
'cebo/engorde (6 sem)': 'Nursery',
'cebo/engorde ( 9 sem)': 'Nursery',
'cebo/engorde (50-60 kg)': 'Finishing',
'destetado/transicion (5 semanas)': 'Nursery',
'cebo/engorde (14 sem)': 'Finishing',
'cebo/engorde (4 dias)': 'Nursery',
'18 kgs': 'Nursery',
'cebo/engorde (41 dias)': 'Nursery',
'destetado/transicion (42 dias)': 'Nursery',
'cebo/engorde (80 dias)': 'Finishing',
'cebo/engorde (21 semanas)': 'Finishing',
'destetado/transicion (35-40 dias)': 'Nursery',
'cebo/engorde (2 semanas)': 'Nursery',
'cebo/engorde ( 1 mes)': 'Nursery',
'lact.>15 y transicion': 'Mixed',
'24 horas de vida destetados': 'Mixed',
'lact.<15': 'Suckling',
'destetados transicion': 'Nursery',
'lact.<15 (3-4 dias)': 'Suckling',
'lact.<15 (1 semana)': 'Suckling',
'lact.<15 (2 dias)': 'Suckling',
'lact.<15  (3 dias)': 'Suckling',
'24 horas de vida': 'Suckling',
'lact.<15 lact.>15': 'Suckling',
'iberico adultos lact.<15': 'Mixed',
'24 horas de vida y lact.<15': 'Suckling',
'(21/28 dias)transicion': 'Weaning',
'cebo 3 meses': 'Finishing',
'adultos (gestante 107d)': 'Sow',
'iberico cebo (20 kg)': 'Finishing',
'transicion 9 sem lactante': 'Mixed',
'lact.<15 (10 dias)': 'Suckling',
'cebo ( entrada)': 'Finishing',
'reposicion primerizas adultos': 'Mixed',
'lact. ( 2 semanas)  transicion (9 semanas)': 'Mixed',
'adulto y lechon': 'Mixed',
'lact. ( 1dias)': 'Suckling',
'3-6 semanas': 'Mixed',
'lechones 6 kg': 'Weaning',
'cebo ( 60kg.)': 'Finishing',
'transicion (21 dias)': 'Weaning',
'cebo (  3  meses)': 'Finishing',
'adultos (reproductoras)cebo (    meses) transicion': 'Mixed',
'lact. (1-5 dias)': 'Suckling',
'lact. ( <15dias)': 'Suckling',
'lact. ( 1 dias)': 'Suckling',
'cebo ( 2	5 meses)': 'Finishing',
'transicion (31 dias)': 'Nursery',
'lact. (3 dias)': 'Suckling',
'cebo (    meses)': 'Finishing',
'lact. ( 3 dias)': 'Suckling',
'lact. (<15  dias)': 'Suckling',
'verracos 5 meses': 'Boar',
'lact. (<7 dias)': 'Suckling',
'cebo de vida': 'Finishing',
'lact. (3-4 semanas)': 'Weaning',
'lact. transicion adultos': 'Mixed',
'lactante': 'Suckling',
'lact. (24 horas y 15 dias)': 'Suckling',
'transicion (7-9 semanas)': 'Nursery',
'transicion (1-2 meses)': 'Nursery',
'cebo 11 semanas de vida': 'Finishing',
'10 semanas transicion': 'Nursery',
'primerizas multiparas': 'Mixed',
'cebo (  6  meses)': 'Finishing',
'transicion (20 dias)': 'Nursery',
'lact. (<24 dias) 	 1 sem	 24 dias': 'Suckling',
'transicion (7 dias)': 'Nursery',
'cebo ( 3-4 meses) iberico': 'Finishing',
'lact. (  <15 dias)': 'Suckling',
'cebo (  4	5  meses)': 'Finishing',
'lechones y reproductoras': 'Mixed',
'lact. ( <15  dias)': 'Suckling',
'lact. (1-6 dias)': 'Suckling',
'adultos verracos': 'Boar',
'lact. ( 6-12 dias)': 'Suckling',
'varias edades': 'Mixed',
'lact. final': 'Suckling',
'cebo ( 2	5  meses)': 'Finishing',
'lact. ( 2-3   dias)': 'Suckling',
'transicion 4-8 semanas': 'Nursery',
'lact. (3 dias)  adultos': 'Mixed',
'lechones 1 y 3 semanas': 'Suckling',
'cebo (  5  meses)': 'Finishing',
'lact. (3-4  dias)': 'Suckling',
'lact. (    dias) reposicion': 'Mixed',
'lact. ( recien nacidos)': 'Suckling',
'destetados (12-18 dias)': 'Weaning',
'cebo ( 1	5 meses)': 'Finishing',
'cebo 16-17 semanas': 'Finishing',
'lact. 1 semana': 'Suckling',
'4-8 semanas': 'Nursery',
'adultos (pool de 4 cerdas del mismo lote)': 'Other',
'lact. (  5  dias)': 'Suckling',
'adultos (recria)': 'Gilt',
'cebo (  4  meses)': 'Finishing',
'lact. (  4  dias)': 'Suckling',
'lact. (1-3  dias)': 'Suckling',
'cebo ( 4   meses)': 'Finishing',
'transicion ibericos': 'Nursery',
'adultos cebo lact.': 'Mixed',
'lact. (  1  dias)': 'Suckling',
'lact. (1 dias)': 'Suckling',
'lact. ( 20 dias)': 'Suckling',
'lact. (  2  dias)': 'Suckling',
'lact. 1 dia': 'Suckling',
'cebo ( 3  meses)': 'Finishing',
'verracos (8 meses)': 'Boar',
'lact. ( recien nacidos )': 'Suckling',
'cebo (6  meses)': 'Finishing',
'lact. (2-7 dias)': 'Suckling',
'lact. (4-8 dias)': 'Suckling',
'cebo ( 2.5 meses)': 'Finishing',
'lact. (destetado': 'Weaning',
'lact. (1 y 21 dias)': 'Suckling',
'lact. (  2-3  dias)': 'Suckling',
'cebo transicion ( 5/9/12 semanas de vida)': 'Mixed',
'lechon adultos': 'Mixed',
'lact. ( 5   dias)': 'Suckling',
'adultos gestantes': 'Sow',
'cebo (1	5 meses)': 'Finishing',
'lact. (2 semanas)': 'Suckling',
'desteteados': 'Weaning',
'lact. y  adultos': 'Mixed',
'lact. (2 dias y 3 semanas)': 'Suckling',
'transicion (18 kg)': 'Nursery',
'transicion (35 dias)': 'Nursery',
'lact. (0-2 dias)': 'Suckling',
'cebo transicion': 'Mixed',
'adultos iberico': 'Other',
'lact. ( 1 dias) y reposicion': 'Mixed',
'1-3-4-6-8 semanas': 'Mixed',
'lact. (3 dias) reposicion': 'Mixed',
'lact. (20 dias)  iberico': 'Suckling',
'lechones cebo reproductora': 'Mixed',
'lact. (4-9 dias)': 'Suckling',
'lact. (10 dias)': 'Suckling',
'cebo ( 5	5  meses)': 'Finishing',
'lact. (  22 dias)': 'Suckling',
'cebo (1mes)': 'Finishing',
'lechones y adultos': 'Mixed',
'lact. (1 dia)': 'Suckling',
'lact. (4 dias)': 'Suckling',
'lact. (  3 dias)': 'Suckling',
'lact. (7  dias)': 'Suckling',
'cebo 50-60kg. 16-17 sem.': 'Finishing',
'transicion (5 y 9 sem)': 'Nursery',
'lact. (15  dias)': 'Suckling',
'lact. (7 dias)': 'Suckling',
'lact. ( 3-4  dias)': 'Suckling',
'lact. (7-10  dias)': 'Suckling',
'destetados (6 y 12 dias)': 'Weaning',
'lact. ( <15   dias)': 'Suckling',
'reposicion (jovenes)': 'Gilt',
'lact. (5 dias)': 'Suckling',
'reposicion (7 meses)': 'Gilt',
'cebo (40kg)': 'Finishing',
'reposicion y partos': 'Mixed',
'cebo (50 dias)': 'Nursery',
'cebo (2 meses	 60 kg)': 'Finishing',
'lact. (6 dias)': 'Suckling',
'lact. (1-20-26 dias papillera)': 'Suckling',
'cebo (4  meses)': 'Finishing',
'lact. (3-4 dias)': 'Suckling',
'cebo 50 kg.': 'Finishing',
'lact. (2-3 dias)': 'Suckling',
'cebo (2	5 meses)': 'Finishing',
'lact. (3  dias)': 'Suckling',
'final lact. transicion': 'Mixed',
'adultos (aborto 108 dias)': 'Sow',
'transicion (28-30 dias)': 'Nursery',
'adultas hibridas': 'Sow',
'destete (5 kg)': 'Weaning',
'adultos verraco': 'Boar',
'lact (3 dias)': 'Suckling',
'lact (2 dias)': 'Suckling',
'lechones 7 sem': 'Nursery',
'lechones 3 semanas': 'Weaning',
'lact. (4 dias y 3 sem)': 'Suckling',
'lact. (8 dias)': 'Suckling',
'semanas 17 y 18': 'Finishing',
'transicion (5 sem)': 'Nursery',
'lact. (1 y 10 dias)': 'Suckling',
'transicion (10 d postdestete)': 'Nursery',
'cebo	 adultas y lechones': 'Mixed',
'transicion  (3 semanas)': 'Nursery',
'lact. (recien nacidos)': 'Suckling',
'cebo (  2  meses)': 'Finishing',
'3-7-8 semanas': 'Mixed',
'adulto (75-80 kg)': 'Other',
'lactante (2-10-20 dias)': 'Suckling',
'lact. ( 2-3 dias)': 'Suckling',
'lact. (recien nacidos) transicion': 'Mixed',
'lact. (neonatales)': 'Suckling',
'lact. (20 dias) transicion (30 dias)': 'Mixed',
'lact. ( 2 dias)': 'Suckling',
'transicion (49 dias)': 'Nursery',
'lact. (15 dias)': 'Suckling',
'lact. (4 semanas)': 'Weaning',
'cebo ( 2	3	4 meses)': 'Finishing',
'lact. (4-5 dias)': 'Suckling',
'transicion (2	5 meses)': 'Nursery',
'adultos  (2o ciclo)': 'Sow',
'lact. (3 semanas)  transicion (7 semanas)': 'Mixed',
'cebo (3  meses)': 'Finishing',
'adultos (verracos)': 'Boar',
'cebo (2 semanas)': 'Finishing',
'lact. (recien nacido)': 'Suckling',
'transicion (4 dias postdestete)': 'Nursery',
'adultos y transicion': 'Mixed',
'transicion / cebo': 'Mixed',
'2	5-3 meses': 'Finishing',
'nuliparas (2 meses en granja)': 'Gilt',
'lact. (    dias)': 'Suckling',
'transicion (1 mes)': 'Weaning',
'lact. (1 semana)': 'Suckling',
'lact. (2  dias)': 'Suckling',
'transicion (3 y 6 semanas)': 'Nursery',
'lact. (20  dias)': 'Suckling',
'adultos (cerdas en maternidad)': 'Sow',
'lact. (1	 2 dias	 4 semanas)': 'Mixed',
'lact. (3 sem) transicion (6 y 9 semanas)': 'Mixed',
'cebo (30kg)': 'Finishing',
'lact. (18  dias)': 'Suckling',
'lact (18-20 dias)': 'Suckling',
'lact. (2-3  dias)': 'Suckling',
'transicion (10 dias postdestete)': 'Nursery',
'transicion (5 y 9 semanas)': 'Nursery',
'lact. (2 y 10 dias)': 'Suckling',
'lact. (10-16 dias)': 'Suckling',
'cebo (3	5 meses)': 'Finishing',
'transicion (15 dias)': 'Nursery',
'transicion (20 kg)': 'Nursery',
'lact. (19 dias)': 'Suckling',
'lact. (5-6 dias)': 'Suckling',
'lact. ( 15 dias)': 'Suckling',
'transicion lact.': 'Mixed',
'transicion lact': 'Mixed',
'adultos y lact. (    dias)': 'Mixed',
'transicion (50 dias)': 'Nursery',
'cebo (6-8 semanas )': 'Finishing',
'cebo (    meses) adultos': 'Mixed',
'cebo (2  meses)': 'Finishing',
'lact. (nacidos muertos)': 'Sow',
'transicion (5 dias)': 'Nursery',
'lact. (15-21 dias) transicion': 'Mixed',
'cebo (5	 9 y 17 semanas)': 'Mixed',
'lact. (3-5  dias)': 'Suckling',
'lact. transicion reposicion': 'Mixed',
'transicion (25-30kg)/entrada cebo': 'Mixed',
'lact. (5  dias)': 'Suckling',
'lact. (  15  dias)': 'Suckling',
'transicion lact. cebo': 'Mixed',
'lact. ( 3-5 dias)': 'Suckling',
'transicion (7 y 9 semanas)': 'Nursery',
'transicion (5-7-9 sem)': 'Nursery',
'cebo (12-14 meses)': 'Finishing',
'lact. (1 sem) transicion (21 dias vida)': 'Mixed',
'transicion (3	 5 y 7 semanas)': 'Nursery',
'lact. (26 dias)': 'Suckling',
'lact. (14  dias)': 'Suckling',
'lact. (22 dias)': 'Suckling',
'lact. (1  dias)': 'Suckling',
'lact. (10-15 dias)': 'Suckling',
'lact. adultos': 'Mixed',
'transicion (1 sem posdestete)': 'Nursery',
'cebo (10-11 semanas)': 'Finishing',
'transicion (3	 6 y 9 semanas)': 'Nursery',
'lact. (mortinatos)': 'Sow',
'cebo (22 semanas	 100 kg)': 'Finishing',
'lact.transicion': 'Mixed',
'cebo (7 meses	 80-90 kg)': 'Finishing',
'transicion (1	5 meses)': 'Finishing',
'lact. cebo	 transicion': 'Mixed',
'lact. (10  dias)': 'Suckling',
'transicion (1	5-2 meses)': 'Nursery',
'cebo (180 dias)': 'Finishing',
'destetado transicion (9 semanas)': 'Mixed',
'lact. ( dias)	 transicion': 'Mixed',
'lact. (4 y 10 dias)': 'Suckling',
'cebo (1  meses)': 'Finishing',
'adultos (machos)': 'Boar',
'lact. (nm y lactantes)': 'Suckling',
'transicion (2-4 semanas)': 'Nursery',
'lact': 'Suckling',
'transicion (30 dias) lact.': 'Mixed',
'reposicion (verracos)': 'Boar',
'cebo  (70 dias)': 'Finishing',
'cebo (2 meses de vida)': 'Nursery',
'cebo (7-8 semanas de vida)': 'Nursery',
'cebo (150 dias )': 'Finishing',
'transicion (17-18 kg)': 'Nursery',
'lact. cebo adultos': 'Mixed',
'lact. (10-12 dias)': 'Suckling',
'lact. (0 dias)': 'Suckling',
'transicion ( 30 dias  de vida)': 'Nursery',
'lact. (6  dias)': 'Suckling',
'lact. (15 dias) transicion (6 semanas de vida)': 'Mixed',
'lact. (1 dias) transicion': 'Mixed',
'lact. (7-15 dias)': 'Suckling',
'cebo  (4  meses)': 'Finishing',
'reposicion (entrada reposicion)': 'Gilt',
'lact. (2-15 dias)': 'Suckling',
'lact. (2-14 dias)': 'Suckling',
'lact (nm)': 'Suckling',
'lact. transicion (4 y 8 semanas de vida)': 'Mixed',
'transicion (4 y 10 semanas de vida)': 'Nursery',
'cebo (1	5 meses) iberico': 'Finishing',
'lact. (3-5 dias)': 'Suckling',
'lact. (1-2 dias)': 'Suckling',
'cebo (6-7 meses)': 'Finishing',
'transicion cebo adultos': 'Mixed',
'transicion (4-7 y 9 semanas)': 'Nursery',
'transicion (40 dias de vida)': 'Nursery',
'lact. (2 y 26 dias) reposicion': 'Mixed',
'lact. transicion (3-5-7 semanas)': 'Mixed',
'lact. (nm) transicion': 'Mixed',
'cebo (8 semanas de vida)': 'Nursery',
'lact. (8-13 dias)': 'Suckling',
'lact. (nm)': 'Suckling',
'cebo (2-3 meses)': 'Finishing',
'transicion (entrada)': 'Nursery',
'reposicion adultos': 'Mixed',
'lact (nm) transicion (9 semanas)': 'Mixed',
'lact. (4 sem)': 'Weaning',
'lact. ( 7 dias)': 'Suckling',
'lact. (1-3 dias)': 'Suckling',
'cebo (50 kg.)': 'Finishing',
'lact. (<15 dias)  adultos': 'Mixed',
'lact. destetados': 'Mixed',
'transicion (5-6 semanas)': 'Nursery',
'cebo (100 - 120 kg)': 'Finishing',
'transicion (6 semanas en transicion)': 'Nursery',
'reproductores': 'Other',
'cebo ( 3	5 meses)': 'Finishing',
'lechones	 adulto': 'Mixed',
'lact. (24 horas de vida)': 'Suckling',
'lact. (1 semana )': 'Suckling',
'transicion (28 dias y 42 dias)': 'Nursery',
'transicion (2 semanas en transicion)': 'Nursery',
'cebo (3	5 meses de vida)': 'Finishing',
'lact. (4  dias)': 'Suckling',
'lact (3-7 dias)': 'Suckling',
'lact. (5 dias) transicion (41 dias)': 'Mixed',
'primerizas	adultos': 'Sow',
'adultos primerizas': 'Sow',
'lact. (<15 dias) transicion': 'Mixed',
'cebo (9 semanas vida)': 'Nursery',
'transicion (30-37 dias de vida)': 'Nursery',
'cebo (40 kg.)': 'Finishing',
'cebo (11-12/13-14/15-16 semanas)': 'Finishing',
'transicion (27-28 dias)': 'Weaning',
'transicion lact. ( 1-7  dias)': 'Mixed',
'lact. transicion (8 semanas)': 'Mixed',
'transicion (70-75 dias)': 'Finishing',
'lact. (12  dias)': 'Suckling',
'cebo (9 semanas de vida)': 'Nursery',
'30-45 semanas': 'Finishing',
'nulipara': 'Gilt',
'lact. (21 dias) transicion (6-9 semanas)': 'Mixed',
'transicion (3 semanas postdestete)': 'Nursery',
'lact. cebo  adultos': 'Mixed',
'cebo (50-80 kg)': 'Finishing',
'cebo (70-95 dias)': 'Finishing',
'cebo (52 dias)': 'Nursery',
'lact. ( 3-4 dias)': 'Suckling',
'lact. (1-2  dias)': 'Suckling',
'lact. (0 dias) reposicion': 'Mixed',
'lact. ( dias)	transicion': 'Mixed',
'transicion lact. (    dias)': 'Mixed',
'entrada a cebo (12 kg.)': 'Nursery',
'reposicion (100 dias)': 'Gilt',
'transicion (23 dias de transicion)': 'Nursery',
'lact (3-5 dias)': 'Suckling',
'cebo ( 80 kg)': 'Finishing',
'9	 12	 15 semanas': 'Finishing',
'lact. (nm) transicion (9 semanas)': 'Mixed',
'lact. (nm) transicion (8 semanas)': 'Mixed',
'transicion (3 y 8 semanas de vida)': 'Nursery',
'transicion (dia 0)': 'Weaning',
'lact. (2-4 dias)': 'Suckling',
'transicion (5-9 semanas)': 'Nursery',
'cebo vida (100 kg)': 'Finishing',
'transicion (18 dias postdestete)': 'Nursery',
'transicion (7 dias postdestete)': 'Nursery',
'lact. (7 y 14 dias)': 'Suckling',
'reproductoras (multiparas)': 'Sow',
'transicion (11 semanas de vida)': 'Finishing',
'5 semanas de vida': 'Nursery',
'transicion (entrada	 3 sem y 6 sem.)': 'Nursery',
'lact. (3 y 15 dias)': 'Suckling',
'lact.transicion (5-10 semanas de vida)': 'Mixed',
'lact (15 dias)': 'Suckling',
'lact. (15	 40	 80 dias)': 'Mixed',
'reproductoras lechones cebo': 'Mixed',
'lact. (2-3 dias) adultos': 'Mixed',
'lact.  transicion': 'Mixed',
'adultos reposicion': 'Mixed',
'lact. (nm y >4 dias)': 'Suckling',
'transicion (3	5 meses; 20kg)': 'Finishing',
'cebo (4-4	5 meses)': 'Finishing',
'lact. (7-10 dias)': 'Suckling',
'adultos (>12meses)': 'Other',
'transicion (40-52 dias)': 'Nursery',
'transicion (34 dias)': 'Nursery',
'transicion (3 y 8 semanas)': 'Mixed',
'varias': 'Mixed',
'cebo (50	 70 y 100 kg.)': 'Finishing',
'transicion (destetado)': 'Nursery',
'lact. ( dias)	adultos': 'Mixed',
'cebo (13 meses)': 'Finishing',
'lact. ( 1-3 dias)': 'Suckling',
'cebo (60-90 kg)': 'Finishing',
'lechones': 'Suckling',
'lact. (3 sem) transicion (6 y 9 sem)': 'Mixed',
'lact. (24 horas )': 'Suckling',
'cebo (5 meses	 65kg)': 'Finishing',
'lact (1-3 dias)': 'Suckling',
'cebo (18 kg)': 'Nursery',
'cebo (2 meses	 25-30 kg)': 'Finishing',
'adultos  lact. (7 a 10	 17 dias) transicion (50 dias)': 'Mixed',
'lact. (  0  dias)': 'Suckling',
'lact. (12 y 20 semanas) adultos': 'Mixed',
'reposicion lact.': 'Mixed',
'cebo (70 dias )': 'Finishing',
'transicion (1-2 semanas)': 'Nursery',
'cebo ( 25 kg.)': 'Finishing',
'iberico (6 meses)': 'Finishing',
'lact. (21 dias) transicion (8 semanas) reposicion': 'Mixed',
'lact. (2-5  dias)': 'Suckling',
'lact (25 dias)': 'Suckling',
'cebo (5 5 meses)': 'Finishing',
'cebo (4   meses)': 'Finishing',
'cebo (10-12 semanas)': 'Finishing',
'lact. (nm) reposicion adultos': 'Mixed',
'cebo inicio': 'Finishing',
'transicion (4	6 y 8 semanas)': 'Nursery',
'lact. (  2 dias)': 'Suckling',
'transicion (70 dias de vida)': 'Nursery',
'recria reposicion': 'Gilt',
'transicion (entrada cebo)': 'Finishing',
'cebo (75 dias)': 'Finishing',
'cebo ( 6  meses)': 'Finishing',
'transicion 20kg (7-8 dias en cebo)': 'Finishing',
'lact (10 dias)': 'Suckling',
'lact (1 dia)': 'Suckling',
'transicion (1-9 semanas) adultos reposicion': 'Mixed',
'adultos lact. (1-2 dias)': 'Mixed',
'transicion (5 semanas vida)': 'Nursery',
'lact (4 semanas de vida)': 'Weaning',
'transicion (70 dias)	 cebo (5 meses)': 'Mixed',
'lact. transicion (9 semanas)': 'Mixed',
'transicion (3 y 9 semanas)': 'Mixed',
'lact. (nm) (21 dias) transicion (65 dias de vida)': 'Mixed',
'cebo (   2 meses y medio)': 'Finishing',
'transicion (1	 3	 5 semanas)': 'Nursery',
'recria (50 kg)': 'Gilt',
'cebo (50 y 80 dias )': 'Finishing',
'lact. (5-10 dias)': 'Suckling',
'lact. (5-7 dias)': 'Suckling',
'lact. (1-2 dias) adultos': 'Mixed',
'cebo ( final)': 'Finishing',
'transicion (11 semanas de vida) lact (nm)': 'Mixed',
'transicion (10 semanas de vida) lact (nm)': 'Mixed',
'transicion (36	 47 y 83 dias de vida)': 'Mixed',
'cebo iberico': 'Finishing',
'verracos adultos': 'Boar',
'transicion (12 dias post-destete)': 'Nursery',
'lact. (18-25 dias)': 'Mixed',
'lact. (1	 7 y 10 dias)': 'Suckling',
'transicion (6	 8 y 10 semanas)': 'Nursery',
'cebo (40-50 kg)': 'Finishing',
'lact.transicion (2	 4 y 5 semanas)': 'Mixed',
'lact. (21 dias) transicion (5 semanas)': 'Mixed',
'transicion  (25 dias)': 'Weaning',
'machos adultos': 'Boar',
'lact. (  0   dias)': 'Suckling',
'transicion (0	3 y 6 semanas)': 'Mixed',
'reposicion primerizas': 'Mixed',
'lact (0 dias)': 'Suckling',
'recria (3-6 meses)': 'Gilt',
'lact. (4 y 28 dias)': 'Mixed',
'cebo (17 dias en cebo )': 'Finishing',
'lact. (<15    dias)': 'Suckling',
'lact. (0 dias) adultos': 'Mixed',
'lact. ( 0   dias)': 'Suckling',
'lact. (  3  dias)': 'Suckling',
'lact. (10-15  dias)': 'Suckling',
'lact (nm)transicion (3	 6 y 9 semanas)': 'Mixed',
'lact. (  0  dias) transicion 7 semanas cebo 11/14 semanas': 'Mixed',
'cebo (0.5 meses)': 'Finishing',
'lact. ( 10   dias)': 'Suckling',
'lact. ( 2  dias)': 'Suckling',
'transicion (7-8 semanas vida)': 'Nursery',
'reposicion 14 semanas': 'Gilt',
'cebo ( 4	5   meses)': 'Finishing',
'lact. transicion (3	5 meses)': 'Mixed',
'cebo (recria)': 'Gilt',
'reposicion lact': 'Mixed',
'primerizas reposicion adultos': 'Mixed',
'cebo 16 semanas': 'Finishing',
'transicion (2	 6 y 9 semanas)': 'Mixed',
'cebo (20-30 kg)': 'Finishing',
'lact. ( 3-4   dias)': 'Suckling',
'nacidos muertos': 'Sow',
'lact.	 transicion	 adultos': 'Mixed',
'lact. transicion (3 semanas)': 'Mixed',
'transicion (3 semanas) primerizas': 'Mixed',
'cebo ( 3-5 meses)': 'Finishing',
'cebo (  3-5  meses)': 'Finishing',
'lact. (  <15  dias)': 'Suckling',
'reposicion (verraco)': 'Boar',
'cebo ( 2	5   meses)': 'Finishing',
'lactante (0 dias)': 'Suckling',
'lact (4 dias)': 'Suckling',
'lactantes (2 dias) transicion (20 dias)': 'Mixed',
'lactantes (7 dias)': 'Suckling',
'lactantes (1-2 dias)': 'Suckling',
'lactantes (    dias)': 'Suckling',
'lactantes (recien nacidos)': 'Suckling',
'transicion (3	 6	 9 semanas)': 'Mixed',
'cebo ( 4	5 meses)': 'Finishing',
'reposicion (cuarentena)': 'Mixed',
'3 semanas post destetado': 'Nursery',
'lactantes (  2  dias)': 'Suckling',
'lactantes ( 2-3 dias)': 'Suckling',
'lactantes (1 y 7 dias)': 'Suckling',
'lactantes (1 dias)': 'Suckling',
'lactantes (3-5 dias)': 'Suckling',
'lactante transicion (4 semanas)': 'Mixed',
'lactantes ( 3 dias) transicion': 'Mixed',
'lactantes (1 dia) transicion (4	7 y 9 semanas)': 'Mixed',
'lactantes primerizas': 'Mixed',
'lactantes (1 dias) adultos': 'Mixed',
'lactantes (1 dias) lactantes (>15 dias)': 'Mixed',
'lactantes (23 dias)': 'Weaning',
'adultos (primerizas y multiparas)': 'Sow',
'lactantes (nm)': 'Suckling',
'lactante (5 dias)': 'Suckling',
'cebo ( 21 sem.)': 'Finishing',
'lactantes (7 dias) transicion': 'Mixed',
'cebo (inicio	 15 kg)': 'Nursery',
'destetado (24 kg)': 'Finishing',
'lactantes (<15 dias)': 'Suckling',
'lactantes (2 y 7 dias)': 'Suckling',
'cebo ( 11 semanas)': 'Finishing',
'lactantes ( 1   dias) adultos': 'Mixed',
'lactantes ( 3-5 dias)': 'Suckling',
'lactantes transicion reposicion': 'Mixed',
'desconocida': 'Missing',
'lactantes nm': 'Suckling',
'transicion (6-8 semanas de vida)': 'Nursery',
'transicion (3	6 y 9 semanas)': 'Mixed',
'cebo (0 meses)': 'Finishing',
'lactantes (21dias)': 'Suckling',
'lactantes ( 0dias)': 'Suckling',
'transicion 3-6-9 semanas': 'Mixed',
'transicion 3-6 semanas': 'Mixed',
'nm': 'Missing',
'transicion (9-10 semanas de vida)': 'Mixed',
'post destetado 30 dias': 'Nursery',
'lactantes (4 dias)': 'Suckling',
'recria y sem 16 nm': 'Gilt',
'lactantes  transicion': 'Mixed',
'lact (1-2 dias) transicion': 'Mixed',
'lactantes (2 dias de vida)': 'Suckling',
'cebo ( 5 semanas de vida)': 'Nursery',
'cebo ( 7 semanas de vida)': 'Nursery',
'cebo (4	5  meses)': 'Finishing',
'lact. (21 y 28 dias)': 'Weaning',
'lactantes (1 semana)': 'Suckling',
'destetado 40 dias lactantes (11 dias)': 'Mixed',
'cebo entrada': 'Finishing',
'lactantes ( 1-2 dias)': 'Suckling',
'reposicion lactantes (nm)': 'Mixed',
'transicion (4	 6 y 9 semanas)': 'Mixed',
'recriatransicion': 'Mixed',
'transicion (6	 9 y 12 semanas)': 'Mixed',
'lactantes (4 semanas)': 'Weaning',
'final transicion': 'Mixed',
'transicion lactantes': 'Mixed',
'lactantes ( ><15   dias)': 'Suckling',
'transicion (35-75 dias)': 'Finishing',
'lact (nm) destetado (26 dias vida)': 'Mixed',
'lactantes (<1 semana)': 'Suckling',
'lactantes (0-15 dias)': 'Suckling',
'transicion (entrada y semana 2)': 'Nursery',
'transicion (86 dias)': 'Finishing',
'lactantes (11 dias) destetado (33 dias)': 'Mixed',
'cebo ( 2-3   meses)': 'Finishing',
'recien destetados': 'Weaning',
'reproductoras	 transicion	 cebo': 'Mixed',
'lactantes (  21  dias)': 'Suckling',
'lactantes (nm y bajas)': 'Suckling',
'lactantes (    dias) transicion': 'Mixed',
'transicion 1-2 semanas': 'Nursery',
'transicion 9 semaanas': 'Nursery',
'transicion  5 semanas': 'Nursery',
'transicion 7 semanas de vida': 'Nursery',
'adultos  verraco >12 meses': 'Boar',
'transicion-cebo (24 kg)': 'Mixed',
'lactantes ( 20   dias)': 'Suckling',
'lact (6-12 dias)': 'Suckling',
'lactantes adultos': 'Mixed',
'transicion (3-6-9 semanas) y primerizas': 'Mixed',
'transicion (26 dias de vida)': 'Weaning',
'reproductoras reposicion': 'Mixed',
'lactantes (0 dias)': 'Suckling',
'transicion 2-4 semanas': 'Weaning',
'transicion  4-5 semanas de vida': 'Nursery',
'transicion (1	 4 y 6 semanas)': 'Mixed',
'10 semanas': 'Nursery',
'adultos lactantes': 'Mixed',
'cebo (22 dias entrados)': 'Finishing',
'transicion (0	 2	 4 y 6 semanas entrados)': 'Mixed',
'transicion	 reproductoras': 'Mixed',
'lactantes ( 3  dias)': 'Suckling',
'lactantes ( 1-2  dias)': 'Suckling',
'lactantes (<7 dias)': 'Suckling',
'lactantes (1 y 20 dias)': 'Suckling',
'digestivo': 'Missing',
'transicion (4 y 7 semanas)': 'Nursery',
'cebo (6	5 meses)': 'Finishing',
'destetado (50 dias)': 'Nursery',
'lactantes (0-1 dias)': 'Suckling',
'lactantes (2-3 dias)': 'Suckling',
'cebo ( 4-6 semanas)': 'Nursery',
'lactantes (  2  dias) destetado': 'Mixed',
'lactantes ( 0-3 dias)': 'Suckling',
'cebo (>12  meses)': 'Finishing',
'lactantes (nm) primerizas': 'Mixed',
'cebo transicion lactantes': 'Mixed',
'lactantes ( 2-4 dias)': 'Suckling',
'lactantes (4 y 10 dias)': 'Suckling',
'transicion (60 dias) cebo ( 4   meses)': 'Mixed',
'entrdada a cebo': 'Finishing',
'lactantes (5 dias)': 'Suckling',
'lactantes (   1 dias)': 'Suckling',
'lactantes (2-4 dias)': 'Suckling',
'lactantes (    dias) cebo (    meses)': 'Mixed',
'lactantes ( 3dias)': 'Suckling',
'cebo (  0	5  meses)': 'Finishing',
'cebo (0	5 meses)': 'Finishing',
'lactantes (  10-20  dias)': 'Suckling',
'lactantes ( nm)': 'Suckling',
'lactantes (   7 dias)': 'Suckling',
'3-5 semanas': 'Mixed',
'lactantes ( 1 dias)': 'Suckling',
'adultos transicion (4 y 9 semanas)': 'Mixed',
'lactantes (<15 dias) destetado transicion cebo (    meses)': 'Mixed',
'destetado 5-6 semanas': 'Nursery',
'lactantes (15 dias	 nm)': 'Suckling',
'transicion (40-60 dias de vida)': 'Nursery',
'lactantes ( 2 dias)': 'Suckling',
'lactantes (nacidos muertos y debiles)': 'Mixed',
'reposicionadultos': 'Mixed',
'lactantes <15 dias': 'Suckling',
'lactantes (nacidos muertos)': 'Sow',
'1-3-7 semanas': 'Mixed',
'lactantes (1-3 dias)': 'Suckling',
'transicion (9 semanas) reposicion (3-6meses)': 'Mixed',
'cebo (1 dia)': 'Finishing',
'lactantes ( 2   dias)': 'Suckling',
'semana 27': 'Finishing',
'destetado lactantes+': 'Mixed',
'destetado lactantes ( 4-7   dias)': 'Mixed',
'cebo ( 40-80kg )': 'Finishing',
'cebo ( 40-80 kg)': 'Finishing',
'lactantes (  1  dias)': 'Suckling',
'reposicion hembras': 'Gilt',
'lactantes (0 dias) reposicion': 'Mixed',
'lactantes (3-7  dias)': 'Suckling',
'cebo ( 5   meses)': 'Finishing',
'transicion lactantes (    dias)': 'Mixed',
'lactantes ( 12   dias)': 'Suckling',
'destetado (3 dias postdestete)': 'Nursery',
'lactantes ( 25-28 dias)': 'Weaning',
'lactantes ( 0 dias)': 'Suckling',
'lactantes (  nm)': 'Suckling',
'destetado (22 dias de vida) lactante': 'Mixed',
'cebo (60-70 kg)': 'Finishing',
'lactantes (0 dias) transicion': 'Mixed',
'cebo (15 meses)': 'Finishing',
'lactantes (7-14 dias)': 'Suckling',
'lactantes (10 y 30 dias de vida)': 'Mixed',
'reposicion	 transicion (2 meses)': 'Mixed',
'transicion  4 sem': 'Weaning',
'transicion  (8 semanas)': 'Nursery',
'adultos (gestacion 105 dias)': 'Sow',
'lactantes (12 dias)': 'Suckling',
'cebo (  9-10  meses)': 'Finishing',
'mortinatos': 'Sow',
'transicion primerizas reproductoras': 'Mixed',
'transicion adultos primerizas': 'Mixed',
'lactantes ( 0 dias) 1a semana': 'Suckling',
'lactantes (3-4 dias)': 'Suckling',
'cebo ( 7 semanas)': 'Nursery',
'adultos lactantes (0 dias)': 'Mixed',
'lactantes (    dias) destetado (> 3 meses)': 'Mixed',
'transicion 42 dias de vida': 'Nursery',
'reposicion	adultos': 'Mixed',
'cebo (    meses) adultos transicion': 'Mixed',
'lactantes ( 2-3   dias)': 'Suckling',
'lactantes ( 0   dias)': 'Suckling',
'gestantes 70 d': 'Sow',
'lactantes ( 18 dias)': 'Suckling',
'destetado lactantes (0 dias)': 'Mixed',
'destetado (3	4 y 5 semanas postdestete)': 'Nursery',
'cebo (30-100 kg)': 'Finishing',
'lactantes (final de lactacion': 'Suckling',
'lactantes (  4  dias)': 'Suckling',
'lnm': 'Missing',
'destetado 15 dias': 'Weaning',
'lactante (5/6 dias)': 'Suckling',
'destetado (1 semana postdestete)': 'Nursery',
'cebo (  6   meses)': 'Finishing',
'transicion 40 dias': 'Nursery',
'transicion (1 y 2 semanas postdestete)': 'Nursery',
'lactantes (7-10  dias)': 'Suckling',
'cebo ( 1   meses)': 'Finishing',
'lactantes ( 3   dias)': 'Suckling',
'cebo ( 50 kg.': 'Finishing',
'lactantes (3-10 dias)': 'Suckling',
'cebo (2	5 meses vida)': 'Finishing',
'cebo (8-10 kg)': 'Nursery',
'cebo ( 2   meses)': 'Finishing',
'lactantes (15 dias) transicion  (7 semanas)': 'Mixed',
'cebo (5 meses vida)': 'Finishing',
'transicion (9 semanas) lactantes (nm)': 'Mixed',
'transicion (2 semanas postdestete)': 'Nursery',
'reposicion (6-7 meses)': 'Gilt',
'adultos (6 meses)': 'Other',
'lactantes (  28  dias)': 'Suckling',
'lactantes (  10-12  dias)': 'Suckling',
'cebo (5	5 meses)': 'Finishing',
'cebo (  1  meses)': 'Finishing',
'cebo ( meses)	 reposicion': 'Mixed',
'destetado 3-6-9 semanas': 'Nursery',
'primerizas	reposicion': 'Mixed',
'transicion (1 y 4 semanas)': 'Nursery',
'lactantes (1	 2 y 25 dias)': 'Suckling',
'transicion( 4 semanas de vida)': 'Weaning',
'lactantes (  <15  dias)': 'Suckling',
'lactantes ( 20dias)': 'Suckling',
'lactantes (1-3 dias) destetado': 'Mixed',
'transicion (1 y 3 semanas) destetado': 'Mixed',
'transicion (18-24	 60 dias)': 'Mixed',
'adultos (60 y 90 dias)': 'Other',
'lactantes (3-7 dias)': 'Suckling',
'transicion (25 dias) lact (nm) reposicion': 'Mixed',
'lactantes ( 3 dias) transicion 1 mes': 'Mixed',
'reproductoras destetado': 'Mixed',
'destetado cebo (4 meses) primerizas': 'Mixed',
'transicion 1 mes': 'Weaning',
'transicion (4	 7 y 10 semanas)': 'Mixed',
'lactantes (2  dias)': 'Suckling',
'lactantes (10-14 dias)': 'Suckling',
'lactantes 3 sem de vida': 'Weaning',
'transicion  7 sem.': 'Nursery',
'lactantes (10-15 dias)': 'Suckling',
'lactantes (    dias) adultos': 'Mixed',
'lactantes (2 dias) destetado': 'Mixed',
'lactantes (12 dias) adultos': 'Mixed',
'transicion (3 y 5 semanas)': 'Mixed',
'transicion (1	 3 y 5 semanas)': 'Mixed',
'destetado (26 dias)': 'Weaning',
'lactantes (2 dias) transicion': 'Mixed',
'lactantes (0-4	 7 y >14 dias)': 'Suckling',
'lactantes (6 dias)': 'Suckling',
'cebo (30 y 60 kg)': 'Finishing',
'destetado (35-40 dias de vida)': 'Nursery',
'lactantes ( <7  dias) primerizas': 'Mixed',
'destetado (4	 7 y 10 semanas)': 'Nursery',
'lactantes (4 y 20 dias)': 'Suckling',
'transicion (24 dias de vida)': 'Weaning',
'destetado (30-40 dias de vida)': 'Nursery',
'lactantes (2-5 dias)': 'Suckling',
'transicion (45-50 dias de vida)': 'Nursery',
'lactantes (1 dia)': 'Suckling',
'cebo (4 y 5meses)': 'Finishing',
'cebo adultos destetados': 'Mixed',
'transicion 4-5-6 semanas': 'Mixed',
'transicion (45 dias)': 'Nursery',
'transicion cebo (6 meses) adultos': 'Mixed',
'destetado (4 semanas de vida)': 'Nursery',
'nacidos muertos y bajas': 'Sow',
'lactantes (  0  dias)': 'Suckling',
'lactantes (15-17 dias)': 'Suckling',
'reposicion lactantes transicion': 'Mixed',
'lactantes ( 5-10dias)': 'Suckling',
'transicion 24-25 dias': 'Weaning',
'lactantes (  27  dias)': 'Suckling',
'adultos destetado': 'Mixed',
'destetado (30 dias de vida)': 'Nursery',
'lactantes (1-3 dias) final maternidad': 'Mixed',
'destetado (4 y 8 semanas de vida)': 'Nursery',
'transicion (4 dias)': 'Nursery',
'transicion (11 dias)': 'Nursery',
'destetado cebo (4 meses)': 'Mixed',
'destetado (1 meses)': 'Weaning',
'destetados (35 dias de vida)': 'Nursery',
'destetados (30 dias de vida)': 'Nursery',
'destetado (5 semanas de vida)': 'Nursery',
'transicion (3 semanas post-destete)': 'Nursery',
'adultos transicion lactantes ( 21 dias)': 'Mixed',
'transicion (2 dias)': 'Nursery',
'transicion (10 dias)': 'Nursery',
'transicion (17 dias)': 'Nursery',
'transicion (24 dias)': 'Nursery',
'adultos cebo': 'Mixed',
'lactantes (2-5  dias)': 'Suckling',
'destetados (40 dias)': 'Nursery',
'lactantes ( 3 semanas de vida)': 'Suckling',
'transicion (4	5 semanas y 30 dias de vida)': 'Nursery',
'destetados (3	5-4	5 semanas de vida)': 'Finishing',
'lactantes (0 y 25 dias)': 'Mixed',
'destetado (4-5 semanas)': 'Nursery',
'destetados (6 kg)': 'Nursery',
'destetado (3-4-5 semanas)': 'Mixed',
'lactantes (5-14 dias)': 'Suckling',
'destetado (28-35 dias)': 'Nursery',
'lactantes 3 y 10 dias': 'Suckling',
'destetado	transicion': 'Mixed',
'cebo (4-5-6 meses)': 'Finishing',
'lactantes (3  dias)': 'Suckling',
'transicion (6 sem vida)': 'Nursery',
'transicion (18-24 dias) cebo (60 dias)': 'Mixed',
'adultos lactantes ( 5  dias)': 'Mixed',
'transicicon (28-49 y 63 dias)': 'Nursery',
'destetado (23 dias postdestete)': 'Nursery',
'transicion primerizas adultos': 'Mixed',
'cebo ( kg)': 'Finishing',
'lactantes (1 y 3 semanas)': 'Suckling',
'destetados (6 semanas)': 'Nursery',
'adultos 90-100 dias': 'Other',
'lactantes (5 y 7 dias)': 'Suckling',
'cebo (10-15 kg)': 'Nursery',
'lactantes (10  dias)': 'Suckling',
'lactantes (15 dias) transicion': 'Mixed',
'lactantes  cebo adultos': 'Mixed',
'transicion (9 semanas de vida) adultos': 'Mixed',
'lactantes (30 dias)': 'Weaning',
'lactantes (10 dias) transicion': 'Mixed',
'cebo (365 dias)': 'Finishing',
'lactantes	destetado': 'Mixed',
'destetado reposicion transicion': 'Mixed',
'lactantes ( 2-8  dias)': 'Suckling',
'destetado (14 dias)': 'Weaning',
'transicion (3	5 meses)': 'Nursery',
'lactantes (4 y 21 dias)': 'Suckling',
'lactantes (18  dias)': 'Suckling',
'lactantes	 destetado	 cebo': 'Mixed',
'lactantes	 destetado': 'Mixed',
'transicion	adultos': 'Mixed',
'transicion( 0-2-4 semanas postdestete)': 'Mixed',
'transicion (3	 5	 7 y 9 semanas de vida)': 'Mixed',
'transicion (30 y 60 dias)': 'Nursery',
'cebo ( 3   meses)': 'Finishing',
'lactantes (25 dias) transicion': 'Mixed',
'lactantes (25 dias) adultos': 'Mixed',
'lactantes (': 'Suckling',
'lactantes ( 14   dias)': 'Suckling',
'lactantes (>15 dias)': 'Suckling',
'adultos (multiparas)': 'Sow',
'lactacion (3 semanas)': 'Weaning',
'lactantes (  0 dias)': 'Suckling',
'lactantes ( 0  dias)': 'Suckling',
'lactantes (5	 15-20 dias) transicion (17-20 dias)': 'Mixed',
'transicion (44 dias de vida)': 'Nursery',
'transicion (35-42 dias)': 'Nursery',
'transicion (49-56 dias)': 'Nursery',
'lactantes (nm) semana 2-3': 'Mixed',
'destetado lactantes': 'Mixed',
'lactantes ( 3-4 dias)': 'Suckling',
'transicion (1 semana) lactantes (3 semanas)': 'Mixed',
'transicion  (6 semanas)': 'Nursery',
'lactantes (21 dias) transicion (6 semanas)': 'Mixed',
'transicion  (6 y 9 semanas)': 'Nursery',
'lactantes (2-7 dias)': 'Suckling',
'transicion (7 semanas vida)': 'Nursery',
'transicion (42 dias de vida)': 'Nursery',
'transicion (35 dias de vida)': 'Nursery',
'lactantes	 transicion': 'Mixed',
'transicion (10-11 semanas) lactantes (7 dias)': 'Mixed',
'transicion (3	 9	 10	 11 y 12 semanas) reposicion (14 semanas)': 'Mixed',
'transicion  (9 semanas vida)': 'Nursery',
'reposicion (18 semanas)': 'Gilt',
'transicion (55 dias vida)': 'Nursery',
'transicion (13/14 semanas vida)': 'Finishing',
'transicion adultos reposicion': 'Mixed',
'cebo (15 semanas vida)': 'Finishing',
'lactantes (9 dias)': 'Suckling',
'lactantes (5-8 dias)': 'Suckling',
'cebo (3 meses de vida) transicion adultos': 'Mixed',
'lactantes (17 dias) transicion (2 y 3 semanas)': 'Mixed',
'destetado (3 meses)': 'Finishing',
'cebo (110-120 kg)': 'Finishing',
'lactantes ( <7  dias)': 'Suckling',
'transicion reposicion verraco': 'Mixed',
'lactantes ( 15-40   dias)': 'Mixed',
'lactantes ( 1  dias)': 'Suckling',
'cebo (3-8 meses)': 'Finishing',
'lactantes (0 dias) transicion (5 y 9 semanas)': 'Mixed',
'transicion (10-15 dias postdetete)': 'Nursery',
'lactantes (7  dias)': 'Suckling',
'lactantes ( 21 dias) adultos': 'Mixed',
'lactantes ( 7 y 21 dias)': 'Suckling',
'cebo (  1	5  meses)': 'Finishing',
'lactantes (  3  dias)': 'Suckling',
'cebo (  60 kg)': 'Finishing',
'transicion (2	 6 y 7 semanas)': 'Mixed',
'transicion (12 semanas vida)': 'Finishing',
'lactantes (10 dias) transicion (4 semanas)': 'Mixed',
'lactantes ( 5 dias)': 'Suckling',
'transicion (2	 3 y 5 semanas)': 'Mixed',
'cebo (  50 kg)': 'Finishing',
'lactantes (0 dias) transicion (6	 7 y 8 semanas)': 'Mixed',
'lactantes ( 10   dias)': 'Suckling',
'cebo (	5 meses )': 'Finishing',
'lactantes (  15  dias)': 'Suckling',
'lactantes ( 7-15   dias)': 'Suckling',
'lactantes	transicion': 'Mixed',
'lactantes (5 dias) transicion': 'Mixed',
'destetado 5-7 semanas': 'Nursery',
'lactantes (  10  dias)': 'Suckling',
'lactantes ( <7 dias)': 'Suckling',
'transicion (4	 7 y 9 semanas)': 'Mixed',
'adultos 3 anos': 'Other',
'cebo (9 meses)': 'Finishing',
'lactantes (1-22 dias)': 'Suckling',
'lactacion (2 semanas)': 'Suckling',
'lactantes (3 sem.)': 'Suckling',
'lactantes ( 4-5   dias)': 'Suckling',
'lactantes (5-7 dias)': 'Suckling',
'transicion 7	5 semanas': 'Nursery',
'adultos transicion (3 y 9 semanas)': 'Mixed',
'transicion 3	5 semanas': 'Mixed',
'cebo ( 8 semanas)': 'Nursery',
'lactantes (    0dias)': 'Suckling',
'lactantes (   0 dias)': 'Suckling',
'lactantes (<7 dias) transicion (2 semanas)': 'Mixed',
'transicion (3-4 dias)': 'Nursery',
'destetado (65 dias)': 'Nursery',
'lactantes (7-9	 14-16 dias)': 'Suckling',
'cebo (  1 meses)': 'Finishing',
'lactantes  reposicion': 'Mixed',
'destetado (3-4 semanas)': 'Weaning',
'transicion  1 semana posdestete': 'Nursery',
'destetado (4-7 dias)': 'Nursery',
'lactantes (1-4 dias)': 'Suckling',
'adultostransicion': 'Mixed',
'cebo (<2 meses)': 'Finishing',
'cebo (  3	5  meses)': 'Finishing',
'destetado	 transicion (4-5 semanas)': 'Mixed',
'destetado	 transicion (9-10 semanas)': 'Mixed',
'transicion cebo (7 meses)': 'Mixed',
'lactantes (2-25 dias)': 'Suckling',
'lactantes transicion (3 y 9 semanas)': 'Mixed',
'lactantes transicion (6 semanas)  primerizas': 'Mixed',
'cebo (9-14 semanas)': 'Finishing',
'destetado primerizas adultos': 'Mixed',
'lactantes (14 dias) transicion (2 semanas)': 'Mixed',
'lactantes (  21 dias)': 'Suckling',
'lactantes (17  dias)': 'Suckling',
'lactantes (8 dias)': 'Suckling',
'lactantes reposicion': 'Mixed',
'lactantes (    dias) destetado transicion': 'Mixed',
'lactantes ( dias)	 transicion': 'Mixed',
'lactantes transicion (9 semanas)': 'Mixed',
'cebo (2 5 meses)': 'Finishing',
'reposicion transicion (1 y 3 semanas)': 'Mixed',
'lactantes (    dias) destetado': 'Mixed',
'lactantes (0 dias) destetado': 'Mixed',
'transicion 26-29 dias': 'Nursery',
'transicion 35 dias': 'Nursery',
'lactantes (  21  dias) transicion (5-7 semanas)': 'Mixed',
'lactantes (  7-21  dias)': 'Suckling',
'lactantes (  7-21  dias) transicion 6 semanas': 'Mixed',
'lactantes ( 7   dias)': 'Suckling',
'reposicion (4	 5 y 6 meses)': 'Gilt',
'lactantes reproductores': 'Mixed',
'lactantes (21 dias) destetado primerizas adultos': 'Mixed',
'cebo (  50 kg. )': 'Finishing',
'lactantes (2 dias) transicion (4	 6 y 9 semanas) reposicion adultos': 'Mixed',
'lactantes reposicion adultos': 'Mixed',
'transicion (1	 2 y 5 semanas)': 'Nursery',
'reposicion lactantes': 'Mixed',
'cebo (23 kg)': 'Finishing',
'transicion (4 y 5 semanas)': 'Nursery',
'transicion	 cebo (4	 6	 8 y 9 semanas)': 'Mixed',
'adultos lactantes (  0 dias)': 'Mixed',
'lactantes (  7  dias)': 'Suckling',
'lactantes ( <3 dias)': 'Suckling',
'lactantes ( 0 dias) reposicion': 'Mixed',
'lactantes (1	 2 y 3 semanas)': 'Suckling',
'lactantes primerizas reposicion adultos': 'Mixed',
'cebo ( 3	5  meses)': 'Finishing',
'lactantes (  20  dias)': 'Suckling',
'primerizas (40 kg)': 'Sow',
'destetado (4 dias)': 'Nursery',
'lactantes ( 21   dias)': 'Suckling',
'lactantes (26 dias) transicion': 'Mixed',
'transicion 4-10 semanas': 'Nursery',
'destetado	cebo ( meses)': 'Mixed',
'lactantes (1 dias) transicion (3 semanas) reposicion': 'Mixed',
'cebo ( 8   meses)': 'Finishing',
'lactantes (2-20 dias)': 'Suckling',
'lactantes (1-2 semanas)': 'Suckling',
'cebo ( 2  meses)': 'Finishing',
'lactantes (7-21   dias)': 'Suckling',
'lactantes ( 25   dias)': 'Suckling',
'transicion (4-7 dias)': 'Nursery',
'lactantes ( 2  dias)': 'Suckling',
'transicion 33 dias': 'Nursery',
'transicion 4 y 6 semanas': 'Nursery',
'cebo (  2	5  meses)': 'Finishing',
'transicion (10 kg) cebo (30-40 kg)': 'Mixed',
'transicion (5 semanas) adultos': 'Mixed',
'lactantes (6	 13 y 21 dias)': 'Suckling',
'cebo (  7  meses)': 'Finishing',
'transicion (28 dias)': 'Weaning',
'reproductorastransicion': 'Mixed',
'cebo (  9 semanas)': 'Nursery',
'lactantes ( 7  dias)': 'Suckling',
'lactantes y transicion': 'Suckling',
'transicion 5 dias post destete': 'Nursery',
'destetado (4 dias postdestete)': 'Nursery',
'lactantes (21  dias)': 'Suckling',
'lactantes (1-15  dias)': 'Suckling',
'destetado (<1 semana)': 'Nursery',
'destetado (7 dias post.destete)': 'Nursery',
'lactantes ( 4-10  dias)': 'Suckling',
'destetado  (30 y 70 dias)': 'Nursery',
'destetado (5 dias)': 'Nursery',
'adultos/ verracos cia': 'Boar',
'transicion (10 semanas vida)': 'Nursery',
'lactantes (3 dias) (nm)': 'Suckling',
'destetado (6-9 semanas vida)': 'Nursery',
'destetado (6 semanas vida)': 'Nursery',
'destetado (9 semanas vida)': 'Nursery',
'transicion (6 y 9 semanas vida)': 'Nursery',
'cebo (7-8 semanas vida)': 'Nursery',
'reproductoras (70-100 dias gestacion)': 'Sow',
'destetado (42 dias vida)': 'Nursery',
'destetado (3 sem vida)': 'Weaning',
'destetado (6 sem vida)': 'Nursery',
'destetado (6 y 9 sem vida)': 'Nursery',
'lactantes (21 dias) transicion 6 semanas': 'Mixed',
'lactantes (1 y 2 semanas) destetado': 'Mixed',
'lactantes (2 semanas) destetado': 'Mixed',
'reproductoras (100-105 dias gestacion)': 'Sow',
'reposicion reproductoras': 'Mixed',
'lactantes (1 dias) transicion': 'Mixed',
'reproductoras ( 70 dias gestacion)': 'Sow',
'lactantes (<7 dias) adultos': 'Mixed',
'lactantes destetado (3-12 sem)': 'Mixed',
'destetado (8 semanas)': 'Nursery',
'cebo (5  meses)': 'Finishing',
'reproductoras (21 y 28 dias gestacion)': 'Sow',
'lactantes ( 24  dias)': 'Suckling',
'reposicion / lactantes': 'Mixed',
'lactantes (24 dias) transicion 7 semanas': 'Mixed',
'cebo ( 95 dias)': 'Finishing',
'cebo (  9  meses)': 'Finishing',
'reposicion  reproductoras (    dias gestacion)': 'Mixed',
'lactantes ( 2dias)': 'Suckling',
'transicion (40-60 dias)': 'Nursery',
'destetado (4	5	6 y 7 sem)': 'Nursery',
'destetado (7-14 dias )lactantes (0-1 dias)': 'Mixed',
'lactantes (0-21 dias) reposicion destetado': 'Mixed',
'cebo (12 meses) transicion': 'Mixed',
'reproductoras ( dias gestacion)': 'Sow',
'lactantes (16 dias)': 'Suckling',
'reproductoras nuliparas': 'Mixed',
'reproductoras (    dias gestacion)': 'Sow',
'lactantes (  <7  dias)': 'Suckling',
'transicion  4 - 7 semanas': 'Nursery',
'adultos (2 anos)': 'Other',
'cebo (6 y 9 semanas)': 'Nursery',
'lactantes	 transicion (3 semanas)': 'Mixed',
'lactantes cebo (40 dias)': 'Mixed',
'lactantes (0 dias) transicion (24 dias) reposicion': 'Mixed',
'reproductoras	 recria (14-15 semanas)': 'Mixed',
'destetado (l1) transicion (l2)': 'Mixed',
'cebo (9 semanas y 4 meses)': 'Mixed',
'lactantes (0dias)': 'Suckling',
'lactantes (5 y 15 dias)': 'Suckling',
'reproductoras (semana 30 gestacion)': 'Sow',
'precebo (    meses)': 'Nursery',
'reproductoras (ultimas semanas gestacion)': 'Sow',
'cebo (1	5 meses) vida': 'Finishing',
'transicion (45-60 dias)': 'Nursery',
'lactantes transicion (3 semanas)': 'Mixed',
'nuliparas	 adultos': 'Gilt',
'transicion 5-7 semanas': 'Nursery',
'destetado 3 y 7 semanas de vida': 'Mixed',
'lactantes (  21 dias) destetado 6 semanas': 'Mixed',
'lactantes (  7  dias) transicion 6 semanas': 'Mixed',
'lactantes (  14  dias)': 'Suckling',
'cebo (4 y 8 semanas) adultos': 'Mixed',
'transicion (5 y 6 semanas)': 'Nursery',
'transicion (5 y 9-10 semanas de vida)': 'Nursery',
'transicion  7 semanas': 'Nursery',
'destetado 38 dias': 'Nursery',
'lactantes (  0  dias) destetado': 'Mixed',
'lactantes (   17 dias)': 'Suckling',
'lactantes ( 15   dias)': 'Suckling',
'lactantes ( 0-20 dias) reproductoras (    dias gestacion)': 'Mixed',
'destetado 5 semanas': 'Nursery',
'cebo (5 y 8 semanas)': 'Nursery',
'cebo ( 60 kilos)': 'Finishing',
'cebo (  40 kg.)': 'Finishing',
'destetadotransicion 6 semanas': 'Nursery',
'reproductoras ( 105   dias gestacion)': 'Sow',
'reproductoras (40  dias gestacion)': 'Sow',
'transicion (5	 6 y 9 semanas)': 'Nursery',
'lactantes (15 dias) adultos': 'Mixed',
'destetado (26-28 dias)': 'Weaning',
'cebo (  7  dias)': 'Finishing',
'destetado (6 semanas)': 'Nursery',
'reproductoras (85 dias gestacion)': 'Sow',
'reproductoras transicion primerizas': 'Mixed',
'lactantes (5-10 dias)': 'Suckling',
'transicion 28 dias': 'Weaning',
'adultos (60 dias)': 'Other',
'lactantes (21-26 dias)': 'Weaning',
'reproductoras (60 dias gestacion)': 'Sow',
'cebo (vida) reposicion': 'Mixed',
'destetado (7 kg)': 'Nursery',
'cebo destetado': 'Mixed',
'cebo (1 meses - 30 kg)': 'Finishing',
'destetadotransicion': 'Mixed',
'lactantes (21dias) destetado  transicion': 'Mixed',
'lactantes (<15 dias) transicion': 'Mixed',
'cebo (4	5 meses)': 'Finishing',
'reproductoras (  105  dias gestacion)': 'Sow',
'reproductoras ( 80-110   dias gestacion)': 'Sow',
'adultos (nulipara)': 'Gilt',
'transicion  (1 semanas)': 'Nursery',
'reproductoras (110 dias gestacion)': 'Sow',
'lactantes ( 21 dias) transicion 6 y 9 semanas de vida': 'Mixed',
'reproductoras (  35  dias gestacion)': 'Sow',
'lactantes (7 dias)	 destetado (3 semanas)	 transicion (6 semanas)': 'Mixed',
'transicion (32 dias)': 'Nursery',
'cebo ( 0	5   meses)': 'Finishing',
'destetado (70-80 dias)': 'Nursery',
'destetado (5 semanas)': 'Nursery',
'cebo ( 5-6   meses)': 'Finishing',
'reproductoras (  23  dias gestacion)': 'Sow',
'lactantes (22 dias) reproductoras': 'Mixed',
'adultos (1	5 anos)': 'Other',
'cebo (14 meses)': 'Finishing',
'transicion cebo (    meses)': 'Mixed',
'lactantes (14-21 dias)': 'Suckling',
'adultos	 reproductoras': 'Mixed',
'cebo ( 4  meses)': 'Finishing',
'lactantes (  12 dias) transicion': 'Mixed',
'reproductoras ( 28dias gestacion)': 'Sow',
'cebo (  2.5meses)': 'Finishing',
'reproductoras (35 dias gestacion)': 'Sow',
'cebo (   5 meses)': 'Finishing',
'cebo (  9 semanas de vida)': 'Nursery',
'destetado 3-4 semanas': 'Weaning',
'cebo (  4 meses)': 'Finishing',
'cebo (9 sem)': 'Nursery',
'transicion destetado': 'Mixed',
'cebo/engorde (    sem/mes/ano)': 'Finishing',
'destetado/transicion (8-12 sem)': 'Mixed',
'destetado/transicion 4-8-12 semanas': 'Mixed',
'lactantes (  1  dias) cebo/engorde': 'Mixed',
'adultos lactantes (  0  dias)': 'Mixed',
'semanas 41 y 42': 'Finishing',
'semana 41': 'Finishing',
'cebo/engorde (8 meses)': 'Finishing',
'lactantes  destetado/transicion': 'Mixed',
'cebo/engorde ( 7 meses   sem/mes/ano)': 'Finishing',
'cebo/engorde (  2 meses)': 'Finishing',
'lactantes (2 semanas)	 transicion': 'Mixed',
'lactantes destetado/transicion': 'Mixed',
'destetado/transicion 60 dias de vida': 'Mixed',
'destetado/transicion 4-7 semanas': 'Mixed',
'lactantes (  13  dias)': 'Suckling',
'destetado/transicion (4 semanas)': 'Mixed',
'cebo/engorde (  3 meses)': 'Finishing',
'cebo/engorde (3	5 meses)': 'Finishing',
'reproductoras/es destetado/transicion': 'Mixed',
'cebo/engorde (2 meses	 <60kg)': 'Finishing',
'cebo/engorde (2 semanas	 <30 kg)': 'Finishing',
'cebo/engorde (2	5 meses)': 'Finishing',
'cebo/engorde ( 2	5 meses)': 'Finishing',
'cebo/engorde ( 3-4 meses)': 'Finishing',
'cebo/engorde (9 semanas)': 'Finishing',
'destetado/transicion 32 dias': 'Mixed',
'cebo/engorde (4-50 kg)': 'Mixed',
'cebo/engorde (18	 21 y 24 semanas)': 'Finishing',
'destetado/transicion (4-10 sem)': 'Mixed',
'cebo/engorde (25 kg)': 'Finishing',
'destetado/transicion (5 sem)': 'Mixed',
'destetado/transicion (4 dias)': 'Mixed',
'destetado/transicion (<20 kg)': 'Mixed',
'cebo (1	5 meses	 <60 kg)': 'Finishing',
'cebo/engorde (2 y 5 meses)': 'Finishing',
'cebo/engorde (1 semanas)': 'Finishing',
'cebo/engorde (20 kg)': 'Finishing',
'transicion (8 dias)': 'Nursery',
'lactantes ( 22  dias)': 'Suckling',
'cebo/engorde  60 dias': 'Finishing',
'destetado/transicion 74 dias': 'Mixed',
'reproductoras/es lactantes (    dias)': 'Mixed',
'cebo/engorde ( 5 meses)': 'Finishing',
'destetado/transicion 1-4 semanas': 'Mixed',
'lactantes (5-8-20 dias)': 'Suckling',
'reproductoras/es	 destetado/transicion': 'Mixed',
'lactantes reproductoras/es': 'Mixed',
'destetado/transicion 3 semanas': 'Mixed',
'adultos recria/reposicion': 'Mixed',
'cebo/engorde ( 0 meses- 9 semanas de vida)': 'Mixed',
'lactantes (  5 dias)': 'Suckling',
'lactantes ( 3 semanas )': 'Suckling',
'cebo/engorde (10 meses)': 'Finishing',
'destetado/transicion 1 semana': 'Mixed',
'destetado/transicion 2-6 semanas': 'Mixed',
'cebo/engorde ( 9 meses)': 'Finishing',
'lactantes (  1 dias)': 'Suckling',
'destetado/transicion (1 semana)': 'Mixed',
'lactantes (nm semanas 44 y 45)': 'Other',
'cebo/engorde (10 semanas)': 'Nursery',
'destetado/transicion (semanas 43 y 44)': 'Mixed',
'destetado/transicion (2 semanas)': 'Mixed',
'primerizas y multiparas': 'Sow',
'lactantes (3 y 21 dias)': 'Suckling',
'lactantes ( 2-3	8 dias)': 'Mixed',
'recria (lote 45 y 27 semanas de vida)': 'Gilt',
'7 y 9 semanas': 'Nursery',
'destetado/transicion (5-6 dias post-destete)': 'Mixed',
'cebo/engorde ( 1	5 meses)': 'Finishing',
'lactantes (  30  dias)': 'Suckling',
'destetado/transicion 3 sem': 'Mixed',
'destetados (6-8 dias)': 'Nursery',
'cebo/engorde ( 5mes)': 'Finishing',
'lactantes (  10 dias)': 'Suckling',
'semana 45': 'Finishing',
'semana 46': 'Finishing',
'cebo/engorde (60 dias)': 'Nursery',
'lactantes 18-20 dias transicion 30-35 dias': 'Mixed',
'lactantes (nm	 semanas 45 y 46)': 'Other',
'lactantes (1 dias	 nm)': 'Suckling',
'cebo/engorde ( 4 mes)': 'Finishing',
'destetado/transicion 4 sem vida': 'Mixed',
'destetado/transicion 7-9 semanas': 'Mixed',
'semanas 44	 45	 46 y 47': 'Finishing',
'semanas 45	 46 y 47': 'Finishing',
'cebo/engorde (  8  sem)': 'Nursery',
'destetado/transicion (7-8 meses)': 'Mixed',
'lactantes ( 26  dias)': 'Weaning',
'lactantes (0 dias) destetado/transicion': 'Mixed',
'cebo/engorde ( 1mes)': 'Nursery',
'destetado/transicion (5	 6 y 7 dias)': 'Mixed',
'destetado/transicion (10 semanas)': 'Mixed',
'reproductoras/es	 adultos': 'Mixed',
'nuliparas y multiparas (semana 47)': 'Mixed',
'cebo/engorde ( 2 meses': 'Nursery',
'reproductoras/es	 lactantes	 destetado/transicion': 'Mixed',
'cebo/engorde (24 dias)': 'Finishing',
'lactantes (1 dias)  destetado/transicion': 'Mixed',
'destetado/transicion (4-5 semanas)': 'Mixed',
'lactantes (    dias) recria/reposicion': 'Mixed',
'lactantes (  3-4  dias)': 'Suckling',
'destetado/transicion recria/reposicion': 'Mixed',
'reproductoras/es recria/reposicion': 'Mixed',
'lactantes )': 'Suckling',
'lactantes (10 dias)	 reproductoras/es': 'Mixed',
'reposicion	 lactantes (nm)': 'Mixed',
'cebo/engorde (3-4 meses)': 'Finishing',
'reproductoras/es primerizas': 'Sow',
'recria/reposicion	 reproductoras/es	 lactantes (20 dias)': 'Mixed',
'lactantes (2-8 dias)': 'Suckling',
'destetado/transicion (40 dias)': 'Mixed',
'lactantes	 destetado/transicion	 cebo/engorde': 'Mixed',
'cebo/engorde (50 kg)': 'Finishing',
'cebo/engorde (90 kg)': 'Finishing',
'destetado/transicion (1	5 meses)': 'Mixed',
'reproductoras/es lactantes ( 4-18 dias)': 'Mixed',
'cebo/engorde ( 3meses)': 'Finishing',
'cebo/engorde (6 meses': 'Finishing',
'lactantes ( 18 dias)  destetado/transicion': 'Mixed',
'cebo/engorde ( 12 semanas)': 'Finishing',
'35 dias': 'Nursery',
'lactantes (    dias) destetado/transicion': 'Mixed',
'cebo/engorde ( 7-8 semanas)': 'Nursery',
'lactantes ( 21  dias)': 'Suckling',
'cebo/engorde (7 mes/ano)': 'Finishing',
'cebo/engorde (2 sem )': 'Finishing',
'cebo/engorde ( 12   sem)': 'Finishing',
'destetado/transicion 35 dias': 'Mixed',
'destetado/transicion (45 dias)': 'Mixed',
'cebo/engorde (14  sem)': 'Finishing',
'cebo/engorde ( 30 kg.)': 'Finishing',
'lactantes (nm)  primerizas': 'Mixed',
'cebo/engorde (12/12/23)': 'Finishing',
'cebo/engorde (18 kg)': 'Finishing',
'cebo/engorde (10@)': 'Finishing',
'lactantes recria/reposicion': 'Mixed',
'recria/reposicion adultos': 'Mixed',
'adultos (114 dias)': 'Other',
'cebo/engorde (20 dias)': 'Finishing',
'cebo/engorde (1	5 meses)': 'Finishing',
'cebo/engorde (11 semanas)': 'Finishing',
'cebo/engorde (40 kg)': 'Finishing',
'destetado/transicion cebo/engorde adultos': 'Mixed',
'reproductoras/es destetado/transicion cebo/engorde': 'Mixed',
'lactantes ( 28  dias)': 'Weaning',
'cebo/engorde ( 3   sem)': 'Finishing',
'destetado/transicion (25-30kg)': 'Mixed',
'destetado/transicion	 cebo/engorde': 'Mixed',
'lactantes	 destetado/transicion': 'Mixed',
'cebo/engorde (20-30 kg)': 'Finishing',
'destetado/transicion (3 y 6 semanas)': 'Mixed',
'lactantes (7 semanas)': 'Nursery',
'final transicion / cebo': 'Mixed',
'cebo/engorde (12  sem)': 'Finishing',
'cebo/engorde (  11  sem)': 'Finishing',
'cebo/engorde (2	6 meses)': 'Finishing',
'primerizas recria/reposicion': 'Gilt',
'cebo/engorde 14-18 sem.': 'Finishing',
'destetado/transicion 10 semanas': 'Mixed',
'cebo/engorde (  2  sem)': 'Finishing',
'cebo/engorde ( 1 semana)': 'Finishing',
'destetado/transicion (6 kg)': 'Mixed',
'destetado/transicion reproductoras/es': 'Mixed',
'recria/reposicion reproductoras/es': 'Mixed',
'lactantes ( dias)	destetado/transicion': 'Mixed',
'lactantes (9-13 dias)': 'Suckling',
'lactantes (2-4 dias) destetado/transicion (12-19 dias)': 'Mixed',
'lactantes (2-9 dias)': 'Suckling',
'lactantes ( 20 dias)	 destetado/transicion': 'Mixed',
'destetado/transicion (12 dias)': 'Mixed',
'destetado/transicion (3	 6 y 9 semanas)': 'Mixed',
'lactantes (0-3 semanas)  destetado/transicion (>4 semanas)': 'Mixed',
'nuliparas	 multiparas	 transicion (28 dias)': 'Mixed',
'nuliparas	 multiparas	 recria (22 y 25 semanas)': 'Mixed',
'destetado/transicion (12 semanas)': 'Mixed',
'cebo/engorde (42 dias)': 'Nursery',
'adultos 4-5 anos': 'Other',
'cebo/engorde ( 50 dias)': 'Nursery',
'destetado/transicion (21-25 dias)': 'Mixed',
'lactantes ( 20-25 dias)': 'Weaning',
'3 dias	 3-6-9 semanas': 'Mixed',
'destetado/transicion (4 y 8 semanas)': 'Mixed',
'lactantes destetado/transicion reproductoras/es': 'Mixed',
'destetado/transicion  32 dias': 'Mixed',
'destetado/transicion (4 y 7 semanas)': 'Mixed',
'destetado/transicion (30 kg)': 'Mixed',
'lactantes destetado/transicion 842 y 63 dias)': 'Mixed',
'cebo/engorde (1 semana	 25-30 kg)': 'Finishing',
'entrada cebo/engorde': 'Finishing',
'cebo/engorde (  3  meses)': 'Finishing',
'reproductoras/es lactantes (27 dias)': 'Mixed',
'cebo/engorde (90kg)': 'Finishing',
'lactantes (nm) destetado/transicion (3 semanas)': 'Mixed',
'lactantes (7	 15	 21 dias)': 'Suckling',
'desconocidocebo/engorde (    sem/mes/ano)': 'Mixed',
'cebo/engorde ( 50 kg.)': 'Finishing',
'cebo/engorde recria/reposicion': 'Mixed',
'lactantes (5 dias) primerizas': 'Mixed',
'destetado/transicion (15 kg)': 'Mixed',
'cebo/engorde (30 kg)': 'Finishing',
'cebo/engorde (12 semanas)': 'Finishing',
'destetado/transicion (5	 8 y 9 semanas)': 'Mixed',
'cebo/engorde (12 sem)': 'Finishing',
'lactante (1 y 4 semanas)': 'Suckling',
'destetado/transicion (35 dias	 1 semana de destete)': 'Mixed',
'destetado/transicion (4 y 5 semanas)': 'Mixed',
'destetado/transicion 40 dias': 'Mixed',
'cebo/engorde (  1	5 meses)': 'Finishing',
'destetado/transicion (8 semanas)': 'Mixed',
'destetado/transicion 30 kg': 'Mixed',
'cebo/engorde (120 kg)': 'Finishing',
'recria/reposicion (24 y 21 semanas)': 'Gilt',
'cebo/engorde (4-6 meses)': 'Finishing',
'cebo/engorde (16 y 17 semanas)': 'Finishing',
'cebo/engorde (    9 sem)': 'Finishing',
'cebo/engorde (2-2	5 meses)': 'Finishing',
'cebo/engorde (23 semanas)': 'Finishing',
'semana 4-8-11': 'Nursery',
'lactantes (  5  dias)': 'Suckling',
'destetado/transicion (3 y 9 semanas)': 'Mixed',
'cebo/engorde (14 semanas)': 'Finishing',
'destetado/transicion  10 semanas': 'Mixed',
'destetado/transicion (32 y 56 dias)': 'Mixed',
'lactantes ( 10 dias)': 'Suckling',
'destetado/transicion (1	3 y 4-5 semanas)': 'Mixed',
'lactantes ( 4-5  dias)': 'Suckling',
'destetado/transicion 6 kg': 'Mixed',
'lactantes ( <15 dias)': 'Suckling',
'lactantes (4-5 dias)': 'Suckling',
'destetado/transicion  2 meses': 'Mixed',
'cebo/engorde (45 dias de vida	 21 dias destetados)': 'Nursery',
'cebo/engorde (90 dias de vida	 67 dias destetados)': 'Finishing',
'lactante (20 dias)': 'Suckling',
'cebo/engorde ( 60-70 kg)': 'Finishing',
'cebo/engorde ( 90 kg)': 'Finishing',
'cebo/engorde (3 sem)': 'Finishing',
'destetado/transicion (3 meses)': 'Mixed',
'reproductoras/es	destetado/transicion': 'Mixed',
'cebo/engorde (8 y 18 semanas)': 'Finishing',
'nuliparas (12 meses)': 'Gilt',
'destetado/transicion 8 semanas': 'Mixed',
'recria/reposicion	reproductoras/es	destetado/transicion': 'Mixed',
'destetado/transicion (5 y 6 semanas)': 'Mixed',
'lactantes destetado/transicion (5 y 8 semanas)': 'Mixed',
'lactantes ( < 7  dias)': 'Suckling',
'lactantes ( 3 dias)': 'Suckling',
'cebo/engorde ( 4	5 meses)': 'Finishing',
'cebo/engorde ( 60 dias)': 'Finishing',
'lactantes	destetado/transicion': 'Mixed',
'cebo/engorde (13	 14 y 15 semanas)': 'Finishing',
'lactantes (7 y 20 dias)': 'Suckling',
'lactantes (  2 dias)': 'Suckling',
'cebo/engorde ( 7 meses)': 'Finishing',
'recria/reposicion	 lactantes	 destetado/transicion': 'Mixed',
'63 y 45 dias': 'Nursery',
'lactantes (  3 dias)': 'Suckling',
'cebo/engorde (70 dias)': 'Finishing',
'cebo/engorde (1 semana)': 'Finishing',
'cebo/engorde 8 sem': 'Finishing',
'destetado/transicion (5 y 8 semanas)': 'Mixed',
'lactantes (5  dias)': 'Suckling',
'cebo/engorde (  3 mes)': 'Finishing',
'cebo/engorde (    sem/mes/ano)reproductoras/es': 'Mixed',
'cebo/engorde reproductoras/es reproductoras/es reproductoras/es': 'Mixed',
'cebo/engorde (65 dias)': 'Nursery',
'lactantes (  0-1 dias)': 'Suckling',
'reproductoras/es	': 'Other',
'recria/reposicion destetado/transicion': 'Mixed',
'cebo/engorde (  4 mes)': 'Finishing',
'cebo/engorde ( 8 semanas)': 'Nursery',
'cebo/engorde (100 dias)': 'Finishing',
'semana 16': 'Finishing',
'cebo/engorde ( 19 semanas)': 'Finishing',
'cebo/engorde (4 mes': 'Finishing',
'cebo/engorde 3 meses': 'Finishing',
'lactantes (20 dias) destetado/transicion _x001c__x001c_>35 dias': 'Mixed',
'destetado/transicion 3-6-9 semanas': 'Mixed',
'cebo/engorde (5-6 meses)': 'Finishing',
'reposicion  reproductoras/es': 'Mixed',
'transicion 6kg': 'Nursery',
'lactantes (semanas 15 y 16)': 'Nursery',
'recria/reposicion (5 meses)': 'Gilt',
'cebo/engorde (1 sem/mes/ano)': 'Finishing',
'lactantes (4 a 15 dias) + destetado transicion (35 -40 dias)': 'Mixed',
'lactantes (  18 dias)': 'Suckling',
'cebo/engorde (1-2semanas)': 'Finishing',
'cebo/engorde (20kg)': 'Finishing',
'destetado/transicion (30 dias)': 'Mixed',
'destetado/transicion cebo/engorde': 'Mixed',
'lactantes (6 kg)': 'Weaning',
'cebo/engorde (3-4 sem/mes/ano)': 'Weaning',
'lactantes (  1-4  dias)': 'Suckling',
'destetado/transicion 30 dias': 'Mixed',
'cebo/engorde (18 semanas)': 'Finishing',
'lactantes (    dias) otra': 'Mixed',
'cebo/engorde destetado/transicion': 'Mixed',
'destetado/transicion (31dias)': 'Mixed',
'destetado/transicion (40 dias y 2 semanas)': 'Mixed',
'destetado/transicion (3	 5	 7	 9 y 12 semanas)': 'Mixed',
'destetado/transicion 2 semanas': 'Mixed',
'destetado/transicion 4 semanas estancia': 'Mixed',
'cebo/engorde (18 kg.)': 'Nursery',
'cebo/engorde ( 20 kg.)': 'Finishing',
'cebo/engorde (3 meses	  80 kg.)': 'Finishing',
'lactantes (nm) nuliparas': 'Mixed',
'cebo/engorde ( 13 semanas)': 'Finishing',
'recria/reposicion	 destetado/transicion': 'Mixed',
'lactantes (4-8 dias)': 'Suckling',
'cebo/engorde (40 dias)': 'Nursery',
'cebo/engorde ( 13 sem)': 'Finishing',
'lactantes (2-3 sem.) destetado/transicion (4 sem. y final transicion)': 'Mixed',
'cebo/engorde (65 kg)': 'Finishing',
'cebo/engorde (40-50 kg)': 'Finishing',
'cebo/engorde ( 45-60 dias)': 'Finishing',
'lactantes (1-3 dias) destetado/transicion (3	 6 y 8 semanas)': 'Mixed',
'cebo/engorde (12-13 sem)': 'Finishing',
'cebo/engorde (6-7 meses)': 'Finishing',
'primiparas	 multiparas': 'Sow',
'cebo/engorde adultos': 'Mixed',
'destetado/transicion (7 sem vida)': 'Mixed',
'destetado/transicion 24 dias': 'Mixed',
'destetado/transicion (6 sem vida)': 'Mixed',
'cebo/engorde (5 mes)': 'Finishing',
'reproductoras/es (150 kg)': 'Other',
'destetado/transicion (4-5 dias)': 'Mixed',
'cebo/engorde (10-13 sem)': 'Finishing',
'destetado/transicion (1 sem)': 'Mixed',
'lactantes (nm) recria/reposicion': 'Mixed',
'cebo/engorde ( 20 semanas)': 'Finishing',
'lactantes ( 2-3-4-5 sem.) destetado/transicion': 'Mixed',
'destetado/transicion 7-8 semanas': 'Mixed',
'cebo/engorde ( 3 5 meses)': 'Finishing',
'lactantes ( 8-10 dias)': 'Suckling',
'cebo/engorde (  2meses)': 'Finishing',
'reproductoras/es destetado/transicion primerizas': 'Mixed',
'lactantes otra': 'Mixed',
'cebo/engorde (16 semanas)': 'Finishing',
'destetado/transicion 11 semanas': 'Mixed',
'cebo/engorde ( 3 meses )': 'Finishing',
'transicion /cebo/engorde': 'Mixed',
'cebo/engorde (  4-5 meses)': 'Finishing',
'adultos reproductoras/es': 'Other',
'cebo/engorde (  5 meses)': 'Finishing',
'adultos lactantes (    dias)': 'Mixed',
'cerdas': 'Sow',
'lactantes (    dias   )reproductoras/es': 'Mixed',
'lactantes (7 y 10 dias)': 'Suckling',
'cebo/engorde (13 semanas)': 'Finishing',
'cebo/engorde (  1 mes)': 'Finishing',
'destetado/transicion lactantes (    dias)': 'Mixed',
'lactantes ( dias)	destetado/transicion	cebo/engorde ( sem/mes/ano)': 'Mixed',
'nuliparas	 multiparas': 'Mixed',
'cebo/engorde ( 12 meses)': 'Finishing',
'destetado/transicion (7 dias postdestete)': 'Mixed',
'cebo/engorde (8 semanas)': 'Nursery',
'cebo/engorde (10  sem/)': 'Nursery',
'cebo/engorde (70 kg)': 'Finishing',
'destetado/transicion 1 mes': 'Mixed',
'cebo/engorde 14-15 sem': 'Finishing',
'cebo/engorde (  5-6 meses)': 'Finishing',
'lactantes  destetado': 'Mixed',
'cebo/engorde ( 13-19 sem)': 'Finishing',
'lactantes (0 dias) reproductoras/es': 'Mixed',
'adultos lactantes (  20  dias)': 'Mixed',
'cebo/engorde ( 4 mes )': 'Finishing',
'destetado/transicion	primerizas': 'Mixed',
'cebo/engorde 60 dias': 'Finishing',
'cebo/engorde 2 meses': 'Finishing',
'cebo/engorde 4 meses': 'Finishing',
'cebo/engorde (0 meses)': 'Finishing',
'lactantes	 destetado/transicion	 adultos': 'Mixed',
'cebo/engorde (19 sem)': 'Finishing',
'lactantes	destetado/transicion	primerizas	adultos': 'Mixed',
'destetado/transicion (0	 1	 2	 3	 4 y 5 semanas)': 'Mixed',
'cebo/engorde (5	5 meses)': 'Finishing',
'lactantes (nm	 semanas 27 y 28)': 'Other',
'lactantes (nm	 semana 29)	 cuarentena': 'Mixed',
'reproductoras/es cebo/engorde (2-2.5 meses)': 'Mixed',
'cebo/engorde (50 dias)': 'Nursery',
'cebo/engorde (30 dias)': 'Nursery',
'lactantes	 adultos': 'Mixed',
'cebo/engorde (<1 mes)': 'Nursery',
'lactantes ( 1-3  dias)': 'Suckling',
'cebo/engorde ( 10sem)': 'Nursery',
'cebo/engorde 25kg': 'Finishing',
'cebo/engorde (19 semanas)': 'Finishing',
'cebo/engorde (22 y 29-32 semanas)': 'Finishing',
'lactantes (primerizas y multiparas)': 'Mixed',
'destetado/transicioncebo/engorde (    sem/mes/ano)': 'Mixed',
'cebo/engorde (15 kg)': 'Nursery',
'lactantes (  1-3  dias)': 'Suckling',
'destetado/transicion (2	 4	 6 y 8 semanas)': 'Mixed',
'cebo/engorde (80 kg)': 'Finishing',
'semana 31': 'Finishing',
'semana 32': 'Finishing',
'lactantes (semanas 30 y 31)': 'Nursery',
'lactantes ( 21 dias) destetado/transicion': 'Mixed',
'lactantes ( 12-14 dias)': 'Suckling',
'cebo/engorde (  4 meses)': 'Finishing',
'lactantes (nuliparas+multiparas) recria/reposicion': 'Mixed',
'lactantes (0) destetado/transicion (6 semanas)': 'Mixed',
'lactantes  destetado /transicion': 'Mixed',
'lactantes (  0 y 21  dias)': 'Suckling',
'destetado/transicion (27 dias)': 'Mixed',
'cebo/engorde (6 mese)': 'Finishing',
'cebo/engorde (   6 meses)': 'Finishing',
'cebo/engorde (2 meses	 60 kg)': 'Finishing',
'cebo/engorde (7 meses)': 'Finishing',
'reproductoras/es	destetado/transicion	adultos': 'Mixed',
'6-9 semanas': 'Nursery',
'cebo/engorde ( 17 sem.)': 'Finishing',
'destetado/transicion (3 sem.)': 'Mixed',
'destetado/transicion 3 sem.': 'Mixed',
'destetado/transicion (1 mes)': 'Mixed',
'cebo/engorde (primales de 20 kg)': 'Finishing',
'lactantes (0 dias	 semana 33)': 'Mixed',
'cebo/engorde (>3 meses)': 'Finishing',
'cebo/engorde (10 sem.)': 'Finishing',
'cebo/engorde (  100kg.)': 'Finishing',
'lactantes (10 dias) destetado/transicion (35 dias) cebo/engorde (55 dias)': 'Mixed',
'adultos destetado/transicion': 'Mixed',
'cebo 21 semanas': 'Finishing',
'cebo	 22 semanas': 'Finishing',
'15 semanas': 'Finishing',
'3-4 semanas': 'Weaning',
'lactantes (11-16 dias)': 'Suckling',
'destetado/transicion (4 sem.)': 'Mixed',
'lactantes (4 sem.)': 'Suckling',
'destetado/transicion 25 dias': 'Mixed',
'150 dias': 'Finishing',
'cebo/engorde (20 y 110 kg)': 'Finishing',
'lactantes (    dias)destetado/transicion': 'Mixed',
'lactantes (nm) destetado/transicion (4 semanas)': 'Mixed',
'lactantes (  4 dias)': 'Suckling',
'lactantes ( 4 dias)': 'Suckling',
'lactantes ( 6 dias)': 'Suckling',
'destetado/transicion  (8 semanas)': 'Mixed',
'lactantes (  28 dias)': 'Weaning',
'destetado/transicion 6-10 semanas': 'Mixed',
'cebo/engorde (6-8 meses)': 'Finishing',
'destetado/transicion cebo/engorde (4 meses)': 'Mixed',
'4-6-8 semanas.': 'Nursery',
'cebo 20kg': 'Finishing',
'lactantes ( 1 dia)': 'Suckling',
'destetado/transicion (45dias)': 'Mixed',
'sem.36': 'Finishing',
'lactantes (  22 dias)': 'Suckling',
'cebo/engorde ( 11 y 12 semanas)': 'Finishing',
'cebo/engorde (140 dias)': 'Finishing',
'cebo/engorde ( 17-18 semanas)': 'Finishing',
'cebo/engorde (15 semanas)': 'Finishing',
'cebo/engorde ( 11 semanas)': 'Finishing',
'cebo/engorde ( 3 sem/mes/ano)': 'Finishing',
'lactantes (  14 dias) destetado/transicion': 'Mixed',
'destetado/transicion 21-28 dias': 'Mixed',
'lactantes ( 18-23 dias)': 'Suckling',
'lactantes (  30 dias)': 'Suckling',
'lactantes ( 28   dias)': 'Weaning',
'desconocidodestetado/transicion': 'Mixed',
'lactantes (   21 dias)': 'Suckling',
'cebo/engorde (  5  mes)': 'Finishing',
'lactantes (    dias) destetado/transicion reproductoras/es': 'Mixed',
'destetado/transicion (3 y 4 semanas)': 'Mixed',
'lactantes (  23  dias)': 'Suckling',
'lactantes (0 y 22 dias)': 'Suckling',
'reproductoras/es (5 y 6 meses)': 'Gilt',
'lactantes (10-18 dias)': 'Suckling',
'lactantes (16	 23 y 25 dias)': 'Mixed',
'cebo/engorde ( 22 semanas)': 'Finishing',
'destetado/transicion 90 dias': 'Mixed',
'cebo/engorde (11 sem.)': 'Finishing',
'destetado/transicion (9 sem.)': 'Mixed',
'cebo/engorde (entrada)': 'Finishing',
'destetado/transicion (5 sem.)': 'Mixed',
'destetado/transicion (21	 28 y 35 dias)': 'Mixed',
'cebo/engorde (  4-5 mes)': 'Finishing',
'destetado/transicion (6-7 semanas)': 'Mixed',
'cebo/engorde (25    sem/mes/ano)': 'Finishing',
'cebo/engorde ( 100-120 kg)': 'Finishing',
'cebo/engorde (160 dias)': 'Finishing',
'cebo/engorde (  6 meses)': 'Finishing',
'recria/reposicion (semana 11)': 'Gilt',
'destetado/transicion 9 semanas': 'Mixed',
'cebo/engorde ( sem/mes/ano)destetado/transicion': 'Mixed',
'destetado/transicion 23 dias': 'Mixed',
'destetado/transicion 7 dias': 'Mixed',
'destetado/transicion 45 dias': 'Mixed',
'reproductoras/es lactantes': 'Mixed',
'lactantes (1 dias) destetado/transicion': 'Mixed',
'lactantes (predestete)': 'Suckling',
'destetado/transicion (4 y 6 semanas)': 'Mixed',
'cebo/engorde ( 2.5 meses)': 'Finishing',
'cebo/engorde (60-70 kg)': 'Finishing',
'lactantes (0 dias) recria/reposicion': 'Mixed',
'cebo/engorde (>5 meses)': 'Finishing',
'reproductoras/es adultos': 'Mixed',
'destetado/transicion (5-9 semanas)': 'Mixed',
'semanas 36-37-38-39-40-41': 'Finishing',
'destetado/transicion  4-9 dias': 'Mixed',
'cebo/engorde (50-60 dias)': 'Nursery',
'destetado/transicion (6 y 9 sem.)': 'Mixed',
'cebo/engorde (24 semanas)': 'Finishing',
'cebo/engorde (80-90 kg)': 'Finishing',
'destetado/transicion (7 sem.)': 'Mixed',
'destetado/transicion (4-9 dias)': 'Mixed',
'cebo/engorde (  1 dia)': 'Finishing',
'lactantes (    dias) /transicion': 'Mixed',
'destetado/transicion (6 sem.)': 'Mixed',
'transicion ( 8 semanas)': 'Nursery',
'destetado/transicion (7-8 semanas)': 'Mixed',
'destetado/transicion (3	 6 y 9 sem	)': 'Mixed',
'cebo/engorde ( 4 meses))': 'Finishing',
'destetado/transicion (3 y 6 sem.)': 'Mixed',
'destetado/transicion (6-9 sem.)': 'Mixed',
'adultos )2	5 anos)': 'Other',
'cebo/engorde ( 22 sem)': 'Finishing',
'cebo/engorde ( 4  sem/mes/ano)': 'Finishing',
'destetado/transicion (50 dias)': 'Mixed',
'cebo/engorde (  6  sem)': 'Finishing',
'cebo/engorde ( 14   sem)': 'Finishing',
'cebo/engorde ( 9  sem)': 'Finishing',
'destetado lactantes ( 10 dias)': 'Mixed',
'cebo/engorde (    sem/mes/ano) reproductoras/es': 'Mixed',
'destetado/transicion (8-9 sem.)': 'Mixed',
'destetado/transicion (4-5 sem.)': 'Mixed',
'recria/reposicion	destetado/transicion': 'Mixed',
'cebo/engorde (10-11 meses; 140kg)': 'Finishing',
'lactantes ( 4-15 dias)': 'Suckling',
'cebo/engorde (12 sem.)': 'Finishing',
'pre-destete': 'Suckling',
'lactantes (25 dias) destetado/transicion': 'Mixed',
'lactantes (semana 45)': 'Other',
'lactantes (semana 46)': 'Other',
'destetado/transicion 4 semanas': 'Mixed',
'cebo/engorde ( 8 meses)': 'Finishing',
'lactantes ( 26 dias)': 'Weaning',
'lactantes (  24  dias)': 'Weaning',
'lactantes (  16  dias)': 'Suckling',
'lactantes ( 6 kg.)': 'Weaning',
'cebo/engorde ( 4	 5 meses)': 'Finishing',
'lactantes ( 3-5   dias)': 'Suckling',
'cebo/engorde (    4 mes)': 'Finishing',
'recria/reposicion (28/11)': 'Gilt',
'lactantes (21 dias) destetado/transicion': 'Mixed',
'recria/reposiciondestetado/transicion': 'Mixed',
'cebo/engorde ( 4 muestras)': 'Finishing',
'destetado/transicion (6 sem.) cebo/engorde (9 sem.)': 'Mixed',
'cebo/engorde (1 mes': 'Finishing',
'cebo/engorde (14 sem.)': 'Finishing',
'lactantes (  9  dias)': 'Suckling',
'recria/reposicion (22 sem.)': 'Gilt',
'destetado/transicion (8 sem.) cebo/engorde (10 sem.)': 'Mixed',
'lactantes (1-8 dias)': 'Suckling',
'desconocidolactantes (    dias)': 'Suckling',
'lactantes (2-6 y 14-19 dias)': 'Suckling',
'reproductoras lactantes': 'Mixed',
'cebo/engorde (1 mes entrados)': 'Finishing',
'cebo/engorde (5 meses	 80 kg)': 'Finishing',
'lactantes (5 dias)  destetado/transicion': 'Mixed',
'cebo/engorde (9-10 sem.)': 'Finishing',
'cebo/engorde (60 kg)': 'Finishing',
'cebo/engorde (100 kg)': 'Finishing',
'cebo/engorde (4	5 meses)': 'Finishing',
'cebo/engorde (15 sem.)': 'Finishing',
'semanas 44 a 49': 'Finishing',
'lactantes destetado/transicion cebo/engorde': 'Mixed',
'lactantes destetado/transicion adultos': 'Mixed',
'lactantes ( 7-10 dias)': 'Suckling',
'semana 50': 'Finishing',
'cebo/engorde (13 sem.)': 'Finishing',
'destetado/transicion (20-30kg)': 'Mixed',
'destetado/transicion (26 dias)': 'Mixed',
'mamones 3 meses': 'Suckling',
'cebo (4 semanas)': 'Finishing',
'lactantes (  18  dias)': 'Suckling',
'cebo/engorde ( 20 kg)': 'Finishing',
'lactantes (  4-12  dias)': 'Suckling',
'destetado/transicion 7-9 sem': 'Mixed',
'cebo  (3 meses)': 'Finishing',
'lactantes (  7	 14 y 20 dias)': 'Suckling',
'lactantes (   2 dias)': 'Suckling',
'lactante destetado/transicion': 'Mixed',
'cebo/engorde (  13 sem)': 'Finishing',
'cebo/engorde ( 11 y 14 sem)': 'Finishing',
'cebo/engorde ( 12 sem)': 'Finishing',
'cebo/engorde (  11 y 14 sem)': 'Finishing',
'destetado/transicion (18 kg)': 'Mixed',
'destetado/transicion  (40 dias)': 'Mixed',
'cebo/engorde (4-5 meses)': 'Finishing',
'5-9 semanas': 'Nursery',
'cebo/engorde ( 70 kg.)': 'Finishing',
'cebo/engorde (2 sem.)': 'Finishing',
'lactantes ( 10  dias)': 'Suckling',
'cebo/engorde (  10 sem)': 'Finishing',
'cebo/engorde (8 sem.)': 'Finishing',
'cebo/engorde ( 1-2 meses)': 'Finishing',
'recria/reposicion	reproductoras/es': 'Mixed',
'cebo/engorde (   2 meses)': 'Finishing',
'cebo/engorde (16 sem.)': 'Finishing',
'transicion 1 sem': 'Nursery',
'transicion  3 sem': 'Nursery',
'6 weeks': 'Nursery',
'lactantes (   20-25 dias)': 'Weaning',
'lactantes (  26 dias)': 'Weaning',
'destetado/transicion (5 semanas': 'Mixed',
'cebo 6 meses': 'Finishing',
'lactantes ( 24-48horas de vida)': 'Suckling',
'lactantes (    dias) destetado /transicion': 'Mixed',
'lactantes (   21  dias)': 'Suckling',
'cebo/engorde (  14 sem)  sem/mes/ano)': 'Finishing',
'primerizas	 multiparas': 'Sow',
'destetado/transicion (4-5-6 sem.)': 'Mixed',
'cebo/engorde ( 9 semanas)': 'Finishing',
'cebo/engorde ( 14-24 semanas)': 'Finishing',
'lactantes (25  dias)': 'Weaning',
'cebo/engorde ( 18 semanas)': 'Finishing',
'destetado/transicion (38-40 dias)': 'Mixed',
'adultos cebo (88-61-41 dias)': 'Mixed',
'cebo/engorde ( 5-6 meses)': 'Finishing',
'reproductoras/es 60 sem': 'Other',
'lactante 2 dias': 'Suckling',
'lactantes (   12-15 dias)': 'Suckling',
'destetado/transicion (10 sem)': 'Mixed',
'cebo/engorde (15-16 kg)': 'Nursery',
'lactantes (15  dias)': 'Suckling',
'cebo/engorde ( 15sem/)': 'Finishing',
'reposicion externa': 'Mixed',
'25 kilos': 'Finishing',
'25kg': 'Finishing',
'cebo/engorde 1 mes)': 'Finishing',
'adultos/primerizas': 'Sow',
'cebo/engorde ( 1.5 meses)': 'Finishing',
'cebo/engorde (35 y 60 dias)': 'Finishing',
'destetado/transicion (5 dias postdestete)': 'Mixed',
'cebo/engorde (1 sem.)': 'Finishing',
'cebo/engorde (30	 37	 51 y 65 dias)': 'Finishing',
'cebo/engorde (37 dias)': 'Finishing',
'cebo/engorde ( 2 semanas)': 'Finishing',
'destetado/transicion 9 semnaas': 'Mixed',
'cebo/engorde (22 kg)': 'Finishing',
'destetado/transicion (7 sem)': 'Mixed',
'lactante/ destetado': 'Mixed',
'semanas 7 y 8': 'Nursery',
'destetado/transicion (3	5 y 5 sem.)	 recria/reposicion (16 sem.)	 reproductoras/es': 'Mixed',
'semanas 5/6 y 8': 'Nursery',
'lactantes (1 sem.) destetado/transicion (4 y 7 sem.)': 'Mixed',
'destetado/transicion (28 dias)': 'Mixed',
'cebo/engorde (  20 dias)': 'Finishing',
'cebo/engorde ( 20 dias)': 'Finishing',
'lactantes (semanas 6	 7	 9 y 9)': 'Nursery',
'reproductoras/es reproductoras/es': 'Other',
'lactantes ( 10-13 dias)': 'Suckling',
'destetado/transicion 5-6 sem': 'Mixed',
'lactantes (25 dias) destetado/transicion (3-9 semanas)': 'Mixed',
'recria/reposicion lactantes': 'Mixed',
'recria/reposicion	reproductoras/esreproductoras/es': 'Mixed',
'lactantes (    dias) reproductoras/es': 'Mixed',
'cebo/engorde (50 kg)reproductoras/es': 'Mixed',
'lactantes (    dias)  de  primerizas': 'Mixed',
'destetado/transicion reproductoras/es reproductoras/es': 'Mixed',
'(20 kg) cebo/engorde (20 sem.)': 'Finishing',
'destetado/transicion reproductoras/es reproductoras/es reproductoras/es': 'Mixed',
'cebo/engorde ( sem/mes/ano)reproductoras/es': 'Mixed',
'lactantes (    dias)reproductoras/es': 'Mixed',
'destetado/transicionreproductoras/es': 'Mixed',
'lactantes (1 y 19 dias)': 'Suckling',
'lactantes (1 y 21 dias)': 'Suckling',
'recria/reposicionreproductoras/es reproductoras/es': 'Mixed',
'cebo/engorde (9	 13	 17 y 18 sem.)': 'Finishing',
'reproductoras/es reproductoras/es reproductoras/es': 'Other',
'cebo/engorde (   10 semanas)': 'Finishing',
'cebo/engorde ( 24 sem)': 'Finishing',
'0': 'Missing',
'cebo/engorde (  17 sem)': 'Finishing',
'lactantes (2 dias) destetado/transicion': 'Mixed',
'lactantes (2 dias) reproductoras/es': 'Mixed',
'cebo/engorde ( 2 meses)   sem/mes/ano)': 'Finishing',
'desconocidoreproductoras/es reproductoras/es reproductoras/es': 'Other',
'recria/reposicionreproductoras/es reproductoras/es reproductoras/es reproductoras/es reproductoras/es reproductoras/es': 'Mixed',
'cebo/engorde ( 2 meses)reproductoras/es reproductoras/es reproductoras/es': 'Mixed',
'lactantes (    dias) reproductoras/es reproductoras/es': 'Mixed',
'11w': 'Finishing',
'destetado/transicion 8-9 semanas': 'Mixed',
'cebo/engorde (    sem/mes/ano) reproductoras/es reproductoras/es': 'Mixed',
'recria/reposicion reproductoras/es reproductoras/es reproductoras/es': 'Mixed',
'destetado/transicionreproductoras/es reproductoras/es': 'Mixed',
'cebo/engorde (    sem/mes/ano)reproductoras/es reproductoras/es': 'Mixed',
'reproductoras/es lactantes (    dias) reproductoras/es': 'Mixed',
'cebo/engorde (7 sem)': 'Nursery',
'lactantes (0dias) destetado/transicion': 'Mixed',
'destetado/transicion (32 dias)': 'Mixed',
'destetado/transicion (56 dias)': 'Mixed',
'destetado/transicion (6 sem)': 'Mixed',
'lactantes (1-20 dias)': 'Suckling',
'destetado/transicion (sem. 4 y  6)': 'Mixed',
'cuarentena 1': 'Other',
'lactantes destetado/transicion adultos (nuliparas)': 'Mixed',
'cebo/engorde (16 weeks)': 'Finishing',
'cebo/engorde ( 10 sem)': 'Nursery',
'recria/reposicionlactantes (    dias)': 'Mixed',
'15  semanas': 'Finishing',
'cebo/engorde ( 5 dias)': 'Finishing',
'cebo/engorde (7	5 meses)': 'Finishing',
'destetado/transicion (2 sem.)': 'Mixed',
'cebo/engorde (3 sem.)': 'Finishing',
'cebo/engorde ( 42 dias)': 'Nursery',
'lactantes destetado/transicion (5 y 8 sem.)': 'Mixed',
'8-10 semanas': 'Nursery',
'7 meses': 'Finishing',
'destetado/transicion (10 sem.)': 'Mixed',
'lactantes (  0-1  dias)': 'Suckling',
'lactantes ( 3	5 semanas)': 'Other',
'lactante 3 dias': 'Suckling',
'recria/reposicion lactantes (    dias)': 'Mixed',
'lactantes (  2-3  dias)': 'Suckling',
'destetado/transicion (semana 17)': 'Mixed',
'lactantes (30 dias) destetado/transicion (40 dias)': 'Mixed',
'cebo/engorde (~40 kg)': 'Finishing',
'lactantes (sem. 18)': 'Suckling',
'cebo/engorde ( 4 meses sem/mes/ano)': 'Finishing',
'destetado/transicion (8 sem.)': 'Mixed',
'reproductoras/es 8 meses': 'Other',
'cebo/engorde ( 6-18 kg.)': 'Nursery',
'cebo/engorde (1	 3 y 6 sem. post-destete)': 'Nursery',
'lactantes (3 dias) adultos': 'Mixed',
'broilers (   dia/sem.)': 'Missing',
'cebo/engorde (5 mese)': 'Finishing',
'destetado/transicion cebo/engorde (    sem/mes/ano)': 'Mixed',
'destetado/transicion (25 dias) reproductoras/es': 'Mixed',
'destetado/transicion (09/05/2024)': 'Mixed',
'cebo/engorde (  42 dias)': 'Nursery',
'cebo/engorde ( 3 semanas)': 'Finishing',
'lactantes (  15 dias)': 'Suckling',
'lactantes ( 1-5  dias)': 'Suckling',
'destetado/transicion 10 dias': 'Mixed',
'cebo/engorde ( 6 mes)': 'Finishing',
'destetado/transicion (1 sem': 'Mixed',
'lactantes (  12  dias)': 'Suckling',
'cebo/engorde ( 4   sem/mes/ano)': 'Finishing',
'lactantes	 cebo/engorde (2 meses)': 'Mixed',
'lactantes ( 4 semanas)': 'Weaning',
'destetado/transicion (2 meses)': 'Mixed',
'lactantes (1 dias) reproductoras/es (7 meses)': 'Mixed',
'lactantes ( 3-5dias)': 'Suckling',
'destetado/transicion (1 sem.)': 'Mixed',
'recria/reposicion reproductoras/es recria/reposicion': 'Mixed',
'destetado/transicion (31 dias)': 'Mixed',
'semana 21': 'Finishing',
'semana 20/21': 'Finishing',
'otradestetado/transicion': 'Mixed',
'cebo/engorde ( 2 mes)': 'Finishing',
'cebo/engorde ( 9 semans)': 'Finishing',
'reproductoras/es (115 dias de gestacion) recria/reposicion': 'Mixed',
'cebo/engorde (15-20 sem.)': 'Finishing',
'destetado/transicion (3 y 4 sem.)': 'Mixed',
'7-11 semanasreproductoras/es': 'Mixed',
'lactantes (  19  dias)': 'Suckling',
'cebo/engorde ( 9 sem.)': 'Finishing',
'cebo/engorde (9 sem.)': 'Finishing',
'destetado/transicion (6-8 sem.)': 'Mixed',
'cebo/engorde (2 mes)': 'Finishing',
'adultos cebo/engorde (    sem/mes/ano)': 'Mixed',
'primerizas lactantes ( 7 dias)': 'Mixed',
'lactantes (  25  dias) destetado/transicion': 'Mixed',
'recria/reposicion (6 meses)': 'Gilt',
'recria/reposicion (7 meses)': 'Gilt',
'lactantes/ adultos': 'Mixed',
'lactantes/destetado': 'Mixed',
'cebo/engorde (3	5-4 meses)': 'Finishing',
'reproductoras/es (nm': 'Other',
'destetado/transicion (8 dias)': 'Mixed',
'cebo/engorde ( 4  mes)': 'Finishing',
'adultos (110 dias gestacion)': 'Sow',
'cebo/engorde (10 dias)': 'Finishing',
'cebo/engorde (9 sem)': 'Nursery',
'reproductoras/es	lactantes ( dias)': 'Mixed',
'cebo/engorde ( 4 semanas)': 'Finishing',
'lactantes (  12-13 dias)': 'Suckling',
'lactantes (  21dias)': 'Weaning',
'lactantes ( 21dias)': 'Weaning',
'lactantes (  21  dias) 6-9 sem': 'Mixed',
'lactantes/destetado/transicion': 'Mixed',
'cebo/engorde ( 6 sem)': 'Finishing',
'cebo/engorde ( 6 y 9  sem)': 'Nursery',
'reproductoras/es (5-6 meses)': 'Other',
'reproductoras/es lactantes (    dias) destetado/transicion': 'Mixed',
'cebo/engorde ( 1 sem)': 'Finishing',
'reproductoras/es 1 ano': 'Other',
'destetado/transicion sem27': 'Mixed',
'lactantes (7 dias) destetado/transicion (28 dias)': 'Mixed',
'destetado/transicion 1-2sem': 'Mixed',
'cebo/engorde (    10 sem/mes/ano) cebo/engorde (    sem/mes/ano)': 'Finishing',
'destetado/transicion (20 kg)': 'Mixed',
'destetado/transicion 12 dias': 'Mixed',
'lactantes (20 dias) adultos': 'Mixed',
'destetado/transicion (4 meses)': 'Mixed',
'primerizas (4 meses)': 'Gilt',
'destetado/transicion (7 a 11 sem.)': 'Mixed',
'cebo/engorde ( 80-90 kg.)': 'Finishing',
'recria/reposicion	 lactantes ( dias)': 'Mixed',
'lactantes ( dias)	 destetado/transicion': 'Mixed',
'cebo/engorde ( 4   sem/mes': 'Finishing',
'destetado/transicion (2	5 meses)': 'Mixed',
'cebo/engorde (9 meses)': 'Finishing',
'cebo/engorde (18 sem.)': 'Finishing',
'reproductoras/es (verracos)': 'Boar',
'destetado/transicion cebo /engorde (    sem/mes/ano)': 'Mixed',
'cebo/engorde ( 3	5-4 meses)': 'Finishing',
'cebo/engorde ( 25 meses)': 'Finishing',
'destetado/transicion 1 mes postdestete': 'Mixed',
'destetado/transicion 8 sem': 'Mixed',
'cebo/engorde (8 sem)': 'Nursery',
'destetado/transicion 10 dias transicion': 'Mixed',
'destetado/transicion	 lactantes (    dias)': 'Mixed',
'recria/reposicion (1 ano)': 'Gilt',
'reproductoras/es recria/reposicion destetado/transicion': 'Mixed',
'lactantes (1 dias) recria/reposicion': 'Mixed',
'reproductoras/es lactantes (28 dias)': 'Mixed',
'cebo/engorde (4 sem.)': 'Nursery',
'recria/reposicion 11-22 sem': 'Gilt',
'reproductoras/es	lactantes ( dias)	destetado/transicion': 'Mixed',
'90 dias': 'Finishing',
'lactantes (0  dias)': 'Suckling',
'recria/reposicion 40 kg.': 'Gilt',
'transicion y cebo': 'Mixed',
'lactantes (   4 dias)': 'Suckling',
'destetado/transicion 4-6-7-8-9 semanas': 'Mixed',
'cebo/engorde ( 2-3 meses)': 'Finishing',
'destetado/transicion 4 sem': 'Mixed',
'cebo/engorde (1 dias)': 'Finishing',
'cebo/engorde (final)': 'Finishing',
'lactantes ( 20  dias)': 'Suckling',
'cebo/engorde (  16  sem)': 'Finishing',
'lactantes destetado/transicion (3 sem.)': 'Mixed',
'lactantes (4 sem.) recria/reposicion': 'Mixed',
'cebo/engorde ( 2 -12 meses)': 'Finishing',
'lactantes (  4-5  dias)': 'Suckling',
'adultos (sementales)': 'Boar',
'reproductoras/es destetado/transicion lactantes': 'Mixed',
'cebo/engorde ( 30-40 kg)': 'Finishing',
'reproductoras/es	destetado/transicion	primerizas': 'Mixed',
'destetado/transicion y cebo/engorde (8 sem)': 'Mixed',
'reproductoras/es lactantes (21 dias)': 'Mixed',
'cebo/engorde (  16 meses)': 'Finishing',
'cebo/engorde (llevan 1 sem.)': 'Finishing',
'lactantes ( 25  dias)': 'Weaning',
'cebo/engorde (1	5 mes)': 'Finishing',
'reproductoras/es (30 dias de gestacion)': 'Sow',
'lactantes ( 2  dias) destetado/transicion': 'Mixed',
'recria/reposicion (sem. 37)': 'Gilt',
'abortos 90 dias': 'Sow',
'destetado/transicion 6 sem.': 'Mixed',
'lactantes (  20 dias)': 'Suckling',
'7-11 semanas': 'Nursery',
'lactantes (  4  sem)': 'Suckling',
'destetado/transicion 5 sem.': 'Mixed',
'cebo/engorde (  10 semanas)': 'Finishing',
'cebo/engorde (  1 semana)': 'Finishing',
'cebo/engorde (14 mes)': 'Finishing',
'destetado/transicion (sem. 40)': 'Mixed',
'destetado/transicion (3 y 9 sem.)': 'Mixed',
'lactantes (   25 dias)': 'Weaning',
'primerizas (70 dias)': 'Gilt',
'recria/reposicioncebo/engorde (    sem/mes/ano)': 'Finishing',
'destetado/transicion (3 sem	)': 'Mixed',
'semana 42': 'Finishing',
'cebo/engorde (2  sem)': 'Finishing',
'destetado/transicion 8sem': 'Mixed',
'cebo/engorde ( 1 semana en cebo)': 'Finishing',
'recria/reposicion	reproductoras/es	lactantes ( dias)': 'Mixed',
'lactantes (   1 y 17 dias de vida) recria/reposicion': 'Mixed',
'cebo/engorde ( 16 kg.)': 'Nursery',
'cebo/engorde ( 8 semanas de vida)': 'Nursery',
'destetado/transicion 4 sem.': 'Mixed',
'lactantes (28 dias) destetado/transicion': 'Mixed',
'lactantes ( 7-28 dias)': 'Mixed',
'lactantes (pre-destete)': 'Suckling',
'reproductoras/es 44-88': 'Other',
'cebo/engorde (3-5 meses)': 'Finishing',
'lechones (1 mes)': 'Weaning',
'lactante 5 dias': 'Suckling',
'destetado/transicion 1 sem': 'Mixed',
'lactantes (  2   dias)': 'Suckling',
'destetado/transicionrecria/reposicion': 'Mixed',
'lactantes ( dias)	cebo/engorde ( sem/mes/ano)': 'Mixed',
'destetado/transicion (1 sem	)': 'Mixed',
'lactantes (10 dias) destetado/transicion': 'Mixed',
'lactantes ( dias)reproductoras/es': 'Mixed',
'recria/reposicion (9 sem.)': 'Gilt',
'lactantes (  5-25  dias)': 'Mixed',
'recria/reposicion	adultos': 'Mixed',
'lactantes (   20 dias)': 'Suckling',
'recria/reposicion  30kg': 'Gilt',
'transicion 1a semana': 'Nursery',
'lactantes (  25  dias)': 'Mixed',
'destetado/transicion  42 dias': 'Mixed',
'recria/reposicion cebo/engorde (    sem/mes/ano)': 'Mixed',
'cebo/engorde (  50 dias)': 'Nursery',
'cebo/engorde ( 6 semanas)': 'Nursery',
'cebo/engorde (2 sem)': 'Finishing',
'reproductoras/es 110 dias': 'Other',
'destetado/transicion 4 semanas post destete': 'Mixed',
'cebo/engorde ( 15 dias)': 'Finishing',
'cebo/engorde ( 1 semana )': 'Finishing',
'destetado/transicion 4 semanas postdestete': 'Mixed',
'recria/reposicion  destetado/transicion': 'Mixed',
'lactantes 3-5 dias)': 'Suckling',
'cebo/engorde ( 60 kg)': 'Finishing',
'cebo/engorde ( 20 kilos)': 'Finishing',
'cebo/engorde (2-3 @)': 'Finishing',
'destetado/transicion 2	5 semanas': 'Mixed',
'destetado/transicion  2 sem': 'Mixed',
'cebo/engorde (5 sem.)': 'Finishing',
'destetado/transicion 30 kgs': 'Mixed',
'destetado/transicion 3-4 sem': 'Mixed',
'lactantes ( 0 dias) adultos': 'Mixed',
'lactantes ( 15  dias)': 'Suckling',
')reproductoras/es': 'Other',
'destetado/transicion (4 sem.) cebo/engorde (7	 8	 9 y 10 sem.)': 'Mixed',
'destetado/transicion 3-4 semanas': 'Mixed',
'cebo/engorde ( 20kg)': 'Finishing',
'lactantes (maternidad)lactantes (    dias)': 'Mixed',
'lactantes (maternidad)': 'Suckling',
'transicion  (final)': 'Nursery',
'recria/reposicion (nuliparas)': 'Gilt',
'lactantes (  dia)': 'Suckling',
'lactantes (  2 sem)': 'Suckling',
'cebo/engorde 16/12/24': 'Finishing',
'cebo/engorde 17/12/2024': 'Finishing',
'cebo/engorde ( 21 sem)': 'Finishing',
'destetado/ 1 sem': 'Mixed',
'cebo/engorde ( 7 sem)': 'Nursery',
'cebo/engorde ( 45 kilos)': 'Finishing',
'lactantes  )recria/reposicion': 'Mixed',
'cebo/engorde (6 sem.)': 'Nursery',
'reproductoras/es cebo/engorde (    sem/mes/ano)': 'Mixed',
'5 meses': 'Finishing',
'5-6 meses': 'Finishing',
'destetado/transicion 9 sem vida': 'Mixed',
'cebo/engorde ( 80 kg)': 'Finishing',
'recria/reposicion 7 mes': 'Gilt',
'lactantes ( 100  dias)': 'Other',
'final destetado/transicion': 'Mixed',
'cebo/engorde (  5 semanas)': 'Nursery',
'cebo/engorde (45 y 66 dias)': 'Finishing',
'reproductoras/es destetado/transicion lactantes (    dias)': 'Mixed',
'lactantes (0 dias	 3 sem.)': 'Suckling',
'otralactantes (    dias)': 'Mixed',
'lactantes (  3  dias)  destetado/transicion 2-4-6 semanas': 'Mixed',
'cebo/engorde (  50-60 dias)': 'Finishing',
'cebo/engorde ( 14  sem)': 'Finishing',
'lactantes ( 1   dias)': 'Suckling',
'cebo/engorde tienen 4-5 meses': 'Finishing',
'destetado/transicion	cebo/engorde ( sem/mes/ano)': 'Mixed',
'destetado/transicion 7 sem': 'Mixed',
'lactantes ( 5 kg)': 'Suckling',
'lactantes ( 3-4dias)': 'Suckling',
'destetado/transicion (20 sem.)': 'Mixed',
'lactantes (  99  dias)': 'Other',
'cebo/engorde lactantes': 'Mixed',
'destetado/transicion 10sem': 'Mixed',
'cebo/engorde ( 4 sem)': 'Finishing',
'lactantes cebo/engorde': 'Mixed',
'cebo/engorde (  2 sem)': 'Finishing',
'cebo/engorde 90-100kg': 'Finishing',
'lactantes (  3 semanas)': 'Weaning',
'cebo/engorde ( 4-5 meses)': 'Finishing',
'cebo/engorde (2sem )': 'Finishing',
'cebo/engorde (1 dia)': 'Finishing',
'destetado/transicion semana 5': 'Mixed',
'cebo/engorde (45 dias)': 'Nursery',
'lactantes (   2-3 dias)': 'Suckling',
'lactantes (1/2/4)': 'Suckling',
'cebo/engorde (  2 sem  )': 'Finishing',
'1-3 semanas': 'Suckling',
'destetado/transicion sem 5': 'Mixed',
'recria/reposicion (sem. 5)': 'Gilt',
'recria/reposicion (sem. 4)': 'Gilt',
'destetado/transicion 7 w': 'Mixed',
'lactante (1 dias) destetado/transicion (8 sem.)': 'Mixed',
'destetado/transicion (4	 7 y 8 sem.)': 'Mixed',
'destetado/transicion (sem. 6)': 'Mixed',
'recria/reposicion (entrada sem. 5)': 'Gilt',
'cebo/engorde (  3 dias )': 'Finishing',
'adultos recria/reposicion (centinelas)': 'Gilt',
'recria/reposicion	reproductoras/es	cebo/engorde ( sem/mes/ano)': 'Mixed',
'lactantes (3 sem.) destetado/transicion (9 sem.)': 'Mixed',
'lactantes (nm sem. 5)': 'Suckling',
'lactantes (nm sem. 6) cebo/engorde (sem.6)': 'Mixed',
'destetado/transicion (2-3 sem.)': 'Mixed',
'adultos lactantes (nm sem. 3)': 'Mixed',
'destetado/transicion (3-4 sem.)': 'Mixed',
'transicion recria/reposicion': 'Mixed',
'destetado/transicion (35 dias)': 'Mixed',
'destetado/transicion (8 kg)': 'Mixed',
'lactantes ( 3 sem )': 'Weaning',
'lactantes ( 3-4 sem)': 'Weaning',
'destetado/transicion (100 dias)': 'Mixed',
'destetado sem 8': 'Nursery',
'lactante	 destetado/transicion': 'Mixed',
'destetado  lactantes ( 1 dias)': 'Mixed',
'cebo/engorde (  1  sem)': 'Finishing',
'lactantes (3 semanas vida)': 'Weaning',
'lactantes (  2  y 21 dias)': 'Suckling',
'destetado/transicion 9-10 semanas': 'Mixed',
'destetado/transicion  3 sem.': 'Mixed',
'destetado/transicion 2 sem.': 'Mixed',
'destetado/transicion 6 sem': 'Mixed',
'cebo/engorde (30sem)': 'Finishing',
'cebo/engorde (30 sem)': 'Finishing',
'lactantes (0-7 dias)': 'Suckling',
'cebo/engorde (21 dias)': 'Weaning',
'cebo/engorde (34 dias)': 'Nursery',
'cebo/engorde (53 dias)': 'Nursery',
'recria/reposicion 5 meses': 'Gilt',
'recria/reposicion 6 meses': 'Gilt',
'cebo/engorde ( 2  sem/mes/ano)': 'Finishing',
'cebo/engorde ( 2   sem/mes/ano)': 'Finishing',
'reproductoras/es reposicion': 'Gilt',
'destetado/transicion (5 meses)': 'Mixed',
'cebo/engorde (  9 sem)': 'Finishing',
'otrareproductoras/es': 'Mixed',
'reproductoras  recria/reposicion': 'Gilt',
'destetado/transicion 14 dias': 'Mixed',
'destetado/transicion (7 kg)': 'Mixed',
'destetado/transicion (3	 6 y 9 sem.)': 'Mixed',
'cebo/engorde (17 sem.)': 'Finishing',
'adultos cebo/engorde (1	5 meses)': 'Finishing',
'destetado/transicion (30 y 50 dias)': 'Mixed',
'cebo/engorde 45 dias) 56-60 kg': 'Finishing',
'cebo/engorde (44 dias) 40-50 kg': 'Finishing',
'lactantes (nm sem.9)': 'Other',
'lactantes (nm sem.9/10)': 'Other',
'destetado/transicion (2/3 sem.)': 'Mixed',
'lactantes (1-30 dias)': 'Mixed',
'lactantes (0-3 dias)': 'Suckling',
'reproductoras/es lactantes ( 0 dias)': 'Mixed',
'lactantes destetado/transicion (3-4 sem.)': 'Mixed',
'cebo/engorde ( 15 semanas)': 'Finishing',
'destetado/transicion (2 sem destete)': 'Mixed',
'reproductoras/es nuliparas': 'Mixed',
'lactantes (   3 dias)': 'Suckling',
'lactantes (   10 dias)': 'Suckling',
'cebo/engorde (4 sem/mes)': 'Finishing',
'cebo/engorde (30 dias	 40 kg)': 'Finishing',
'destetado/transicion (1-2 sem.)': 'Mixed',
'adultos (cuarentena)': 'Other',
'destetado/transicion 50dias': 'Mixed',
'destetado/transicion 10-18 dias': 'Mixed',
'lactantes (24 dias) recria/reposicion': 'Mixed',
'cebo/engorde ( 0	5 semanas)': 'Finishing',
'destetado/transicion (1	 4 y 7 sem.)': 'Mixed',
'destetado cebo (16 semanas)': 'Mixed',
'destetado/transicion  3 sem': 'Mixed',
'lactantes (  22  dias)': 'Weaning',
'cebo/engorde (3 dias)': 'Finishing',
'lactante (0 dias) reposicion': 'Mixed',
'lactantes ( 15-20 dias)': 'Suckling',
'3 semanas postdestete': 'Nursery',
'cebo/engorde (  1 sem)': 'Finishing',
'lactantes (  0-3 sem)': 'Suckling',
'lactantes ( 0 dias) destetado/transicion  10sem': 'Mixed',
'cebo/engorde (4 sem)': 'Finishing',
'destetado/transicion 0 dias': 'Mixed',
'reproductoras/es	primerizas': 'Mixed',
'cebo/engorde ( 2 sem)': 'Finishing',
'reproductoras/es (1 ano)': 'Other',
'cebo/engorde (90 dias)': 'Finishing',
'20-50 dias destetado /transicion': 'Nursery',
'cebo/engorde (0	5 sem.)': 'Finishing',
'lactantes ( dias)	adultos': 'Mixed',
'lactante (12 dias)': 'Suckling',
'destetado/transicion	 cebo/engorde ( sem/mes/ano)': 'Mixed',
'lactantes ( 21-28   dias)': 'Weaning',
'lactantes (10 dias) cebo/engorde (50 dias)': 'Mixed',
'lactantes (  6  dias)': 'Suckling',
'recria/reposicionreproductoras/es': 'Mixed',
'lact. (<24 dias) , 1 sem, 24 dias': 'Suckling',
'cebo (  4,5  meses)': 'Finishing',
'cebo ( 2,5  meses)': 'Finishing',
'cebo ( 5,5  meses)': 'Finishing',
'cebo (2 meses, 60 kg)': 'Finishing',
'cebo, adultas y lechones': 'Mixed',
'cebo ( 2,3,4 meses)': 'Finishing',
'transicion (2,5 meses)': 'Nursery',
'2,5-3 meses': 'Finishing',
'lact. (1, 2 dias, 4 semanas)': 'Mixed',
'transicion (3, 6 y 9 semanas)': 'Nursery',
'transicion (1,5 meses)': 'Nursery',
'transicion (1,5-2 meses)': 'Nursery',
'lact. ( dias), transicion': 'Mixed',
'cebo (1,5 meses) iberico': 'Finishing',
'lechones, adulto': 'Mixed',
'cebo (3,5 meses de vida)': 'Finishing',
'lact. ( dias),transicion': 'Mixed',
'9, 12, 15 semanas': 'Mixed',
'transicion (entrada, 3 sem y 6 sem.)': 'Nursery',
'lact. (15, 40, 80 dias)': 'Mixed',
'cebo (4-4,5 meses)': 'Finishing',
'lact. ( dias),adultos': 'Mixed',
'cebo (2 meses, 25-30 kg)': 'Finishing',
'adultos  lact. (7 a 10, 17 dias) transicion (50 dias)': 'Mixed',
'transicion (4,6 y 8 semanas)': 'Nursery',
'transicion (1, 3, 5 semanas)': 'Nursery',
'lact. (1, 7 y 10 dias)': 'Suckling',
'transicion (6, 8 y 10 semanas)': 'Nursery',
'lact.transicion (2, 4 y 5 semanas)': 'Mixed',
'transicion (0,3 y 6 semanas)': 'Nursery',
'lact (nm)transicion (3, 6 y 9 semanas)': 'Mixed',
'cebo ( 4,5   meses)': 'Finishing',
'lact. transicion (3,5 meses)': 'Mixed',
'transicion (2, 6 y 9 semanas)': 'Nursery',
'lact., transicion, adultos': 'Mixed',
'cebo ( 2,5   meses)': 'Finishing',
'transicion (3, 6, 9 semanas)': 'Nursery',
'cebo ( 4,5 meses)': 'Finishing',
'lactantes (1 dia) transicion (4,7 y 9 semanas)': 'Mixed',
'transicion (3,6 y 9 semanas)': 'Nursery',
'cebo (4,5  meses)': 'Finishing',
'transicion (4, 6 y 9 semanas)': 'Nursery',
'transicion (6, 9 y 12 semanas)': 'Nursery',
'reproductoras, transicion, cebo': 'Mixed',
'transicion (1, 4 y 6 semanas)': 'Nursery',
'transicion (0, 2, 4 y 6 semanas entrados)': 'Nursery',
'transicion, reproductoras': 'Mixed',
'cebo (  0,5  meses)': 'Finishing',
'lactantes (15 dias, nm)': 'Suckling',
'reposicion, transicion (2 meses)': 'Mixed',
'reposicion,adultos': 'Mixed',
'destetado (3,4 y 5 semanas postdestete)': 'Mixed',
'cebo (2,5 meses vida)': 'Finishing',
'cebo (5,5 meses)': 'Finishing',
'cebo ( meses), reposicion': 'Mixed',
'primerizas,reposicion': 'Mixed',
'lactantes (1, 2 y 25 dias)': 'Suckling',
'transicion (18-24, 60 dias)': 'Mixed',
'transicion (4, 7 y 10 semanas)': 'Mixed',
'transicion (1, 3 y 5 semanas)': 'Mixed',
'lactantes (0-4, 7 y >14 dias)': 'Suckling',
'destetado (4, 7 y 10 semanas)': 'Mixed',
'transicion (4,5 semanas y 30 dias de vida)': 'Mixed',
'destetados (3,5-4,5 semanas de vida)': 'Mixed',
'lactantes,destetado': 'Mixed',
'transicion (3,5 meses)': 'Finishing',
'lactantes, destetado, cebo': 'Mixed',
'lactantes, destetado': 'Mixed',
'transicion,adultos': 'Mixed',
'lactantes (5, 15-20 dias) transicion (17-20 dias)': 'Mixed',
'lactantes, transicion': 'Mixed',
'transicion (3, 9, 10, 11 y 12 semanas) reposicion (14 semanas)': 'Mixed',
'cebo (  1,5  meses)': 'Finishing',
'lactantes (0 dias) transicion (6, 7 y 8 semanas)': 'Mixed',
'transicion (4, 7 y 9 semanas)': 'Mixed',
'transicion 7,5 semanas': 'Nursery',
'transicion 3,5 semanas': 'Mixed',
'lactantes (7-9, 14-16 dias)': 'Suckling',
'cebo (  3,5  meses)': 'Finishing',
'destetado, transicion (4-5 semanas)': 'Mixed',
'lactantes ( dias), transicion': 'Mixed',
'reposicion (4, 5 y 6 meses)': 'Mixed',
'lactantes (2 dias) transicion (4, 6 y 9 semanas) reposicion adultos': 'Mixed',
'transicion, cebo (4, 6, 8 y 9 semanas)': 'Mixed',
'lactantes (1, 2 y 3 semanas)': 'Suckling',
'cebo ( 3,5  meses)': 'Finishing',
'destetado,cebo ( meses)': 'Mixed',
'cebo (  2,5  meses)': 'Finishing',
'lactantes (6, 13 y 21 dias)': 'Suckling',
'lactantes, transicion (3 semanas)': 'Mixed',
'reproductoras, recria (14-15 semanas)': 'Mixed',
'cebo (1,5 meses) vida': 'Finishing',
'nuliparas, adultos': 'Mixed',
'transicion (5, 6 y 9 semanas)': 'Nursery',
'lactantes (7 dias), destetado (3 semanas), transicion (6 semanas)': 'Mixed',
'cebo ( 0,5   meses)': 'Finishing',
'adultos, reproductoras': 'Mixed',
'lactantes (2 semanas), transicion': 'Mixed',
'cebo/engorde (3,5 meses)': 'Finishing',
'cebo/engorde ( 2,5 meses)': 'Finishing',
'cebo/engorde (18, 21 y 24 semanas)': 'Finishing',
'reproductoras/es, destetado/transicion': 'Mixed',
'lactantes ( 2-3,8 dias)': 'Suckling',
'cebo/engorde ( 1,5 meses)': 'Finishing',
'lactantes (nm, semanas 45 y 46)': 'Other',
'lactantes (1 dias, nm)': 'Suckling',
'semanas 45, 46 y 47': 'Finishing',
'destetado/transicion (5, 6 y 7 dias)': 'Mixed',
'reproductoras/es, lactantes, destetado/transicion': 'Mixed',
'lactantes (10 dias), reproductoras/es': 'Mixed',
'reposicion, lactantes (nm)': 'Mixed',
'recria/reposicion, reproductoras/es, lactantes (20 dias)': 'Mixed',
'lactantes, destetado/transicion, cebo/engorde': 'Mixed',
'destetado/transicion (1,5 meses)': 'Mixed',
'cebo/engorde (1,5 meses)': 'Finishing',
'destetado/transicion, cebo/engorde': 'Mixed',
'lactantes, destetado/transicion': 'Mixed',
'cebo/engorde (2,6 meses)': 'Finishing',
'lactantes ( dias),destetado/transicion': 'Mixed',
'lactantes ( 20 dias), destetado/transicion': 'Mixed',
'destetado/transicion (3, 6 y 9 semanas)': 'Mixed',
'nuliparas, multiparas, transicion (28 dias)': 'Mixed',
'nuliparas, multiparas, recria (22 y 25 semanas)': 'Mixed',
'3 dias, 3-6-9 semanas': 'Mixed',
'cebo/engorde (1 semana, 25-30 kg)': 'Finishing',
'lactantes (7, 15, 21 dias)': 'Suckling',
'destetado/transicion (5, 8 y 9 semanas)': 'Mixed',
'destetado/transicion (35 dias, 1 semana de destete)': 'Mixed',
'cebo/engorde (  1,5 meses)': 'Finishing',
'cebo/engorde (2-2,5 meses)': 'Finishing',
'destetado/transicion (1,3 y 4-5 semanas)': 'Mixed',
'cebo/engorde (45 dias de vida, 21 dias destetados)': 'Finishing',
'cebo/engorde (90 dias de vida, 67 dias destetados)': 'Finishing',
'reproductoras/es,destetado/transicion': 'Mixed',
'recria/reposicion,reproductoras/es,destetado/transicion': 'Mixed',
'cebo/engorde ( 4,5 meses)': 'Finishing',
'lactantes,destetado/transicion': 'Mixed',
'cebo/engorde (13, 14 y 15 semanas)': 'Finishing',
'recria/reposicion, lactantes, destetado/transicion': 'Mixed',
'reproductoras/es,': 'Other',
'destetado/transicion (3, 5, 7, 9 y 12 semanas)': 'Mixed',
'cebo/engorde (3 meses,  80 kg.)': 'Finishing',
'recria/reposicion, destetado/transicion': 'Mixed',
'lactantes (1-3 dias) destetado/transicion (3, 6 y 8 semanas)': 'Mixed',
'primiparas, multiparas': 'Sow',
'lactantes ( dias),destetado/transicion,cebo/engorde ( sem/mes/ano)': 'Mixed',
'nuliparas, multiparas': 'Mixed',
'destetado/transicion,primerizas': 'Mixed',
'lactantes, destetado/transicion, adultos': 'Mixed',
'lactantes,destetado/transicion,primerizas,adultos': 'Mixed',
'destetado/transicion (0, 1, 2, 3, 4 y 5 semanas)': 'Mixed',
'cebo/engorde (5,5 meses)': 'Finishing',
'lactantes (nm, semanas 27 y 28)': 'Other',
'lactantes (nm, semana 29), cuarentena': 'Mixed',
'lactantes, adultos': 'Mixed',
'destetado/transicion (2, 4, 6 y 8 semanas)': 'Mixed',
'cebo/engorde (2 meses, 60 kg)': 'Finishing',
'reproductoras/es,destetado/transicion,adultos': 'Mixed',
'lactantes (0 dias, semana 33)': 'Mixed',
'cebo, 22 semanas': 'Finishing',
'lactantes (16, 23 y 25 dias)': 'Mixed',
'destetado/transicion (21, 28 y 35 dias)': 'Mixed',
'destetado/transicion (3, 6 y 9 sem,)': 'Mixed',
'adultos )2,5 anos)': 'Other',
'recria/reposicion,destetado/transicion': 'Mixed',
'cebo/engorde ( 4, 5 meses)': 'Finishing',
'cebo/engorde (5 meses, 80 kg)': 'Finishing',
'cebo/engorde (4,5 meses)': 'Finishing',
'lactantes (  7, 14 y 20 dias)': 'Suckling',
'recria/reposicion,reproductoras/es': 'Mixed',
'primerizas, multiparas': 'Sow',
'cebo/engorde (30, 37, 51 y 65 dias)': 'Finishing',
'destetado/transicion (3,5 y 5 sem.), recria/reposicion (16 sem.), reproductoras/es': 'Mixed',
'lactantes (semanas 6, 7, 9 y 9)': 'Other',
'recria/reposicion,reproductoras/esreproductoras/es': 'Mixed',
'cebo/engorde (9, 13, 17 y 18 sem.)': 'Finishing',
'cebo/engorde (7,5 meses)': 'Finishing',
'lactantes ( 3,5 semanas)': 'Weaning',
'cebo/engorde (1, 3 y 6 sem. post-destete)': 'Finishing',
'lactantes, cebo/engorde (2 meses)': 'Mixed',
'cebo/engorde (3,5-4 meses)': 'Finishing',
'reproductoras/es,lactantes ( dias)': 'Mixed',
'recria/reposicion, lactantes ( dias)': 'Mixed',
'lactantes ( dias), destetado/transicion': 'Mixed',
'destetado/transicion (2,5 meses)': 'Mixed',
'cebo/engorde ( 3,5-4 meses)': 'Finishing',
'destetado/transicion, lactantes (    dias)': 'Mixed',
'reproductoras/es,lactantes ( dias),destetado/transicion': 'Mixed',
'reproductoras/es,destetado/transicion,primerizas': 'Mixed',
'cebo/engorde (1,5 mes)': 'Finishing',
'destetado/transicion (3 sem,)': 'Mixed',
'recria/reposicion,reproductoras/es,lactantes ( dias)': 'Mixed',
'lactantes ( dias),cebo/engorde ( sem/mes/ano)': 'Mixed',
'destetado/transicion (1 sem,)': 'Mixed',
'recria/reposicion,adultos': 'Mixed',
'destetado/transicion 2,5 semanas': 'Mixed',
'destetado/transicion (4 sem.) cebo/engorde (7, 8, 9 y 10 sem.)': 'Mixed',
'lactantes (0 dias, 3 sem.)': 'Mixed',
'destetado/transicion,cebo/engorde ( sem/mes/ano)': 'Mixed',
'destetado/transicion (4, 7 y 8 sem.)': 'Mixed',
'recria/reposicion,reproductoras/es,cebo/engorde ( sem/mes/ano)': 'Mixed',
'lactante, destetado/transicion': 'Mixed',
'destetado/transicion (3, 6 y 9 sem.)': 'Mixed',
'adultos cebo/engorde (1,5 meses)': 'Mixed',
'cebo/engorde (30 dias, 40 kg)': 'Finishing',
'cebo/engorde ( 0,5 semanas)': 'Finishing',
'destetado/transicion (1, 4 y 7 sem.)': 'Mixed',
'reproductoras/es,primerizas': 'Mixed',
'cebo/engorde (0,5 sem.)': 'Finishing',
'lactantes ( dias),adultos': 'Mixed',
'destetado/transicion, cebo/engorde ( sem/mes/ano)': 'Mixed',


}

# Map the 'Edad' column using the English translation mapping
df1['PigStage'] = df1['Age'].map(PigStageMapping).fillna('Missing')
#df1 = df1.dropna(subset=['PigStage'])
df1 = df1.drop(['Age'], axis=1)

In [22]:
print(df1['PigStage'].unique())

['Weaning' 'Finishing' 'Other' 'Missing' 'Mixed' 'Gilt' 'Nursery' 'Sow'
 'Suckling' 'Boar']


In [23]:
df1.head(50)

,SampleNumber,Date,Province,SampleType,Pathogen,Result,DiagnosticTest,Country,Breed,Lab_Reference,PigStage
0,1,2020-01-02,Castellon,Tissue,Porcine reproductive and respiratory syndrome ...,1,PCR,Spain,Meat,5,Weaning
1,1,2020-01-02,Castellon,Tissue,Porcine reproductive and respiratory syndrome ...,0,PCR,Spain,Meat,5,Weaning
2,1,2020-01-02,Castellon,Tissue,Actinobacillus pleuropneumoniae,0,PCR,Spain,Meat,5,Weaning
3,2,2020-01-02,Castellon,Tissue,Porcine reproductive and respiratory syndrome ...,1,PCR,Spain,Meat,5,Finishing
4,2,2020-01-02,Castellon,Tissue,Porcine reproductive and respiratory syndrome ...,0,PCR,Spain,Meat,5,Finishing
5,2,2020-01-02,Castellon,Tissue,Actinobacillus pleuropneumoniae,1,PCR,Spain,Meat,5,Finishing
6,3,2020-01-02,Zaragoza,Tissue,Porcine reproductive and respiratory syndrome ...,1,PCR,Spain,Meat,5,Other
7,3,2020-01-02,Zaragoza,Tissue,Porcine reproductive and respiratory syndrome ...,0,PCR,Spain,Meat,5,Other
8,4,2020-01-02,Huesca,Tissue,Porcine reproductive and respiratory syndrome ...,1,PCR,Spain,Meat,5,Missing
9,4,2020-01-02,Huesca,Tissue,Porcine reproductive and respiratory syndrome ...,0,PCR,Spain,Meat,5,Missing


In [24]:
df1.to_csv('output/PigLab5_2.csv', sep=';', index=False)

### Join both datasets

In [25]:
df_all = pd.concat([df, df1], ignore_index = True)

print(df_all['Pathogen'].unique())

['Swine influenza virus' 'Mycoplasma hyopneumoniae'
 'Lawsonia intracellularis'
 'Porcine reproductive and respiratory syndrome virus'
 'Actinobacillus pleuropneumoniae' 'Porcine epidemic diarrhea virus'
 'Brachyspira hyodysenteriae']


In [26]:
print(df_all['PigStage'].unique())

['Weaning' 'Finishing' 'Missing' 'Gilt' 'Nursery' 'Other' 'Mixed'
 'Suckling' 'Sow' 'Boar']


In [27]:
print(df_all['SampleType'].unique())

['Tissue' 'Fluid' 'Swab' 'Other' 'Blood' 'Faeces' 'Mixed' 'Missing']


In [28]:
df_all.to_csv('output/PigLab5_update20250109.csv', sep=';', index=False)

### Step 02: Create an RDF graph and namespaces.

In [29]:
g = rdflib.Graph()
xsd = Namespace('http://www.w3.org/2001/XMLSchema#')
g.bind('xsd', xsd)

# Define your custom namespace for your ontology's properties
LHO = Namespace("http://www.purl.org/decide/LiveStockHealthOnto/LHO#")
g.bind('LHO', LHO)

SKOS = Namespace("http://www.w3.org/2004/02/skos/core#")
g.bind('skos', SKOS)

decide=  Namespace("http://www.purl.org/decide#")
g.bind('decide', decide)
ncit = Namespace("http://purl.obolibrary.org/obo/NCIT_C25464")
g.bind('ncit', ncit)
agrovoc = Namespace ("http://aims.fao.org/aos/agrovoc")
g.bind('agrovoc', agrovoc)

### Step 03: Iterate over the Panda DataFrame and map to ontology properties:

In [30]:
 # for slugging local names or avoid spaces in names to create URIs along with label names

import re
import urllib.parse

def _safe_local_name(v):
    v = str(v).strip()
    # RFC3986 unreserved: A-Z a-z 0-9 - . _ ~
    v = re.sub(r'[^A-Za-z0-9\-._~]', '_', v)
    v = re.sub(r'_+', '_', v)  # collapse repeated underscores (optional)
    return v

In [31]:
#For Pig Data frame RDF conversion
#for index, row in barometer.head(500).iterrows():
##................................><.........................................................................
##Note: spaces in URIs are not OK. RDFLib (and the RDF specs) require valid URIs; a space makes them invalid. 
##If there is space in names then got errors in the Turtle serialization.
##................................><.........................................................................
for index, row in df_all.iterrows():
    SampleNumber = row["SampleNumber"]
    Date = row["Date"]
    Pathogen = row["Pathogen"]
    Breed = row["Breed"]
    Country = row["Country"]
    Province = row["Province"]
    SampleType = row["SampleType"]
    SampleResult = row["Result"]
    DiagnosticTest = row["DiagnosticTest"]
    PigStage= row["PigStage"]

    
    # Create a unique URI for each sample based on the row index
    PigSample_uri = URIRef(f"http://www.purl.org/decide/LiveStockHealthOnto/LHO#Lab1PigSample_{index}")
    
    # Add sample type assertion
    g.add((PigSample_uri, RDF.type, LHO.PigSample))
    description = "An individual representing a sample from a Pig and Piglets."
    g.add((PigSample_uri, RDFS.comment, Literal(description, lang="en")))

    
    #if "Pathogen" in row and row["Pathogen"]:    # Making little changes to pathogen URI so that it will not create error
        #Pathogen = row["Pathogen"]
        #Pathogen_uri = URIRef(f"http://www.purl.org/decide/LiveStockHealthOnto/LHO#{Pathogen}")
        #g.add((PigSample_uri, LHO.hasPathogen, Pathogen_uri))
        #g.add((Pathogen_uri, RDF.type, decide.Pathogen))
        #description = "An individual representing Pig pathogen."
        #g.add((Pathogen_uri, RDFS.comment, Literal(description, lang="en")))

    if "Pathogen" in row and row["Pathogen"]:
        Pathogen = str(row["Pathogen"]).strip()
        Pathogen_local = _safe_local_name(Pathogen)  # e.g., "Swine_influenza_virus"
        Pathogen_uri = URIRef(f"http://www.purl.org/decide/LiveStockHealthOnto/LHO#{Pathogen_local}")
        g.add((PigSample_uri, LHO.hasPathogen, Pathogen_uri))
        g.add((Pathogen_uri, RDF.type, decide.Pathogen))
        description = "An individual representing Pig pathogen."
        g.add((Pathogen_uri, RDFS.comment, Literal(description, lang="en")))
        g.add((Pathogen_uri, RDFS.label, Literal(Pathogen)))  # human-readable conventional label, e.g., "Swine influenza virus" same thing we can do with other properties like breed and diagnostic test etc. if we have sapce

        
    if "Breed" in row and row["Breed"]:
        Breed = row["Breed"]
        Breed_uri = URIRef(f"http://www.purl.org/decide/LiveStockHealthOnto/LHO#{Breed}")
        g.add((PigSample_uri, LHO.hasBreed, Breed_uri))
        g.add((Breed_uri, RDF.type, LHO.Breed))
        description = "An individual representing a Piglet breed which is Meat."
        g.add((Breed_uri, RDFS.comment, Literal(description, lang="en")))

    if "Lab_Reference" in row and row["Lab_Reference"]:  #Adding that code
        Lab_Reference = row["Lab_Reference"]
        Lab_Reference_uri = URIRef(f"http://www.purl.org/decide/LiveStockHealthOnto/LHO#{Lab_Reference}")
        g.add((PigSample_uri, LHO.hasLab_Reference, Lab_Reference_uri))
        g.add((Lab_Reference_uri, RDF.type, LHO.Lab_Reference))
        description = "An individual representing from which lab refence it belongs."
        g.add((Lab_Reference_uri, RDFS.comment, Literal(description, lang="en")))
        
    if "DiagnosticTest" in row and row["DiagnosticTest"]:
        DiagnosticTest= row["DiagnosticTest"]
        DiagnosticTest_uri = URIRef(f"http://www.purl.org/decide/LiveStockHealthOnto/LHO#{DiagnosticTest}")
        g.add((PigSample_uri, LHO.hasDiagnosticTest, DiagnosticTest_uri))
        g.add((DiagnosticTest_uri, RDF.type, LHO.DiagnosticTest))
    if DiagnosticTest == "PCR":
        description = "An individual representing DNA/RNA amplification for rapid pathogen detection."
    elif SampleType == "Culture":
        description = "An individual representing a Growing live microorganisms for identification."
        g.add((DiagnosticTest_uri, RDFS.comment, Literal(description, lang="en")))
        
    if "Country" in row and row["Country"]:
        Country = row["Country"].strip()  # Use strip() to remove leading and trailing spaces
        # Encode the Country value to create a valid URI
        Country = urllib.parse.quote(Country)
        Country_uri = URIRef(f"http://www.purl.org/decide/LiveStockHealthOnto/LHO#{Country}")
    
        # Map to the specific class URI in your ontology and add the label
        g.add((PigSample_uri, LHO.hasCountry, Country_uri))
        g.add((Country_uri, RDF.type, URIRef("http://purl.obolibrary.org/obo/NCIT_C25464")))  # Use the specific class URI
        g.add((Country_uri, RDFS.label, Literal("Spain")))
        description = "An individual representing different Countries."
        g.add((Country_uri, RDFS.comment, Literal(description, lang="en")))
        
    if "Province" in row and row["Province"]:
        Province = row["Province"].strip()
        Province = urllib.parse.quote(Province)
        Province_uri = URIRef(f"http://www.purl.org/decide/LiveStockHealthOnto/LHO#{Province}")
        g.add((PigSample_uri, LHO.hasProvince, Province_uri))
        g.add((Province_uri, RDF.type, LHO.Province))
        description = "An individual representing different Province."
        g.add((Province_uri, RDFS.comment, Literal(description, lang="en")))
    
    if "PigStage" in row and row["PigStage"]:
        Stage = row["PigStage"]
        Stage_uri = URIRef(f"http://www.purl.org/decide/LiveStockHealthOnto/LHO#{Stage}")
        g.add((PigSample_uri, LHO.hasProductionStages, Stage_uri))
        g.add((Stage_uri, RDF.type, LHO.PigProductionStage))
        description = "An individual representing Production stage ."
        g.add((Stage_uri, RDFS.comment, Literal(description, lang="en")))
    
    if "SampleType" in row and row["SampleType"]:
        SampleType = row["SampleType"]
        SampleType_uri = URIRef(f"http://www.purl.org/decide/LiveStockHealthOnto/LHO#{SampleType}")
        # Add statements for SampleType
        g.add((PigSample_uri, LHO.hasSampleType, SampleType_uri))
        g.add((SampleType_uri, RDF.type, URIRef("http://www.purl.org/decide#SampleType")))
    if SampleType == "Autopsy":
        description = "An individual representing a cattle sample obtained through autopsy."
    elif SampleType == "BAL":
        description = "An individual representing a cattle sample obtained through bronchoalveolar lavage (BAL)."
    elif SampleType == "SWAB":
        description = "An individual representing a cattle sample obtained through swabbing."
    elif SampleType == "Missing":
        description = "An individual representing a missing or unspecified cattle sample type."
        g.add((SampleType_uri, RDFS.comment, Literal(description, lang="en")))
           
    if "Result" in row and row["Result"]:
        SampleResult = row["Result"]
        # Create a unique URI for the Result based on the value
        Result_uri = URIRef(f"http://www.purl.org/decide/LiveStockHealthOnto/LHO#{SampleResult}")
        g.add((PigSample_uri, LHO.hasResult, Result_uri))
        g.add((Result_uri, RDF.type, decide.SampleResult))
    if SampleResult == "1":
        description = "An individual representing a Positive test result"
    elif SampleResult == "0":
        description = "An individual representing a negative test result"
    elif SampleResult == "missing":
        description = "An individual representing an unknown or missing test result."
    g.add((Result_uri, RDFS.comment, Literal(description, lang="en")))


    if "Date" in row and not pd.isnull(row["Date"]):  # Check for NaN or None values
        Date = str(row["Date"]).strip()
    
    # Split the datetime string and take the date part
        Date = Date.split()[0]
    
    # Create a unique URI for the Date based on the value (use the Date directly)
        Date_uri = URIRef(f"http://www.purl.org/decide/LiveStockHealthOnto/LHO#{Date}")
    
    # Add Date assertion
        g.add((PigSample_uri, LHO.hasDate, Date_uri))
        g.add((Date_uri, RDF.type, LHO.Date))
    

# Serialize the RDF graph to a file
rdf_output_file = "output/PigLab1.rdf"
g.serialize(rdf_output_file, format="xml")

# Serialize the RDF graph to Turtle format and print it too see RDF conversion
#turtle_data = g.serialize(format="turtle")
#print(turtle_data)

<Graph identifier=N19a134fdee104162a5e3bb91aa67e823 (<class 'rdflib.graph.Graph'>)>

### Step 4: Load the RDF data and ontology into a Panda DataFrame:

In [32]:
# Parse the ontology file in OWL format and add it to the graph
path_to_ontology = "Ontology/LivestockHealthOntology.owl"
g.parse(path_to_ontology, format="xml")

<Graph identifier=N19a134fdee104162a5e3bb91aa67e823 (<class 'rdflib.graph.Graph'>)>

### Step 05: Query the data from updated ontology

In [33]:
query = """
PREFIX rdf:   <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
PREFIX decide: <http://www.purl.org/decide#>
PREFIX rdfs:  <http://www.w3.org/2000/01/rdf-schema#>
PREFIX LHO:   <http://www.purl.org/decide/LiveStockHealthOnto/LHO#>
PREFIX skos:  <http://www.w3.org/2004/02/skos/core#>
PREFIX xsd:   <http://www.w3.org/2001/XMLSchema#>

SELECT
  (STRAFTER(STR(?Sample), "#") AS ?SampleNumber)
  (STRAFTER(STR(?Province), "#") AS ?Province)
  (?FlooredDate AS ?FlooredDate)
  (COALESCE(STR(?PathogenLabel), REPLACE(STRAFTER(STR(?Pathogen), "#"), "_", " ")) AS ?Pathogen)
  (STRAFTER(STR(?SampleResult), "#") AS ?Result)
  (STRAFTER(STR(?SampleType), "#") AS ?SampleType)
  (STRAFTER(STR(?DiagnosticTest), "#") AS ?DiagnosticTest)
  (STRAFTER(STR(?Country), "#") AS ?Country)
  (STRAFTER(STR(?Breed), "#") AS ?Breed)
  (STRAFTER(STR(?Lab_Reference), "#") AS ?Lab_Reference)
  (STRAFTER(STR(?PigProductionStages), "#") AS ?PigStage)
  (STRAFTER(STR(?AutonomousCommunity), "#") AS ?AutonomousCommunity)
  (?MonthNum AS ?Month)
  (?YearNum  AS ?Year)
WHERE {
  ?Sample rdf:type LHO:PigSample .
  ?Sample LHO:hasDate ?Date .
  ?Sample LHO:hasPathogen ?Pathogen .
  ?Sample LHO:hasBreed ?Breed .
  ?Sample LHO:hasSampleType ?SampleType .
  ?Sample LHO:hasDiagnosticTest ?DiagnosticTest .
  ?Sample LHO:hasProductionStages ?PigProductionStages .
  ?Sample LHO:hasProvince ?Province .
  ?Sample LHO:hasCountry ?Country .
  ?Sample LHO:hasResult ?SampleResult .
  ?Sample LHO:hasLab_Reference ?Lab_Reference .
  OPTIONAL { ?Sample LHO:hasAutonomousCommunity ?AutonomousCommunity . }

  # Pathogen label only (prefer rdfs:label/skos:prefLabel; prefer en or no-lang)
  OPTIONAL {
    ?Pathogen (rdfs:label|skos:prefLabel) ?PathogenLabel .
    FILTER(LANG(?PathogenLabel) = "" || LANGMATCHES(LANG(?PathogenLabel), "en"))
  }

  # ---- Date parsing (supports DD/MM/YYYY and YYYY-MM-DD) ----
  BIND( STRAFTER(STR(?Date), "#") AS ?DateStr )
  BIND( ?DateStr AS ?FlooredDate )

  # month
  BIND(
    xsd:integer(
      IF(CONTAINS(?DateStr, "/"),
         SUBSTR(?DateStr, 4, 2),   # DD/MM/YYYY -> MM is positions 4-5
         SUBSTR(?DateStr, 6, 2)    # YYYY-MM-DD -> MM is positions 6-7
      )
    ) AS ?MonthNum
  )

  # year
  BIND(
    xsd:integer(
      IF(CONTAINS(?DateStr, "/"),
         SUBSTR(?DateStr, 7, 4),   # DD/MM/YYYY -> YYYY starts at 7
         SUBSTR(?DateStr, 1, 4)    # YYYY-MM-DD -> YYYY is positions 1-4
      )
    ) AS ?YearNum
  )
}
"""
results = g.query(query)

data = []
for row in results:
    data.append(list(row))

df = pd.DataFrame(data, columns=[
    "SampleNumber","Province","FlooredDate","Pathogen","Result","SampleType",
    "DiagnosticTest","Country","Breed","LabReference","PigStage",
    "AutonomousCommunity","Month","Year"
])

# Display the dataframe
df

,SampleNumber,Province,FlooredDate,Pathogen,Result,SampleType,DiagnosticTest,Country,Breed,LabReference,PigStage,AutonomousCommunity,Month,Year
0,Lab1PigSample_0,Castellon,2020-01-02,Swine influenza virus,0,Tissue,PCR,Spain,Meat,5,Weaning,None,1,2020
1,Lab1PigSample_1,Castellon,2020-01-02,Mycoplasma hyopneumoniae,0,Tissue,PCR,Spain,Meat,5,Weaning,None,1,2020
2,Lab1PigSample_2,Castellon,2020-01-02,Swine influenza virus,0,Tissue,PCR,Spain,Meat,5,Finishing,None,1,2020
3,Lab1PigSample_3,Castellon,2020-01-02,Mycoplasma hyopneumoniae,1,Tissue,PCR,Spain,Meat,5,Finishing,None,1,2020
4,Lab1PigSample_4,Huesca,2020-01-02,Swine influenza virus,0,Tissue,PCR,Spain,Meat,5,Missing,None,1,2020
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
220825,Lab1PigSample_220825,Teruel,2025-04-11,Porcine reproductive and respiratory syndrome ...,0,Fluid,PCR,Spain,Meat,5,Gilt,None,4,2025
220826,Lab1PigSample_220826,Teruel,2025-04-11,Porcine reproductive and respiratory syndrome ...,0,Fluid,PCR,Spain,Meat,5,Gilt,None,4,2025
220827,Lab1PigSample_220827,Teruel,2025-04-11,Porcine reproductive and respiratory syndrome ...,0,Fluid,PCR,Spain,Meat,5,Gilt,None,4,2025
220828,Lab1PigSample_220828,Teruel,2025-04-11,Porcine reproductive and respiratory syndrome ...,0,Fluid,PCR,Spain,Meat,5,Gilt,None,4,2025


In [34]:
# Save Results: save to CSV
import os
os.makedirs("output", exist_ok=True)

df.to_csv("output/PigLab5_query_results.csv", index=False, encoding="utf-8")
print("Saved: output/PigLab5_query_results_baromterepig.csv")

Saved: output/PigLab5_query_results_baromterepig.csv
